# DuelNetTradingSystem2026
DualNetTradingSystem2025.ipynbよりコピー

## What's new (backlog)
- トレーニングやトレードを開始する前に、一定期間「空トレード」を行いNNに記憶させる
- 「空トレード」の期間は[0,1,2,4,12]週間とする
- BRONZモデルよりも低いモデル（TEST)を作成作成する
- 同じ通貨ペア×時間足では最も成績(eval_value)の良いものだけを残すようにする（成績の悪いものは削除orVOIDフォルダへ）
- pandasのエラーに対応する
- 古い世代の.iniファイル.pthファイルはいらない？


# Package-Environment
- 価格や損益の情報を提供するパッケージ。以下の機能を実装する
 - 価格CSVファイルを結合して1つのpandas dataframeにする

## Dependency

In [58]:
#@title Dependency
import pandas as pd
import numpy as np
import datetime
import glob
import configparser
import math
import sys
# on Stab
from collections import namedtuple
from scipy import stats
import os
import shutil
import csv
import requests
import gc
from sqlalchemy import create_engine, text

## クラス変数
- SYMBOL_5DIGITS：小数点以下が5桁の通貨ペア(EURUSD等)の配列
- SYMBOL_4DIGITS：小数点以下が4桁の通貨ペア(USDZAR等)の配列
- SYMBOL_3DIGITS：小数点以下が3桁の通貨ペア(USDJPY等)の配列
- DIGIT_MAGNIFICATION：通貨ペアごとの倍率辞書(5桁：100倍、4桁10倍、3桁1倍)

In [59]:
# @title Class Valiables

EXEC_ENV = "LOCAL" #@param ["COLABO", "LOCAL"]

#
inifile = configparser.ConfigParser()

# .iniファイルのエンコード
ENC = None
if EXEC_ENV == 'COLABO':
  inifile.read('./drive/MyDrive/Colab Notebooks/06_DuelNetTradingSystem2025/settings.ini',ENC)
else:
  ENC = 'UTF-8'
  inifile.read(r'D:\ColabNotebooks\06_DuelNetTradingSystem2025\settings.ini',ENC)


# ログ出力セッティングファイル
LOGGING_INIFILE_PATH = inifile.get(EXEC_ENV, 'LOGGING_INIFILE_PATH')

# PRICEDATA_PATH = inifile.get('COLABO', 'PRICEDATA_PATH_COLABO')
PRICEDATA_PATH = None
PRICEDATA_PATH_REAL = inifile.get(EXEC_ENV, 'PRICEDATA_PATH_REAL')
PRICEDATA_PATH_DEMO = inifile.get(EXEC_ENV, 'PRICEDATA_PATH_DEMO')
# TRAIN_MODEL_PATH = inifile.get('COLABO', 'TRAIN_MODEL_PATH_COLABO')
TRAIN_MODEL_PATH = None
TRAIN_MODEL_PATH_VOID_REAL = inifile.get(EXEC_ENV, 'TRAIN_MODEL_PATH_VOID_REAL')
TRAIN_MODEL_PATH_VOID_DEMO = inifile.get(EXEC_ENV, 'TRAIN_MODEL_PATH_VOID_DEMO')
# TRAIN_MODEL_PATH_VOID = inifile.get('COLABO', 'TRAIN_MODEL_PATH_COLABO')
TRAIN_MODEL_PATH_VOID = None
TRAIN_MODEL_PATH_REAL = inifile.get(EXEC_ENV, 'TRAIN_MODEL_PATH_REAL')
TRAIN_MODEL_PATH_DEMO = inifile.get(EXEC_ENV, 'TRAIN_MODEL_PATH_DEMO')
# TMP_TRAIN_MODEL_PATH = inifile.get('COLABO', 'TMP_TRAIN_MODEL_PARTH_COLABO')
TMP_TRAIN_MODEL_PATH = None
TMP_TRAIN_MODEL_PATH_REAL = inifile.get(EXEC_ENV, 'TMP_TRAIN_MODEL_PATH_REAL')
TMP_TRAIN_MODEL_PATH_DEMO = inifile.get(EXEC_ENV, 'TMP_TRAIN_MODEL_PATH_DEMO')
# TRADE_MODEL_PATH = inifile.get('COLABO', 'TRADE_MODEL_PATH_COLABO')
TRADE_MODEL_PATH = None
TRADE_MODEL_PATH_REAL = inifile.get(EXEC_ENV, 'TRADE_MODEL_PATH_REAL')
TRADE_MODEL_PATH_DEMO = inifile.get(EXEC_ENV, 'TRADE_MODEL_PATH_DEMO')
# TRADE_RESULT_PATH
TRADE_RESULT_PATH = None
TRADE_RESULT_PATH_REAL = inifile.get(EXEC_ENV, 'TRADE_MODEL_PATH_REAL')
TRADE_RESULT_PATH_DEMO = inifile.get(EXEC_ENV, 'TRADE_RESULT_PATH_DEMO')
# TRAIN_RESULT_PATH
TRAIN_RESULT_PATH = None
TRAIN_RESULT_PATH_REAL = inifile.get(EXEC_ENV, 'TRAIN_RESULT_PATH_REAL')
TRAIN_RESULT_PATH_DEMO = inifile.get(EXEC_ENV, 'TRAIN_RESULT_PATH_DEMO')
# TRAIN_RESULT_PATH_VOID
TRAIN_RESULT_PATH_VOID = None
TRAIN_RESULT_PATH_VOID_REAL = inifile.get(EXEC_ENV, 'TRAIN_RESULT_PATH_VOID_REAL')
TRAIN_RESULT_PATH_VOID_DEMO = inifile.get(EXEC_ENV, 'TRAIN_RESULT_PATH_VOID_DEMO')
# TMP_TRAIN_RESULT_PATH
TMP_TRAIN_RESULT_PATH = None
TMP_TRAIN_RESULT_PATH_REAL = inifile.get(EXEC_ENV, 'TMP_TRAIN_RESULT_PATH_REAL')
TMP_TRAIN_RESULT_PATH_DEMO = inifile.get(EXEC_ENV, 'TMP_TRAIN_RESULT_PATH_DEMO')

BACKTEST_RESULT_ALL_FILE_NAME = inifile.get('COMMOM', 'BACKTEST_RESULT_ALL_FILE_NAME')
BACKTEST_ALL_FILE_NAME = inifile.get('COMMOM', 'BACKTEST_ALL_FILE_NAME')
MODEL_PORTFOLIO_FILE_NAME = inifile.get('COMMOM', 'MODEL_PORTFOLIO_FILE_NAME')

SYMBOL_5DIGITS = eval(inifile.get('COMMOM', 'SYMBOL_5DIGITS'))
SYMBOL_4DIGITS = eval(inifile.get('COMMOM', 'SYMBOL_4DIGITS'))
SYMBOL_3DIGITS = eval(inifile.get('COMMOM', 'SYMBOL_3DIGITS'))

DIGIT_MAGNIFICATION = eval(inifile.get('COMMOM', 'DIGIT_MAGNIFICATION'))
TICKVOL_MAGNIFICATION = int(inifile.get('COMMOM', 'TICKVOL_MAGNIFICATION'))

# TICKVOL_PERIOD,LONG_PERIOD,SHORT_PERIODはPraceDataを初期化する都度
# ランダムに取得することも可能とする。
TICKVOL_PERIOD = int(inifile.get('COMMOM', 'TICKVOL_PERIOD'))
LONG_PERIOD  = int(inifile.get('COMMOM', 'LONG_PERIOD'))
SHORT_PERIOD  = int(inifile.get('COMMOM', 'SHORT_PERIOD'))

PERIOD_FREQ_DICT = eval(inifile.get('COMMOM', 'PERIOD_FREQ_DICT'))
# 2023/5/27 マージンはスプレッドを反映させる
MARGIN_DICT = eval(inifile.get('COMMOM', 'MARGIN_DICT'))

TP_LC_MIN_LEVEL = int(inifile.get('COMMOM', 'TP_LC_MIN_LEVEL'))
TP_LC_MAX_LEVEL = int(inifile.get('COMMOM', 'TP_LC_MAX_LEVEL'))
TP_LC_STEP = int(inifile.get('COMMOM', 'TP_LC_STEP'))

# LINE Notifyを利用するためのトークン
# 2025/3/31 Line Notify 終了に伴い、Line Message APIに切り替える
#LINE_NOTIFY_TOKEN = inifile.get('COMMOM', 'LINE_NOTIFY_TOKEN')
LINE_MESSAGING_API_TOKEN = inifile.get('COMMOM', 'LINE_MESSAGING_API_TOKEN')

# TradeSystemの名称
# TRADE_SYSTEM = 'DuelNet_2026'
TRADE_SYSTEM = inifile.get('COMMOM', 'TRADE_SYSTEM')

GAMMA = 0.99

# トレードモード(MT5より借用)
ACCOUNT_TRADE_MODE_DEMO = 0
ACCOUNT_TRADE_MODE_CONTEST = 1
ACCOUNT_TRADE_MODE_REAL = 2
# MT5実行ファイルの場所
MT5_PATH = None
MT5_REAL_PATH = inifile.get('LOCAL', 'mt5_real_path')
MT5_DEMO_PATH = inifile.get('LOCAL', 'mt5_demo_path')
# ask-bidの差の閾値
REAL_SPREAD_LIMIT_LIST = eval(inifile.get('LOCAL', 'REAL_SPREAD_LIMIT_LIST'))
# RealSpreadが閾値を超えていた場合の再試行回数
MAX_RETRY_NUM = eval(inifile.get('LOCAL', 'MAX_RETRY_NUM'))

# action index に対応した行動
# output_data(行動A[seq,6])
# a0:何もしない
# a1:LongEntry
# a2:ShortEntry
# a3:PositionClose
# a4:PositionClose&LongEntry
# a5:PositionClose&ShortEntry
no_action = 0
long_entry = 1
short_entry = 2
position_close = 3
close_and_long = 4
close_and_short = 5

# トレーニングの詳細を記録するファイル名
TRAIN_DETAIL_INPUT_FILE_NAME = 'TrainDetailInput'
TRAIN_DETAIL_HIDDEN_FILE_NAME = 'TrainDetailHidden'
TRAIN_DETAIL_CELLSTATE_FILE_NAME = 'TrainDetailCellState'
TRAIN_DETAIL_OUTPUT_FILE_NAME = 'TrainDetailOutput'
TRAIN_DETAIL_MASKOUTPUT_FILE_NAME = 'TrainDetailMaskOutput'

# .set ファイルに書きこむNNモデルの素子数
MODEL_INPUT_NUM = 0
MODEL_HIDDEN_NUM = 0
MODEL_OUTPUT_NUM = 0

# 接続先DB情報
RDBMS='postgresql'
HOST='localhost'
DBNAME=None
DBNAME_DEMO='metaquotesdemo'
DBNAME_REAL='xmtradingreal'
PORT='5432'
USER='appop'
PASSWORD='appop'
SCHEMA='public'
ENGINE=None


## Logger
ロギングを設定する。ログレベルは以下
1. CRITICAL
1. ERROR
1. WARNING
1. INFO
1. DEBUG

Environment package では、ログ空間を"DRL.Environment"とする

In [60]:
#@title Logger
import logging
import logging.config

# logging.config.fileConfig('./drive/MyDrive/Colab Notebooks/05_DuelNetTradingSystem2024/Logging.ini')
logconfigfile = configparser.ConfigParser()
logconfigfile.read(LOGGING_INIFILE_PATH,ENC)
logging.config.fileConfig(logconfigfile)
logger = logging.getLogger('DRLLogging')
logger.debug('Debug level massage.')
logger.info('Info level massage.')
logger.warning('Warning level massage.')
logger.error('Error level massage.')
logger.critical('Critical level massage.')

2026-02-15 15:44:32,224 DRLLogging   INFO     Info level massage.
2026-02-15 15:44:32,225 DRLLogging   WARNING  Warning level massage.
2026-02-15 15:44:32,226 DRLLogging   ERROR    Error level massage.
2026-02-15 15:44:32,227 DRLLogging   CRITICAL Critical level massage.


## クラスEnvironmentCommon
Environmentパッケージで使用するクラス関数を定義する
### クラス関数 GetDigitMagnification()
- 通貨ペアの最少桁数によって値を何倍にするかを返す。1,000pips=1.000となるように調整する
  - 100倍：小数5桁(EURUSD等)
  - 10倍：小数4桁(USDZAR等)
  - 1倍：小数3桁(USDJPY等)

### クラス関数getAvailableAction(self,(plofit_loss, long_position, short_position),CountDown)
現在のポジションの状況から、可能な行動をマスク配列で返す。<br>

#### output_data(行動A[seq,6])
a0:何もしない<br>
a1:LongEntry<br>
a2:ShortEntry<br>
a3:PositionClose<br>
a4:PositionClose&LongEntry<br>
a5:PositionClose&ShortEntry<br>
※両建てはしない。ドテンを想定する。

<b>取引可能時間帯：COUNTDOWN < 1 </b>

<li>パターン1：ポジションなし(position = 0)<br>
  新規ポジションエントリー(long,shot)と何もしないが選択可能。<br>
 action_mask[0] = [a0=True, a1=True, a2=True, a3=False, a4=False, a5=False]<br>
<li>パターン2：ロングポジション(position = 1)<br>
  ポジションクローズと、クローズ＆追加エントリ、何もしないが選択可能。<br>
 action_mask[1] = [a0=True, a1=False, a2=False, a3=True, a4=True, a5=True]<br>
<li>パターン3：ショートポジション(position = -1)<br>
  ポジションクローズと、クローズ＆追加エントリ、何もしないが選択可能。<br>
 action_mask[2] =[a0=True, a1=False, a2=False, a3=True, a4=True, a5=True]<br>
<li>パターン4：両建て(定義なし)<br>
  ポジションクローズ(全決済)が選択可能。両建ては禁止とするため、このパターンにはならない。<br>
 action_mask[3] =[a0=False, a1=False, a2=False, a3=True, a4=False, a5=False]<br>

  <b>取引可能時間外：COUNTDOWN \>= 1 </b>
<li>パターン5：ポジションなし(position = 0)<br>
  何もしないが選択可能。<br>
 action_mask[4] = [a0=True, a1=False, a2=False, a3=False, a4=False, a5=False]<br>
<li>パターン6：ロングポジション(position = 1)<br>
  ポジションクローズが選択可能。<br>
 action_mask[5] = [a0=False, a1=False, a2=False, a3=True, a4=False, a5=False]<br>
<li>パターン7：ショートポジション(position = -1)<br>
  ポジションクローズが選択可能。<br>
 action_mask[6] = [a0=False, a1=False, a2=False, a3=True, a4=False, a5=False]<br>
<li>パターン8：両建て(定義なし)<br>
  ポジションクローズ(全決済)が選択可能。両建ては禁止とするため、このパターンにはならない。<br>
 action_mask[7] =[a0=False, a1=False, a2=False, a3=True, a4=False, a5=False]<br>



In [61]:
#@title class EnvironmentCommon
class EnvironmentCommon:
  @classmethod
  def GetDigitMagnification(cls,symbol):
    # EURUSDのような下5桁通貨は100倍する
    if(symbol in SYMBOL_5DIGITS):
      return DIGIT_MAGNIFICATION['SYMBOL_5DIGITS']
    # USDJPYのような下3桁通貨は1倍する
    elif(symbol in SYMBOL_3DIGITS):
      return DIGIT_MAGNIFICATION['SYMBOL_3DIGITS']
    # USDZARのような下4桁通貨は10倍する
    elif(symbol in SYMBOL_4DIGITS):
      return DIGIT_MAGNIFICATION['SYMBOL_4DIGITS']
    else:
      sys.exit('Symbol is not defined.')

  @classmethod
  def SetLimitStopLevel(cls,period):
    # Losscut,Takeprofitのレベルを時間足に対応して決める
    # 基準はH1の時に1.0、D1の時に2.0となるようにする
    # y = (1/23)x + 22/23
    if(period == 'M1'):
      period_value = (1/60)
    elif(period == 'M5'):
      period_value = (5/60)
    elif(period == 'M15'):
      period_value = (15/60)
    elif(period == 'M30'):
      period_value = (30/60)
    elif(period == 'H1'):
      period_value = 1
    elif(period == 'H4'):
      period_value = 4
    elif(period == 'H6'):
      period_value = 6
    elif(period == 'H8'):
      period_value = 8
    elif(period == 'H12'):
      period_value = 12
    elif(period == 'D1'):
      period_value = 24
    else:
      logger.warning('■■Period is not Implemented')
      # period_value = 1
      sys.exit()

    losscut_level =  (1/23)*period_value + (22/23)
    logger.debug('## Losscut Level: %.2f' %(losscut_level))
    return losscut_level

  @classmethod
  def GetPeriodTimeDelta(cls,period):
    if(period == 'M1'):
      delta_period = datetime.timedelta(minutes=1)
    elif(period == 'M5'):
      delta_period = datetime.timedelta(minutes=5)
    elif(period == 'M15'):
      delta_period = datetime.timedelta(minutes=15)
    elif(period == 'M30'):
      delta_period = datetime.timedelta(minutes=30)
    elif(period == 'H1'):
      delta_period = datetime.timedelta(hours=1)
    elif(period == 'H4'):
      delta_period = datetime.timedelta(hours=4)
    elif(period == 'H6'):
      delta_period = datetime.timedelta(hours=6)
    elif(period == 'H8'):
      delta_period = datetime.timedelta(hours=8)
    elif(period == 'H12'):
      delta_period = datetime.timedelta(hours=12)
    elif(period == 'D1'):
      delta_period = datetime.timedelta(days=1)
    else:
      logger.warning('■■Period Delta is not Implemented')
      # delta_period = datetime.timedelta(hours=1)
      sys.exit()

    return delta_period

  # longとshortで分ける
  @classmethod
  # def getAvailableAction(self,pl_, pos_, cd_):
  def getAvailableAction(self,pl_, lg_, st_, cd_):
    action_mask = np.empty(6, dtype='bool')
    # logger.debug('EnvironmentCommon.getAvailableAction:plofit_loss=%.3f position=%d countwdown=%.3f' %(pl_,pos_,cd_))
    logger.debug('EnvironmentCommon.getAvailableAction:plofit_loss=%.3f long_pos=%.1f shrot_pos=%.1f countwdown=%.3f' %(pl_,lg_,st_,cd_))

    #現在の状況によって、選択できる行動を制限する。詳細は上の説明参照。
    # 今回は両建てを想定しないため、パターン4,8は定義しない
    # 取引時間内 (CountDown < 1)
    if(cd_ < 1):
      # パターン2：ロングポジション(position = 1)
      # if(pos_ == 1):
      if(lg_ > 0.0 and st_ == 0.0):
        action_mask = [True, False, False, True, True, True]
        logger.debug('EnvironmentCommon.getAvailableAction:pattern 2 %s' %(action_mask))
      # パターン3：ショートポジション(position = -1)
      # elif(pos_ == -1):
      elif(st_ > 0.0 and lg_ == 0.0):
        action_mask = [True, False, False, True, True, True]
        logger.debug('EnvironmentCommon.getAvailableAction:pattern 3 %s' %(action_mask))
      # パターン1：ポジションなし(position = 0)
      # elif(pos_ == 0):
      elif(lg_ == 0 and st_ == 0.0):
        action_mask = [True, True, True, False, False, False]
        logger.debug('EnvironmentCommon.getAvailableAction:pattern 1 %s' %(action_mask))
      # エラー：ポジションクローズ
      else:
        action_mask = [False, False, False, True, False, False]
        logger.waring('EnvironmentCommon.getAvailableAction:pattern 4 error %s' %(action_mask))
        # for debug エラーとする
        sys.exit()
    # 取引時間外 (CountDown >= 1)
    else:
      # パターン6：ロングポジション(position = 1)
      # if(pos_ == 1):
      if(lg_ > 0.0 and st_ == 0.0):
        action_mask = [False, False, False, True, False, False]
        logger.debug('EnvironmentCommon.getAvailableAction:patten 6 %s' %(action_mask))
      # パターン7：ショートポジション(position = -1)
      # elif(pos_ == -1):
      elif(st_ > 0.0 and lg_ == 0.0):
        action_mask = [False, False, False, True, False, False]
        logger.debug('EnvironmentCommon.getAvailableAction:patten 7 %s' %(action_mask))
      # パターン5：ポジションなし(position = 0)
      # elif(pos_ == 0):
      elif(lg_ == 0 and st_ == 0.0):
        action_mask = [True, False, False, False, False, False]
        logger.debug('EnvironmentCommon.getAvailableAction:patten 5 %s' %(action_mask))
      # エラー：ポジションクローズ
      else:
        action_mask = [False, False, False, True, False, False]
        logger.waring('EnvironmentCommon.getAvailableAction:pattern 8 error %s' %(action_mask))
        # for debug エラーとする
        sys.exit()

    return action_mask

  @classmethod
  def periodRandomSelect(cls):
    # 各Periodの値をランダムに選択する
    prime_number = [5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97,101,103,107,109,113,127,131,137,139,149,151,157,163,167,173,179,181,191]
    period_numbers = random.sample(prime_number,2)
    long_period = max(period_numbers)
    short_period = min(period_numbers)
    tickvol_period = random.choice(prime_number)

    return (long_period,short_period,tickvol_period)

  @classmethod
  def setTradeModePath(cls, trdmd):
    # トレードモードによってパスを変える
    # global PRICEDATA_PATH,TRAIN_MODEL_PATH,TMP_TRAIN_MODEL_PATH,TRADE_MODEL_PATH,TRADE_RESULT_PATH,TRAIN_RESULT_PATH,TMP_TRAIN_RESULT_PATH,MT5_PATH
    global PRICEDATA_PATH,TRAIN_MODEL_PATH,TMP_TRAIN_MODEL_PATH,TRADE_MODEL_PATH,TRADE_RESULT_PATH,TRAIN_RESULT_PATH,TMP_TRAIN_RESULT_PATH,DBNAME
    if trdmd == ACCOUNT_TRADE_MODE_REAL:
      PRICEDATA_PATH = PRICEDATA_PATH_REAL
      TRAIN_MODEL_PATH = TRAIN_MODEL_PATH_REAL
      TRAIN_MODEL_PATH_VOID = TRAIN_MODEL_PATH_VOID_REAL
      TMP_TRAIN_MODEL_PATH = TMP_TRAIN_MODEL_PATH_REAL
      TRADE_MODEL_PATH = TRADE_MODEL_PATH_REAL
      TRADE_RESULT_PATH = TRADE_RESULT_PATH_REAL
      TRAIN_RESULT_PATH = TRAIN_RESULT_PATH_REAL
      TRAIN_RESULT_PATH_VOID = TRAIN_RESULT_PATH_VOID_REAL
      TMP_TRAIN_RESULT_PATH = TMP_TRAIN_RESULT_PATH_REAL
      DBNAME = DBNAME_REAL

    else:
      PRICEDATA_PATH = PRICEDATA_PATH_DEMO
      TRAIN_MODEL_PATH = TRAIN_MODEL_PATH_DEMO
      TRAIN_MODEL_PATH_VOID = TRAIN_MODEL_PATH_VOID_DEMO
      TMP_TRAIN_MODEL_PATH = TMP_TRAIN_MODEL_PATH_DEMO
      TRADE_MODEL_PATH = TRADE_MODEL_PATH_DEMO
      TRADE_RESULT_PATH = TRADE_RESULT_PATH_DEMO
      TRAIN_RESULT_PATH = TRAIN_RESULT_PATH_DEMO
      TRAIN_RESULT_PATH_VOID = TRAIN_RESULT_PATH_VOID_DEMO
      TMP_TRAIN_RESULT_PATH = TMP_TRAIN_RESULT_PATH_DEMO
      DBNAME = DBNAME_DEMO

  @classmethod
  def send_line_notify(cls,notification_message):
    headers = {"Content_Type": "application/json","Authorization": "Bearer " + LINE_MESSAGING_API_TOKEN}
    requests.post("https://api.line.me/v2/bot/message/broadcast",headers=headers,json={"messages": [{"type": "text","text": notification_message}]}).json()

  @classmethod
  def getRealSpreadLimit(cls,sbl,flg=False):
      if flg:
          return REAL_SPREAD_LIMIT_LIST[sbl]
      else:
          return 1.0

  @classmethod
  def connectDB(csl):
      global ENGINE
      ENGINE = create_engine(RDBMS+"://"+USER+":"+PASSWORD+"@"+HOST+"/"+DBNAME)

  @classmethod
  def compareEvalValue(cls,sbl,tf,eval_value):
      # 新しく取得したeval_valueと比較して、新しいほうの値が良ければ、既存のファイルをvoidファイルに移動する
      # 戻り値：true→新規作成　false→変更なし
      # 同じ通貨ペアと時間足のファイルAgentファイルがあるかを確認する
      # フォルダにある同じ通貨ペア、時間足のファイルを取得する
      filelist = glob.glob(TRAIN_MODEL_PATH+'Agent_'+sbl+'_'+tf+'_*.ini')

      # 既存ファイルのほうが結果が良い場合は1位のファイル名が入る
      win_file=None

      for l in filelist:
          logger.debug('EnvironmentCommon.compareEvalValue:AgentFileName:%s' %(l))
          # AgentFileの内容を読みだしてeval_valueの値を取り出す
          agentfile = configparser.ConfigParser()
          agentfile.read(l,ENC)
          # eval_valueを取得する
          ev=float(agentfile.get('COMMOM', 'EVAL_VALUE'))
          logger.info('EnvironmentCommon.compareEvalValue:Evalvale:%s[%s]' %(str(ev),l))

          #新しいeval_valueと既存のevを比較する
          if ev >= eval_value:
              # 既存のほうが大きい場合は残すファイル名とeval_valueを更新する
              eval_value=ev
              win_file=l

      # TRAIN_MODEL_PATH内のファイルリスト
      tm_filelist = glob.glob(TRAIN_MODEL_PATH+'*_'+sbl+'_'+tf+'_*')
      # TRAIN_RESULT_PATH内のファイルリスト
      #   BacktestResult_CHFJPY_D1_B_20240721000428.csv,Result_CHFJPY_D1_B_20240721000428.csv,TrainResult_CHFJPY_D1_B_20240721000428.csv
      tr_filelist = glob.glob(TRAIN_RESULT_PATH+'*_'+sbl+'_'+tf+'_*')

      # Train_Model_Voidディレクトリが存在していないときはディレクトリを作成する
      if not os.path.isdir(TRAIN_MODEL_PATH_VOID):
          logger.info('EnvironmentCommon.compareEvalValue:Make Train_Model_Void Directry.')
          os.makedirs(TRAIN_MODEL_PATH_VOID)
      # Train_Result_Voidディレクトリが存在していないときはディレクトリを作成する
      if not os.path.isdir(TRAIN_RESULT_PATH_VOID):
          logger.info('EnvironmentCommon.compareEvalValue:Make Train_Result_Void Directry.')
          os.makedirs(TRAIN_RESULT_PATH_VOID)

      # win_fileがNoneのであれば、既存ファイルをすべてvoidへ移して、新規ファイルを作成する
      # win_fileに名前があれば、そのファイル以外はvoidへ移す
      if win_file is None:
          for l in tm_filelist:
              # voidへ移す。
              new_path = shutil.move(l, TRAIN_MODEL_PATH_VOID)
              logger.info('EnvironmentCommon.compareEvalValue:%s is moved to %s'%(l,new_path))
          
          for l in tr_filelist:
              # voidへ移す。
              new_path = shutil.move(l, TRAIN_RESULT_PATH_VOID)
              logger.info('EnvironmentCommon.compareEvalValue:%s is moved to %s'%(l,new_path))
          logger.info('EnvironmentCommon.compareEvalValue:AccountFile Need Update')
          return True
      else:
          #win_file以外をvoidへ移す
          logger.debug('EnvironmentCommon.compareEvalValue:WinFile:%s' %(win_file))
          # D:\ColabNotebooks\06_DuelNetTradingSystem2025\Demo\Model\Train\Agent_USDJPY_D1_B_20260125225640.ini
          MAGIC = win_file.split('\\')[-1].split('_')[4].split('.')[0] 
          logger.debug('EnvironmentCommon.compareEvalValue:MAGIC:%s' %(MAGIC))
          for l in tm_filelist:
              logger.debug('EnvironmentCommon.compareEvalValue:File:%s' %(l))
              # MAGICが同じファイルは移動しない
              if MAGIC not in l:
                  new_path = shutil.move(l, TRAIN_MODEL_PATH_VOID)
                  logger.info('EnvironmentCommon.compareEvalValue:%s is moved to %s' %(l,new_path))
          for l in tr_filelist:
              logger.debug('EnvironmentCommon.compareEvalValue:File:%s' %(l))
              # MAGICが同じファイルは移動しない
              if MAGIC not in l:
                  new_path = shutil.move(l, TRAIN_RESULT_PATH_VOID)
                  logger.info('EnvironmentCommon.compareEvalValue:%s is moved to %s' %(l,new_path))
          logger.info('EnvironmentCommon.compareEvalValue:AccountFile NOT Need Update')
          return False

## クラス PriceData
価格データを取り扱うクラス
### コンストラクタ [PriceData()](#init)
- オブジェクト化する通貨ペアと時間足を受け取り、対象となるCSVファイルをPandas Dataframeに変換する
 - 引数：
   - sbl=通貨ペア('EURUSD'など)
   - prd=時間足('M1','H1'など)
 - 戻り値：
 - 入力：
 - 出力：

### オブジェクト関数 [ConvertPriceDataFileToDataFrame()](#method1)
- Google Driveにある価格データをPandasDataframe(price_list)に通貨ペア、時間足ごとに変換する。csvファイルが例えば年単位で複数ある場合は結合する
- DataFrameの内容はMT5から取得したデータのままとする
- DataFrame(price_list)の構成は以下の通り
  - Index:time(datetime)
  - open
  - high
  - low
  - close
  - tick_volume
  - spread
  - real_volume

### オブジェクト関数[AddStaticInfo()](#method2)
- ConvertPriceDataFileToDataFrame()で作成したDataFeame(static_data)に、SMA(単純移動平均)とStdev(標準偏差)、z-score等を追加する
- open,high,low,closeは通貨ペアの小数点以下の桁数によって100倍(5桁(UERUSD等))、10倍(4桁(USDZAR等))、1倍(3桁(USDJPY等))する
- DataFrame(static_data)の構成は以下の通り
  - Index:time
  - tick_volume
    - ~~tick_volume/100,000する~~
    - TANH(LOG10(AVERAGE()))とする
  - Close-Open:price_list.close-price_list.open
  - High-Low:price_list.high-price_list.low
  - STD_Short：短期標準偏差.close
  - STD_Long：長期標準偏差.cloe
  - z_score_short：短期Z-Score.close
  - z_score_long：長期Z-Score.close
  - sma_close_short-long：SMA(Short.close)-SMA(Long.close)
  - std_short-long：STD(Short.close)-STD(Long.close)
  - z_score_short-long：Z-Score(Short.close)-Z-Score(Long.close)
  - sma_close-open_short：SMA(Short.close)-SMA(Short.open)
  - sma_close-open_long：SMA(Long.close)-SMA(Long.open)

  - ※SMALong:price_list.closeの長期単純移動平均
  - ※SMAShort:price_list.closeの短期単純移動平均
  - ※カラム名には通貨ペア名(USDJPY等)つける

### オブジェクト関数[GetPriceData()](#method3)
- 指定した日時の時間足データを取得する
 - 引数
   - 時間(datetime)
 - 戻り値
   - 時間足データ(pandas.DataFrame)


In [62]:
#@title class PriceData
class PriceData:

  def __init__(self, sbl, prd):
    self.symbol = sbl
    self.period = prd
    return

  def ConvertPriceDataFileToDataFrame(self):

    # ファイル名は"EURUSD_H1_202301020000_202312312359.csv"
    filenames=glob.glob('%s%s_%s_*.csv'%(PRICEDATA_PATH,self.symbol,self.period))

    # ファイルリストを1つずつ取り出して、detaframeに加工する
    list_ = []
    for file in filenames:
      # csv(tsv)ファイルをpd.dataframeに変換
      # df = pd.read_table(file)
      df = pd.read_csv(file)
      # logger.debug(df)

      # <DATE>と<TIME>を結合し(間にスペースを挿入)、datetime型に変換
      # 新たな列<DATETIME>に結合結果を追加する
      # df['<DATETIME>'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'], format='%Y.%m.%d %H:%M:%S')
      df['time'] = pd.to_datetime(df['time'])

      #<DATETIME>列をインデックスにした後に、<DATE>列と<TIME>列を削除する
      #InedexがDateTimeIndexとなり、時間でのスライスが可能
      # df = df.set_index('<DATETIME>')
      df = df.set_index('time')
      list_.append(df)

    self.price_list = pd.concat(list_)

    # データの加工
    # Indexを昇順に並べる(念のため)
    self.price_list.sort_index(inplace=True)
    # 重複している行を削除
    logger.debug(('PriceData.ConvertPriceDataFileToDataFrame:Duplicated Rows:',self.price_list[self.price_list.duplicated()]))
    self.price_list.drop_duplicates(inplace=True)

  #@markdown ##AddSMAStdev() <a name = "method2"></a>
  def AddStaticInfo(self, periods=None):
    self.static_data = pd.DataFrame(index=self.price_list.index)
    # self.price_listの各列を追加していく
    # open,high,low,closeは通貨ペアによって桁数調整をする
    self.static_data[['open','high','low','close']]\
    = self.price_list[['open','high','low','close']]*EnvironmentCommon.GetDigitMagnification(self.symbol)

    # LONG_PERIOD,SHORT_PERIOD,TICKVOL_PERIODはインスタンス毎に変更する
    if periods != None:
      self.long_period = periods[0]
      self.short_period = periods[1]
      self.tickvol_period = periods[2]
    else:
      self.long_period = LONG_PERIOD
      self.short_period = SHORT_PERIOD
      self.tickvol_period = TICKVOL_PERIOD
    logger.info('PriceData.AddStaticInfo:self.long_period=%d, self.short_period=%d, self.tickvol_period=%d' %(self.long_period,self.short_period,self.tickvol_period))

    # TickVolは10万分の1にする
    # 2023/7/9 glocab変数を変更しない
    self.static_data['tick_volume'] = np.tanh(np.log10(self.price_list['tick_volume']/self.price_list['tick_volume'].rolling(self.tickvol_period).mean()))
    # 統計情報の追加するものは上記の説明参照
    self.static_data['close-open'] = self.static_data['close'] - self.static_data['open']
    self.static_data['high-low'] = self.static_data['high'] - self.static_data['low']
    self.static_data['sma_close_short'] = self.static_data['close'].rolling(self.short_period).mean()
    self.static_data['sma_close_long'] = self.static_data['close'].rolling(self.long_period).mean()
    self.static_data['sma_open_short'] = self.static_data['open'].rolling(self.short_period).mean()
    self.static_data['sma_open_long'] = self.static_data['open'].rolling(self.long_period).mean()
    self.static_data['std_short'] = self.static_data['close'].rolling(self.short_period).std()
    self.static_data['std_long'] = self.static_data['close'].rolling(self.long_period).std()
    self.static_data['z_score_short'] = (self.static_data['close'] - self.static_data['sma_close_short']) / self.static_data['std_short']
    self.static_data['z_score_long'] = (self.static_data['close'] - self.static_data['sma_close_long']) / self.static_data['std_long']
    self.static_data['sma_close_short-long'] = self.static_data['sma_close_short'] - self.static_data['sma_close_long']
    self.static_data['std_short-long'] = self.static_data['std_short'] - self.static_data['std_long']
    self.static_data['z_score_short-long'] = self.static_data['z_score_short'] - self.static_data['z_score_long']
    self.static_data['sma_close-open_short'] = self.static_data['sma_close_short'] - self.static_data['sma_open_short']
    self.static_data['sma_close-open_long'] = self.static_data['sma_close_long'] - self.static_data['sma_open_long']

    # NNに投入しないカラム(open,high,low,close)は削除する
    self.static_data.drop(columns=['open','high','low','close','sma_close_short','sma_close_long','sma_open_short','sma_open_long']
                          , inplace=True)

    # カラム名に一括で通貨ペアを付加する
    self.static_data.rename(columns=lambda s: self.symbol + '_' + s, inplace = True)
    # logger.debug('PriceData.AddStaticInfo:%s' %(self.static_data))

  def GetPriceData(self,dt):
    return self.price_list[self.price_list.index == dt]


## クラスTrainData
テストデータを扱うクラス
### コンストラクタTrainData()
- 複数通貨ペアのPriceDataを受け取り、結合してTrainDataを作成する
- TradeAgentへ渡す訓練データとして、現在のポジションの情報を格納するカラムを追加する
- DataFrame(train_data_list)の構造は以下の通り
  - static_data:各通貨ペア分結合
  - current_pl:現在の損益
  - long_position_flg:ロングを保有していれば"1"
  - short_position_flg:ショートを保有していれば"1"
  (cuttrent_pl、各position_flgは"0"で初期化)
 - 引数：
   - [PriceDataList]：オブジェクトPriceDataのリスト
 - 入力：
   - PositionOpenPrice:現在保有ポジションのエントリー価格(初期値0.0)
   - LongFlg:保有ポジションがLongならば1(初期値0)
   - ShortFlg:保有ポジションがShortならば1(初期値0)
   - Reward:報酬(初期値0.0)

メモ：GetRewardとResetRewardを作る

In [63]:
#@title class TrainData
class TrainData:

  #@markdown ## Constractor <a name = "init1"></a>
  def __init__(self, pdlst, idx=None):
    # static_dataのデータを結合して1つのデータフレームにする
    # axis=1で、横方向に結合(カラムを追加する)
    # join='outer'を指定することで、全ての行が残る
    self.train_data = pd.concat([p.static_data for p in pdlst], axis=1, join='outer')
    # さらに取引通貨ペアのインデックスと合わせる
    if idx is not None:
      self.train_data = self.train_data.reindex(index=idx)

    # 保有ポジションの情報を格納するカラムを追加する
    # self.train_data['current_pl'] = 0
    # self.train_data['long_pos_flg'] = 0
    # self.train_data['short_pos_flg'] = 0
    return

  #  指定したindexのトレーニングデータを返す
  def GetTrainData(self, indx_timedate):
    return self.train_data.loc[indx_timedate]

  # 指定したindexの期間のトレーニングデータを返す
  def GetTrainDataPeriod(self, start_timedate, end_timedate):
    return self.train_data.loc[start_timedate:end_timedate]

  def GetTrainDataColLen(self):
    return len(self.train_data.columns)

## クラス Account
- 訓練するTradeAgentのポジション情報と報酬を管理する
### コンストラクタ  Account()
 - pandas.DataFrame:Accountを初期化する。構造は以下
   - index(Datetime):時間足のオープン時刻
   - ~~reward:報酬。損益が確定した段階で報酬に反映させる。1,000pips=1とする。報酬が確定するときは①ポジションをクローズしたとき②t/pやl/cにかかったとき~~
   →Rewardは、Agentで保持する
   - position_open_price：現在保有しているポジションのエントリー価格
   - has_long：現在保有しているポジションがLongの場合はTrue
   - has_short：現在保有しているポジションがShortの場合はTrue
   - float_pl：現在保有しているポジションの含み損益
   - close_pl：この時間足で確定した損益。報酬と基本的には同じ
 - 引数(Symbol,TimeFrame)を格納する
 - TimeFrameに応じたLimitStopLevelを決定する。private関数setLimitStopLevel()を呼びだす
 - Symbolに対応するマージンを取得する

### オブジェクト関数　CheckTpLc()
- n:00(ex 1:00)に、(n-1):00(ex0:00)足のデータを用いて、Tp/Lcに引っかかていないかをチェックする。
 - 引数
   - last_datetime:Datetime 直近(1つ前)足のOpentime(n-1:00)
   - last_price_data:DataFrame 直前足のPriceData
   - now_datetime:Datetime 現在時刻のOpenTime(n:00)

 - 入力
   - Account((n-1):00) 直近のAccount情報

 - 処理
   1. 直近のAccount情報を取得する
   1. Account.position_open_priceとLastCloseの差から現在時刻(n:00)のprofit_lossを計算する
     - has_long && !has_short → profit_loss = LastClose - position_open_price
     - !has_long && has_short → profit_loss = position_open_price - LastClose
     - !has_long && !has_short → profit_loss = 0
     - has_long && has_short → sys.exit()
  この時、profit_lossからmarginを引く
   1. Tp/Lcに引っかかっていないかチェックする

### オブジェクト関数 ExecuteAction()
- ニューラルネットの判断(Action)に基づきアクションを実行し、Account情報を更新する
- 実行するアクションは
 1. a0:None
 1. a1:LongEntry
 1. a2:ShortEntry
 1. a3:PositionClose
 1. a4:PositionClose&LongEntry
 1. a5:PositionClose&ShortEntry

### オブジェクト関数 CalcCountdown()
- 1 episodeの最後に向かってカウントダウン(実際は0から1に向かってカウントアップ)をする。
 - 1epi=1日の時は、当日の0:00が最初(=0)、翌日0:00の2つ前の足が最後(=1)
 - 1epi=1週の時は、月曜日の0:00が最初(=0)、土曜日の0:00の2つ前の足が最後(=1)
 - 1epi=1年の時は、当年のISO第1週目の月曜日の0:00が最初(=0)、翌年のISO第1週の2週前の土曜日の0:00の2つ前の足が最後(=1)

In [64]:
#@title class Account
class Account:

  def __init__(self,sbl,tf,inidt,trndrt,load_flg=False,file_name=None,periods=None,real_spread_flg=False):
    # accountを格納する。pandas dataframe
    self.df_account = pd.DataFrame(columns=['symbol','period','pos_open_datetime','pos_open_price','has_long','has_short','float_pl','close_pl','countdown'])
    self.symbol = sbl
    self.period = tf
    self.ini_datetime = inidt
    if load_flg and (file_name.account != None) :
      logger.debug('Account.__init__: INIFile loading. \n %s' %(file_name.account))
      #Load FlagがTrueでファイルパスが指定されている場合は、そのファイルを読み込む
      acc_inifile = configparser.ConfigParser()
      acc_inifile.read(file_name.account)
      self.tp_level = float(acc_inifile.get('COMMOM', 'TAKEPROFIT_LEVEL'))
      self.lc_level = float(acc_inifile.get('COMMOM', 'LOSSCUT_LEVEL'))
      # 保有ポジションのモデルに入力する際の倍率(0.1～1.0倍 0.1刻み)
      self.pos_scale = float(acc_inifile.get('COMMOM', 'POS_SCALE'))
      self.real_sprad_limit = float(acc_inifile.get('COMMOM', 'REAL_SPREAD_LIMIT'))
      # 2025/3/15 「空テスト」を行う期間(週数)
      self.pre_trade_weeks = int(acc_inifile.get('COMMOM', 'PRE_TRADE_WEEKS'))
    else:
      logger.debug('Account.__init__: LC/TP Level is set.')
      [self.tp_level] = random.choices([random.randrange(TP_LC_MIN_LEVEL,TP_LC_MAX_LEVEL + 1, TP_LC_STEP),0],\
                                   [math.floor((math.floor(TP_LC_MAX_LEVEL-TP_LC_MIN_LEVEL)+1)/TP_LC_STEP),1])
      [self.lc_level] = random.choices([random.randrange(TP_LC_MIN_LEVEL,TP_LC_MAX_LEVEL + 1, TP_LC_STEP),0],\
                                   [math.floor((math.floor(TP_LC_MAX_LEVEL-TP_LC_MIN_LEVEL)+1)/TP_LC_STEP),1])
      self.tp_level /= 1000
      self.lc_level /= 1000
       # 保有ポジションのモデルに入力する際の倍率(0.1～1.0倍 0.1刻み)
      self.pos_scale = random.randrange(1, 11, 1)/10
      # real_spread_limitを取得する
      self.real_sprad_limit = EnvironmentCommon.getRealSpreadLimit(sbl,real_spread_flg)
      logger.debug("Account.__init__:RealSpreadLimit:%.3f" %(self.real_sprad_limit))
      # 2025/3/15 「空テスト」を行う期間(週数)
      self.pre_trade_weeks = random.choice([0,1,2,4,12])
      # ★★ForTest 1週間で固定
      # self.pre_trade_weeks = 1
      logger.debug("Account.__init__:pre_trade_weeks:%d" %(self.pre_trade_weeks))

    logger.debug('Account.__init__:tp_level=%.3f, lc_level=%.3f, pos_scale=%.1f' %(self.tp_level, self.lc_level ,self.pos_scale))

    self.margin = MARGIN_DICT[self.symbol]
    self.magnification = EnvironmentCommon.GetDigitMagnification(self.symbol)

    self.pos_open_price = 0.0
    # LongとShortを別々に入力する
    self.has_long = 0.0
    self.has_short = 0.0
    # self.has_position = 0.0
    self.float_pl = 0.0
    self.close_pl = 0.0
    self.train_duration = trndrt
    # 時間足の長さに応じたTimeDeltaを取得する
    self.delta_period = EnvironmentCommon.GetPeriodTimeDelta(self.period)
    self.countdown = 0.0

    self.SetTradePeriod(self.ini_datetime, self.train_duration)

    # INIファイルに記録するため、LONG,SHORT,TICKVOL_PERIODの値をオブジェクトに格納する
    if periods != None:
      self.long_period = periods[0]
      self.short_period = periods[1]
      self.tickvol_period = periods[2]
    else:
      self.long_period = LONG_PERIOD
      self.short_period = SHORT_PERIOD
      self.tickvol_period = TICKVOL_PERIOD

    self.ticks_frame = None

  def SetTradePeriod(self, stdt, trdl):
    self.ini_datetime = stdt
    self.train_duration = trdl
    # 初期化するときにトレードの期間(start,end)と、カウントダウンの最後を計算する
    if(self.train_duration == 'D'):
      # 1epiの期間が1日の場合
      # 開始日時は、その日の0:00。ただし、土日の場合は、翌週の月曜とする
      self.start_period = (self.ini_datetime
                           + datetime.timedelta(days=(0 if self.ini_datetime.isoweekday() < 6 else 8 - self.ini_datetime.isoweekday()))).replace(hour=0,minute=0)
      # トレードの終了は翌日0:00の2つ前の時間足
      self.end_period = (self.start_period
                          + datetime.timedelta(days=1)).replace(hour=0,minute=0)
      self.end_period -= self.delta_period
      self.end_trade = self.end_period - self.delta_period
    elif(self.train_duration == 'W'):
      # 1epiの期間が1週の場合
      # トレード開始はその週の月曜日の0:00(datetime.isoweekday()は、月曜日=1,日曜日=7)
      # 日曜日は次の週とするため剰余を使う
      self.start_period = (self.ini_datetime
                          + datetime.timedelta(days=1 - (self.ini_datetime.isoweekday() % 7))).replace(hour=0,minute=0,second=0,microsecond=0)
      # トレードの終了は、その週の土曜日の0:00の2つ前(*)の時間足
      # (*)金曜日の23:00(=self.end_period)の時点で、22:00(=self.end_trade)の時間足を参照するので、df_acount(22:00)のcountdownが1.0となるように計算する
      self.end_period = (self.ini_datetime
                          + datetime.timedelta(days=6 - (self.ini_datetime.isoweekday() % 7))).replace(hour=0,minute=0,second=0,microsecond=0)
      self.end_period -= self.delta_period
      self.end_trade = self.end_period - self.delta_period
    # 1epiの期間が1年(?)の場合
    elif(self.train_duration == 'Y'):
      # 開始日はその日(self.ini_datetime)の属する年の第2週の月曜日0:00
      self.start_period = datetime.datetime.fromisocalendar(self.ini_datetime.year, 2, 1)
      # トレードの終了は、その年の51週目の土曜日0:00の2つ前の時間足
      self.end_period = datetime.datetime.fromisocalendar(self.ini_datetime.year, 51, 6)
      self.end_period -= self.delta_period
      self.end_trade = self.end_period - self.delta_period

  # 特定のindexのdatetimeが存在していない場合は、からの行を追加する
  def SetAccount(self, set_datetime, symbol=None, period=None, pos_open_datetime=None, pos_open_price=0.0
                 , has_long=0.0, has_short=0.0, float_pl=0.0, close_pl=0.0, countdown=0.0):

    # 格納するdatetimeの行がdf_rewardになければ行を作成する
    if set_datetime in self.df_account.index:
      # 何もしない
      pass
    else:
      # 行を新たに作成する
      if symbol == None:
        symbol = self.symbol
      if period == None:
        period = self.period

      # self.df_account.loc[set_datetime] = [symbol, period, pos_open_datetime, pos_open_price,
      #                                    has_position, float_pl, close_pl, countdown]
      self.df_account.loc[set_datetime] = [symbol, period, pos_open_datetime, pos_open_price,
                                         has_long, has_short, float_pl, close_pl, countdown]
    # logger.debug('Account.SetAccount:df_account(%s)\n%s' %(set_datetime, self.df_account.loc[set_datetime]))

  def CheckTpLc(self, lstdt, lstprc):
    self.last_close_price = 0.0

    self.utc_from = lstdt
    self.utc_to = lstdt + self.delta_period
    # 直近足のすべてのtickを取得する
    # 2025/02/11 MT5ではなく、DBから取得するようにする
    # 2025/2/22 DBからではなく、あらかじめDBから取得したDataFrameから直近の時間足だけ取得する
    _ticks_period = self.ticks_frame.query('@self.utc_from<=index<@self.utc_to')
    logger.debug('Account.CheckTpLc:From %s To %s Count of _ticks_period %d' 
                 %(self.utc_from.strftime('%Y-%m-%d %H:%M'),self.utc_to.strftime('%Y-%m-%d %H:%M'),len(_ticks_period)))

    if len(_ticks_period) == 0:
      # tickが取得できなかった場合は、その期間は取引がなかったため、TpLcの評価はしない
      logger.warning('Account.CheckTpLc:Due to No ticks TpLc Skipped. From %s To %s Count of ticks %d' 
                     %(self.utc_from.strftime('%Y-%m-%d %H:%M'),self.utc_to.strftime('%Y-%m-%d %H:%M'),len(_ticks_period)))
      return
        
    # 直近足のopen(ask/bid)、close(ask/bid)を取得する
    self.last_open_ask, self.last_open_bid=_ticks_period.iloc[0,[_ticks_period.columns.get_loc('ask'),_ticks_period.columns.get_loc('bid')]]
    self.last_close_ask, self.last_close_bid=_ticks_period.iloc[-1,[_ticks_period.columns.get_loc('ask'),_ticks_period.columns.get_loc('bid')]]

    # 2024/12/9 pos_open_priceに対するTP/LC priceを格納する
    self.takeprofit_price = 0.0
    self.losscut_price = 0.0
    # 直近のAccount情報を取得する
    self.pos_open_price,self.has_long,self.has_short,self.close_pl\
      = self.df_account.loc[self.utc_from,['pos_open_price','has_long','has_short','close_pl']]

    logger.debug('Account.CheckTpLc:tp_level=%.3f, lc_level=%.3f' %(self.tp_level, self.lc_level))
    logger.debug('Account.CheckTpLc:%s last_open_ask=%.3f, last_open_bid=%.3f, last_close_ask=%.3f, last_close_bid=%.3f' \
                 %(self.utc_from, self.last_open_ask, self.last_open_bid, self.last_close_ask, self.last_close_bid))
    logger.debug('Account.CheckTpLc:pos_open_price=%.3f, has_long=%.3f, has_short=%.3f, close_pl=%.3f' \
                 %(self.pos_open_price, self.has_long,self.has_short, self.close_pl))

    # ポジションを持っている場合は、TPLCの評価を行う
    if(self.pos_open_price > 0):
      # Account.has_position の状況により、直近時間足(n-1:00)の最大利益と最大損失を算出する
      # has_positionはhas_longとhas_shortに分ける。また、ポジションの倍率を掛け、0.1～1.0の範囲とする
      # 通貨ペアによらず、1,000pips=1.000となるように調整する
      # スプレッド分をマージンとして差し引く
      # ↑2023/5/27 スプレッドはprice_dataのspreadで評価する

      # 2024/12/10 pos_open_priceからTPpriceとLCpricrを計算する
      if(self.has_long > 0.0):
        # Long positionを持っている場合
        # last_close_priceはbidで評価する
        '''
        self.max_profit = (self.last_high_price - self.pos_open_price) * self.magnification
        self.max_loss = (self.last_low_price - self.pos_open_price) * self.magnification
        self.float_pl = (self.last_close_price - self.pos_open_price) * self.magnification
        '''
        self.takeprofit_price = self.pos_open_price + self.tp_level/self.magnification
        self.losscut_price = self.pos_open_price - self.lc_level/self.magnification
        self.last_close_price = self.last_close_bid
        self.float_pl = (self.last_close_price - self.pos_open_price) * self.magnification

        # TP\LCの評価
        # self.ticks_frameから最初にTC/LCの水準を超えたtickを抽出する
        self.ticks_tplc = _ticks_period.query('bid <= %f | bid >= %f' %(self.losscut_price,self.takeprofit_price))
        self.ticks_tplc.sort_index(inplace=True)
        logger.debug('Account.CheckTpLc:Count of rows:%s %d' %(self.utc_from,len(self.ticks_tplc)))
        if len(self.ticks_tplc) > 0:
          self.tplc_bid = self.ticks_tplc.iloc[0,self.ticks_tplc.columns.get_loc('bid')]
          if self.tplc_bid <= self.losscut_price:
            # LCにかかった時には、直近のAccountに利益を追加して保有ポジションを解除する
            logger.debug('Account.CheckTpLc:Loss cut was executed.(10)')
            self.__execLossCut()
          elif self.tplc_bid >= self.takeprofit_price:
            # TPにかかった時には、直近のAccountに利益を追加して保有ポジションを解除する
            logger.debug('Account.CheckTpLc:Take profit was executed.(10)')
            self.__execTakeProfit()
          else:
            # どちらにも引っかからない場合は、何もしない。ここには来ないはず
            logger.debug('Account.CheckTpLc:Neither Take profit nor Loss cut was executed.(10)')
            pass

      elif(self.has_short > 0.0):
        # Short positionを持っている場合
        # open_priceはbid last_priceはask(=bid+spread)で評価する
        self.takeprofit_price = self.pos_open_price - self.tp_level/self.magnification
        self.losscut_price = self.pos_open_price + self.lc_level/self.magnification
        self.last_close_price = self.last_close_ask
        self.float_pl = (self.pos_open_price - self.last_close_price) * self.magnification

        # TP\LCの評価
        # self.ticks_frameから最初にTC/LCの水準を超えたtickを抽出する
        # self.ticks_tplc = self.ticks_frame.query('ask <= %f | ask >= %f' %(self.takeprofit_price,self.losscut_price))
        self.ticks_tplc = _ticks_period.query('ask <= %f | ask >= %f' %(self.takeprofit_price,self.losscut_price))
        self.ticks_tplc.sort_index(inplace=True)
        logger.debug('Account.CheckTpLc:Count of rows:%s %d' %(self.utc_from,len(self.ticks_tplc)))
        if len(self.ticks_tplc) > 0:
          self.tplc_ask = self.ticks_tplc.iloc[0,self.ticks_tplc.columns.get_loc('ask')]

          if self.tplc_ask >= self.losscut_price:
            # LCにかかった時には、直近のAccountに利益を追加して保有ポジションを解除する
            logger.debug('Account.CheckTpLc:Loss cut was executed.(20)')
            self.__execLossCut()
          elif self.tplc_ask <= self.takeprofit_price:
            # TPにかかった時には、直近のAccountに利益を追加して保有ポジションを解除する
            logger.debug('Account.CheckTpLc:Take profit was executed.(20)')
            self.__execTakeProfit()
          else:
            # どちらにも引っかからない場合は、何もしない。ここには来ないはず
            logger.debug('Account.CheckTpLc:Neither Take profit nor Loss cut was executed.(20)')
            pass
      else:
        # ここに入ったらエラー
        sys.exit("Account.CheckTpLc:Position opened but no flags.")

      logger.debug("Account.CheckTpLc:%s takeprofit_price=%.3f, losscut_price=%.3f, last_close_price=%.3f, float_pl=%.3f" \
                   %(self.utc_to, self.takeprofit_price, self.losscut_price, self.last_close_price, self.float_pl))
      # logger.info("Account.CheckTpLc:max_profit=%.3f, max_loss=%.3f, float_pl=%.3f" %(self.max_profit, self.max_loss,self.float_pl))

    # 直近足を更新する
    self.df_account.loc[self.utc_from,['pos_open_price','has_long','has_short','float_pl','close_pl']]\
    = (self.pos_open_price, self.has_long, self.has_short, self.float_pl, self.close_pl)
    # logger.debug('Account.CheckTpLc:df_account(%s)\n%s\n-----'%(self.utc_from,self.df_account.loc[self.utc_from]))
    return

  # countdownを計算して、当該時間のAccount情報に追加する
  def CalcCountdown(self, cntdt):
    self.countdown_datetime = cntdt
    self.countdown = (self.countdown_datetime - self.start_period).total_seconds() / (self.end_trade - self.start_period).total_seconds()
    logger.debug('Account.CalcCountdown:Countown=%.4f' %(self.countdown))
    # 当該足をのカウントダウンを更新する
    self.df_account.loc[self.countdown_datetime,['countdown']] = (self.countdown)
    return

  # DataFrame Accountに格納されている値を返す。カラムを指定した場合はそのカラムだけを返す
  def GetAccountInfo(self, idx_dt, cols=None):
    if cols is None:
      cols = self.df_account.columns

    return self.df_account.loc[idx_dt,cols]

  def GetAccountColLen(self):
    return len(self.df_account.columns)

  # このAccountオブジェクトのトレード開始、終了datetimeを返す
  def GetTradePeriod(self):
    return (self.start_period, self.end_period)

  def EvaluateRewrd(self, actn_idx, test_dt, last_dt, df_price_data):

    no_action = 0
    long_entry = 1
    short_entry = 2
    position_close = 3
    close_and_long = 4
    close_and_short = 5

    utc_from = test_dt
    utc_to = utc_from + datetime.timedelta(minutes=1)
    # PositionOpenPrice(SellOpenPrice/BuyOpenPrice)を初期化する。
    # MAX_RETRY_NUM回試行してもRealSpreadが閾値未満にならなかったときは、PositionOpenPriceを0.0としてオーダー不成立とする
    sell_open_price = 0.0
    buy_open_price = 0.0


    for n in range(MAX_RETRY_NUM[self.period]):
      # 2025/2/22 都度DBから取得せずにあらかじめDBから取得したpandasDataFrameから取得する
      _ticks_first = self.ticks_frame.query('@utc_from<=index<@utc_to')
      logger.debug('Account.EvaluateRewrd:From %s To %s Count of ticks_first %d' 
                   %(utc_from.strftime('%Y-%m-%d %H:%M'),utc_to.strftime('%Y-%m-%d %H:%M'),len(_ticks_first)))
      if len(_ticks_first) == 0:
        # tickが取得できなかった場合は、１分進める
        logger.warning('Account.EvaluateRewrd:No ticks. From %s To %s Count of ticks %d' 
                       %(utc_from.strftime('%Y-%m-%d %H:%M'),utc_to.strftime('%Y-%m-%d %H:%M'),len(_ticks_first)))
        utc_from = utc_from + datetime.timedelta(minutes=1)
        utc_to = utc_from + datetime.timedelta(minutes=1)
        continue

      [sell_price,buy_price]=_ticks_first.loc[_ticks_first.index[0],['bid','ask']]
      real_spread = abs(buy_price - sell_price)

      # if real_spread >= REAL_SPREAD_LIMIT_LIST[self.symbol]:
      if real_spread >= self.real_sprad_limit:
        logger.debug('Account.EvaluateRewrd:%s real_spread(%.3f) exceeds limit (%.3f).' %(utc_from,real_spread,self.real_sprad_limit))
        # tick取得時間を1分進める
        utc_from = utc_from + datetime.timedelta(minutes=1)
      else:
        # sell_open_priceとbuy_open_priceを設定してForループを抜ける
        logger.debug('Account.EvaluateRewrd:real_spread(%.3f) is in limit (%.3f).' %(real_spread,self.real_sprad_limit))
        sell_open_price = sell_price
        buy_open_price = buy_price
        break

    # acount(last_dt(1:00))の状態をaccount(2:00(test_dt))に一旦コピーする
    # logger.debug('Account.EvaluateRewrd:Last_datetime=%s, Account=\n%s' %(last_dt,self.df_account.loc[last_dt]))
    self.df_account.loc[test_dt] = self.df_account.loc[last_dt]

    if(actn_idx == no_action):
      # ただし、account(test_dt(2:00))は次の時間足account(3:00)ですぐに評価される
      logger.debug('Account.EvaluateRewrd:Action_index=%s, No Action.' %(actn_idx))
      # 何もしないので、close_plは0にする
      self.df_account.loc[test_dt, 'close_pl'] = 0.0
    elif(actn_idx == long_entry):
      logger.debug('Account.EvaluateRewrd:Action_index=%s, Long Entry.' %(actn_idx))
      # account(2:00)に、次の値を格納する
      # pos_open_datetime = test_dt(2:00), pos_open_price = price_data(2:00).open, has_position = 1, float_pl = 0.0, close_pl = 0.0
      # pos_open_dataにはAskを格納する
      # longとshortを分割する
      # self.df_account.loc[test_dt, ['pos_open_datetime', 'pos_open_price', 'has_long','has_short', 'float_pl', 'close_pl']]\
      #   = [test_dt, df_price_data.loc[test_dt,'open'] + df_price_data.loc[test_dt,'spread']/(1000*self.magnification), self.pos_scale, 0.0, 0.0, 0.0]
      self.df_account.loc[test_dt, ['pos_open_datetime', 'pos_open_price', 'has_long','has_short', 'float_pl', 'close_pl']]\
         = [test_dt, buy_open_price, self.pos_scale, 0.0, 0.0, 0.0]
    elif(actn_idx == short_entry):
      logger.debug('Account.EvaluateRewrd:Action_index=%s, Short Entry.' %(actn_idx))
      # account(2:00)に、次の値を格納する
      # pos_open_datetime = test_dt(2:00), pos_open_price = price_data(2:00).open, has_position = -1, float_pl = 0.0, close_pl = 0.0
      # pos_open_dataにはBidを格納する
      # longとshortを分割する
      self.df_account.loc[test_dt, ['pos_open_datetime', 'pos_open_price', 'has_long','has_short', 'float_pl', 'close_pl']]\
         = [test_dt, sell_open_price, 0.0, self.pos_scale, 0.0, 0.0]
    elif(actn_idx == position_close):
      logger.debug('Account.EvaluateRewrd:Action_index=%s, Position Close.' %(actn_idx))
      # account(2:00)に、次の値を格納する
      # pos_open_datetime = none, pos_open_price = 0.0, has_position = 0, float_pl = 0.0, close_pl = float_pl(1:00)
      # longとshortを分割する
      self.df_account.loc[test_dt, ['pos_open_datetime', 'pos_open_price', 'has_long','has_short', 'float_pl', 'close_pl']]\
         = [None, 0.0, 0.0, 0.0 , 0.0, self.df_account.loc[last_dt,'float_pl']]
    elif(actn_idx == close_and_long):
      logger.debug('Account.EvaluateRewrd:Action_index=%s, Close and Long.' %(actn_idx))
      # account(2:00)に、次の値を格納する
      # pos_open_datetime = test_dt(2:00), pos_open_price = price_data(2:00).open, has_position = 1, float_pl = 0.0, close_pl = float_pl(1:00)
      # pos_open_dataにはAskを格納する
      # longとshortを分割する
      self.df_account.loc[test_dt, ['pos_open_datetime', 'pos_open_price', 'has_long','has_short', 'float_pl', 'close_pl']]\
         = [test_dt, buy_open_price, self.pos_scale, 0.0, 0.0, self.df_account.loc[last_dt,'float_pl']]
    elif(actn_idx == close_and_short):
      logger.debug('Account.EvaluateRewrd:Action_index=%s, Close and Short.' %(actn_idx))
      # account(2:00)に、次の値を格納する
      # pos_open_datetime = test_dt(2:00), pos_open_price = price_data(2:00).open, has_position = 1, float_pl = 0.0, close_pl = float_pl(1:00)
      # pos_open_dataにはBidを格納する
      # longとshortを分割する
      self.df_account.loc[test_dt, ['pos_open_datetime', 'pos_open_price', 'has_long','has_short', 'float_pl', 'close_pl']]\
         = [test_dt, sell_open_price, 0.0, self.pos_scale, 0.0, self.df_account.loc[last_dt,'float_pl']]
    else:
      # 定義されていないので、エラー
      sys.exit('Account.EvaluateRewrd:Action is not defined.')

    return

  def DropAccount(self):
    self.df_account = self.df_account.drop(self.df_account.index)
    logger.debug('Account.DropAccount:df_account %s' %(self.df_account))

  def SaveIniFile(self, filepath):
    with open(filepath , mode='w') as f:
      f.write('[COMMOM]\n')
      f.write('LOSSCUT_LEVEL=' + str(self.lc_level) + '\n')
      f.write('TAKEPROFIT_LEVEL=' + str(self.tp_level) + '\n')
      f.write('LONG_PERIOD='+str(self.long_period)+'\n')
      f.write('SHORT_PERIOD='+str(self.short_period)+'\n')
      f.write('TICKVOL_PERIOD='+str(self.tickvol_period)+'\n')
      f.write('POS_SCALE='+str(self.pos_scale)+'\n')
      f.write('TRAIN_SYMBOL_LIST='+str(TRAIN_SYMBOL_LIST)+'\n')
      f.write('REAL_SPREAD_LIMIT='+str(self.real_sprad_limit)+'\n')
      f.write('PRE_TRADE_WEEKS='+str(self.pre_trade_weeks)+'\n')
    return

  def SaveSetFile(self, mdlname):
    with open(mdlname.set_file_path , mode='w') as f:
      f.write('MODEL_NAME='+mdlname.model_name+'\n')
      f.write('LC_LEVEL=' + str(self.lc_level) + '\n')
      f.write('TP_LEVEL=' + str(self.tp_level) + '\n')
      f.write('LONG_PERIOD='+str(self.long_period)+'\n')
      f.write('SHORT_PERIOD='+str(self.short_period)+'\n')
      f.write('TICKVOL_PERIOD='+str(self.tickvol_period)+'\n')
      f.write('POS_SCALE='+str(self.pos_scale)+'\n')
      f.write('MODEL_INPUT_NUM='+str(MODEL_INPUT_NUM)+'\n')
      f.write('MODEL_HIDDEN_NUM='+str(MODEL_HIDDEN_NUM)+'\n')
      f.write('MODEL_OUTPUT_NUM='+str(MODEL_OUTPUT_NUM)+'\n')
      f.write('TRAIN_SYMBOL_LIST='+str(TRAIN_SYMBOL_LIST)+'\n')
      f.write('MAGIC='+mdlname.magic+'\n')
      f.write('TRADE_SYSTEM='+TRADE_SYSTEM+'\n')
      f.write('REAL_SPREAD_LIMIT='+str(self.real_sprad_limit)+'\n')
      f.write('PRE_TRADE_WEEKS='+str(self.pre_trade_weeks)+'\n')
    return

  # TakeProfitを実行して利益を確定する。private method
  def __execTakeProfit(self):
    self.close_pl += self.tp_level
    self.pos_open_price = 0.0
    self.has_long = 0.0
    self.has_short = 0.0
    self.float_pl = 0.0
    return

  # LossCutを実行して損失を確定する。private method
  def __execLossCut(self):
    self.close_pl -= self.lc_level
    self.pos_open_price = 0.0
    self.has_long = 0.0
    self.has_short = 0.0
    self.float_pl = 0.0
    return

  # トレード期間のtick dataをDBから取得してDataFrameに格納する
  def SetTickDataPeriod(self, start_date, end_date):
    tick_copy_flg = False
    for i in range(10):
      with ENGINE.connect() as conn:
        schema_tbl = str.lower(SCHEMA+'.pricedata_'+self.symbol+'_tick')
        sql="SELECT * FROM "+schema_tbl+" WHERE time_msc>='"+start_date.strftime('%Y-%m-%d %H:%M') \
          +"' AND time_msc<'"+end_date.strftime('%Y-%m-%d %H:%M')+"' ORDER BY time_msc ASC"
        self.ticks_frame = pd.read_sql(sql=text(sql), con=conn)
      logger.debug('Account.SetTickDataPeriod:From %s To %s Count of ticks_frame %d' 
                   %(start_date.strftime('%Y-%m-%d %H:%M'),end_date.strftime('%Y-%m-%d %H:%M'),len(self.ticks_frame)))

      if len(self.ticks_frame) > 0:
        tick_copy_flg = True
        break
      else:
        continue
    if not tick_copy_flg:
      # コピーに失敗したらエラー
      sys.exit("Account.SetTickDataPeriod:Copy Tick Range Failed.")

    # 秒での時間をdatetime形式に変換する
    self.ticks_frame['time']=pd.to_datetime(self.ticks_frame['time'], unit='s')
    self.ticks_frame['time_msc']=pd.to_datetime(self.ticks_frame['time_msc'], unit='ms')
    self.ticks_frame = self.ticks_frame.set_index('time_msc')
    self.ticks_frame.sort_index(inplace=True)
    return

  def DropTickDataPeriod(self):
    if self.ticks_frame is not None:
        self.ticks_frame.drop(self.ticks_frame.index, inplace=True)
    return

  def GetPreTradeWeeks(self):
    return self.pre_trade_weeks

# Package-Agnent
- 取引のアクションを決めるパッケージ以下の機能を実装する
 -  Environmentから加工された価格データを受け取りLSTMに渡す
 - LSTMのアクションをEnvironmentへ渡す
 - DQLを行う

## Dependency

In [65]:
#@title Dependency
import torch
import torch.nn as nn
# import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import random
import math

## Logger
- Loggerの実行は、Environmentと切り離されたときに実行する

In [66]:
#@title Logger Agent
import logging
import logging.config

# logging.config.fileConfig('./drive/My Drive/Colab Notebooks/02_DRLTradingSystem2020/Logging.ini')
# logging.config.fileConfig(LOGGING_INIFILE_PATH)
logconfigfile = configparser.ConfigParser()
logconfigfile.read(LOGGING_INIFILE_PATH,ENC)
logging.config.fileConfig(logconfigfile)
logger_agent = logging.getLogger('DRLAgent')
logger_agent.debug('Debug level massage.')
logger_agent.info('Info level massage.')
logger_agent.warning('Warning level massage.')
logger_agent.error('Error level massage.')
logger_agent.critical('Critical level massage.')

2026-02-15 15:44:32,401 DRLAgent     INFO     Info level massage.
2026-02-15 15:44:32,402 DRLAgent     WARNING  Warning level massage.
2026-02-15 15:44:32,403 DRLAgent     ERROR    Error level massage.
2026-02-15 15:44:32,403 DRLAgent     CRITICAL Critical level massage.


## クラス Agent
・LSTMのニューラルネットを構築する<br>
・与えられた状態から行動を決定する<br>
・次の行動から得られた報酬とその時の状態からその次の行動関数を算出し、誤差を補正する<br>

###input_data(状態S[seq,8])
s0 = 現在の損益(pips/1000) 例えば101.000USDJPY - 100.000USDJPY=1000pips<br>
s1 = LongPosition (0=none, 1=entry)<br>
s2 = ShortPosition (0=none, 1=entry)<br>
s3 = Weekday(0/6 = Monday, 6/6 = Sunday)<br>
s4 = Countdown：毎週月曜日の0:00足を1、金曜日の22:00足を0となるようにする。金曜日の23:00足はマイナスの値となる。<br>
s5 = Close-Open<br>
s6 = High-Low<br>
s7 = Tickvol(1/100000)<br>
※今後、複数symbolをインプットとする場合は、s5-s7のセットを追加する。

###output_data(行動A[seq,6])
a0:何もしない<br>
a1:LongEntry<br>
a2:ShortEntry<br>
a3:PositionClose<br>
a4:PositionClose&LongEntry<br>
a5:PositionClose&ShortEntry<br>
※両建てはしない。ドテンを想定する。

In [67]:
#@title class Agent { output-height: 200 }
# hidden_num = 8 #@param {type:"integer"}

class Agent():

  def __init__(self,sbl,prd,input_num,output_num,ini_datetime,LoadModel=False, FilePath=None, train_mode=True):
    self.hidden_num = input_num
    self.symbol = sbl
    self.period = prd
    self.train_mode = train_mode

    #引数の各素子数に応じたニューラルネットを作成する
    # DDQNに対応する
    self.main_brain = Brain(input_num,output_num,self.hidden_num)
    self.target_brain = Brain(input_num,output_num,self.hidden_num)
    #これまでのトレーニングパラメータを呼び出す
    if(LoadModel and FilePath.brain != None):
      self.main_brain.loadModel(FilePath.brain)

    self.target_brain.copyNN(self.main_brain.state_dict())

    self.device = self.main_brain.GetDevice()

    #Rewardを格納するデータフレームを作成する
    # 2026/2/14 DataFrameはログ用のデータを格納するためのもの
    # Tensor計算用に辞書を用意する
    self.df_reward = pd.DataFrame(
        columns=['symbol','period','reward','q_max','a_max','q_taken','a_taken','exp_s_a']
    )
    # 初期行は不要。必要になったときに行を追加する方が自然。

    # 学習用 Tensor を保持する辞書
    # この辞書の形はtensor_reward={set_datetime:{'reward':xx,'q_max':xx,'a_max':xx,'q_taken':xx,'a_taken':xx,'exp_s_a':xx}}
    # のように、set_datetimeをキーとして入れ子になっている辞書を取り出す感じ（のはず。Copilotの提案）
    self.tensor_reward = {}

    # float_plに対する評価を0~1の倍でRandomに設定する
    if LoadModel and (FilePath.agent != None) :
      logger.debug('Agent.__init__: INIFile loading. \n %s' %(FilePath.agent))
      #Load FlagがTrueでファイルパスが指定されている場合は、そのファイルを読み込む
      agt_inifile = configparser.ConfigParser()
      agt_inifile.read(FilePath.agent)
      self.est_float_pl = float(agt_inifile.get('COMMOM', 'ESTIMATE_FLOAT_PL'))
    else:
      logger_agent.debug('Agent.__init__: ESTIMATE_FLOAT_PL is set.')
      self.est_float_pl = random.random()

    logger_agent.debug('Agent.__init()__: ESTIMATE_FLOAT_PL= %.3f' %(self.est_float_pl))

  def SetReward(self, set_datetime, df_float_close_pl):
    logger_agent.debug('Agent.SetReward:float_pl=%.3f, close_pl=%.3f, est_float_pl=%.3f' \
                 %(df_float_close_pl.float_pl, df_float_close_pl.close_pl, df_float_close_pl.float_pl * self.est_float_pl))

    # 2026/2/14 Copilotの助けを借りて大幅リファクタリング
    # Tensor学習用の辞書とログ用のDetaFrameにそれぞれ格納する
    # reward を Tensor として計算
    prev_reward = 0.0
    if set_datetime in self.df_reward.index:
        # prev_reward = self.df_reward.loc[set_datetime, 'reward']
        prev_reward = self.tensor_reward[set_datetime]["reward"]

    reward_tensor = torch.tensor(prev_reward, device=self.device) + \
        (df_float_close_pl.float_pl * self.est_float_pl + df_float_close_pl.close_pl)

    # Tensor を保存（計算グラフ保持）
    if set_datetime not in self.tensor_reward:
        self.tensor_reward[set_datetime] = {}
    # reward の shape 統一
    reward_tensor = reward_tensor.view(1)
    self.tensor_reward[set_datetime]["reward"] = reward_tensor

    # ログ用
    self.df_reward.loc[set_datetime, 'symbol'] = self.symbol
    self.df_reward.loc[set_datetime, 'period'] = self.period
    self.df_reward.loc[set_datetime, 'reward'] = reward_tensor.item()

    return

  def CalcExpectStateActionValue(self, set_datetime):
    # 2026/2/14 Copilotの助けを借りて大幅リファクタリング
    # Tensorを計算する辞書とログ用のDataframeを分ける
    # 教師データとなる R + γmax(Q(s_t+1,a))を計算してTensor辞書に格納する
    # rt = self.tensor_reward[set_datetime]
    rt = self.tensor_reward.setdefault(set_datetime, {})

    # exp_s_a を Tensor として計算（計算グラフ保持）
    # q_maxが存在しない場合はq_maxをtensor([0.0])にする
    # 併せてq_maxにfloat（スカラー）が入っている場合はTensor([value]) に変換
    # rt.setdefault("q_max", 0.0)
    if "q_max" not in rt:
        rt["q_max"] = torch.tensor([0.0], device=self.device)
    elif isinstance(rt["q_max"], float):
        rt["q_max"] = torch.tensor([rt["q_max"]], device=self.device)

    # reward も shape=[1] に統一
    if "reward" not in rt:
        rt["reward"] = torch.tensor([0.0], device=self.device)
    elif isinstance(rt["reward"], float):
        rt["reward"] = torch.tensor([rt["reward"]], device=self.device)

    # exp_s_a を Tensor として計算
    exp_s_a_tensor = rt["reward"] + GAMMA * rt["q_max"]
    # shape=[1] を保証
    exp_s_a_tensor = exp_s_a_tensor.view(1)
    # 保存
    rt["exp_s_a"] = exp_s_a_tensor

    # TensorDict に保存← 再代入は不要（rt は参照）だが、明示的に書いても OK
    self.tensor_reward[set_datetime] = rt
    # DataFrame には float を保存
    self.df_reward.loc[set_datetime, 'exp_s_a'] = exp_s_a_tensor.item()

    logger_agent.debug('Agent.CalcExpectStatuActionValue:exp_s_a=%.4f' % exp_s_a_tensor.item())

  def DecideAction(self, set_datetime, input_data, action_mask, epi_num):
    # 2026/2/14 Copilotの助けを借りて大幅リファクタリング
    # 計算用Tensorを格納する辞書とログ用のDataframeに分ける
    # input_dataを用いて、NNからすべてのアクションの行動評価関数Qを取得する
    self.main_state_action_values = self.main_brain(input_data)
    self.target_state_action_values = self.target_brain(input_data)
    logger_agent.debug('Agent.DecideAction:main_state_action_values\n%s\n------'%(self.main_state_action_values))
    logger_agent.debug('Agent.DecideAction:target_state_action_values\n%s\n------'%(self.target_state_action_values))

    # DDQNに対応させる。main_brain(main Q-network)とtarget_brain(target Q-network)からそれぞれQ値
    # Q_m(s_t,a),Q_t(S_t,a)を取得する
    # 現在の状態から取り得る行動と、状態関数が最大の行動を取得する
    # ε-greedy法により、次の行動が状態関数の最大とは限らない
    # DDQN対応。mani_brain(main Q-network)から得られたaction_indexと、target_brain(target Q-network)から得られたmax_action_indexを使用する
    (self.acition_index, _) = self.main_brain.decideAction(action_mask, self.main_state_action_values, epi_num, self.train_mode)
    (_, self.max_action_index) = self.target_brain.decideAction(action_mask, self.target_state_action_values, epi_num, self.train_mode)

    if set_datetime not in self.tensor_reward:
        self.tensor_reward[set_datetime] = {}

    # q_max, a_max , q_taken, a_takenを登録する
    # ---- 学習用 Tensor を保存（計算グラフ保持）----
    self.tensor_reward[set_datetime].update({
        "q_max":   self.target_state_action_values[0][self.max_action_index].view(1),
        "q_taken": self.main_state_action_values[0][self.acition_index].view(1),
        "a_max":   self.max_action_index,
        "a_taken": self.acition_index,
    })

    # ---- ログ用 DataFrame には float を保存 ----
    self.df_reward.loc[set_datetime, 'q_max']   = self.tensor_reward[set_datetime]["q_max"].item()
    self.df_reward.loc[set_datetime, 'q_taken'] = self.tensor_reward[set_datetime]["q_taken"].item()
    self.df_reward.loc[set_datetime, 'a_max']   = self.max_action_index
    self.df_reward.loc[set_datetime, 'a_taken'] = self.acition_index

    return self.acition_index

  def UpdateMainNN(self, output, target):
    self.main_brain.evaluateLossFunction(output,target)
    self.main_brain.updateNN()

  def CopyMainNNToTargetNN(self):
    self.target_brain.copyNN(self.main_brain.state_dict())

  def ResetHiddenCellState(self):
    # Main Q-NNのHiddenCellStateをリセット
    (hdn, cel) = self.main_brain.getHiddenCellState()
    logger_agent.debug('Agent.ResetMainHiddenCellState:Before [HiddenState]%s, [CellState]%s' %(hdn, cel))
    self.main_brain.resetHiddenCellState()
    (hdn, cel) = self.main_brain.getHiddenCellState()
    logger_agent.debug('Agent.ResetMainHiddenCellState:After [HiddenState]%s, [CellState]%s' %(hdn, cel))
    # Target Q-NNのHiddenCellStateをリセット
    (hdn, cel) = self.target_brain.getHiddenCellState()
    logger_agent.debug('Agent.ResetTargetHiddenCellState:Before [HiddenState]%s, [CellState]%s' %(hdn, cel))
    self.target_brain.resetHiddenCellState()
    (hdn, cel) = self.target_brain.getHiddenCellState()
    logger_agent.debug('Agent.ResetTargetHiddenCellState:After [HiddenState]%s, [CellState]%s' %(hdn, cel))

  def DropReward(self):
    self.df_reward = self.df_reward.drop(self.df_reward.index)
    # 2026/2/14 Copilotの助けを借りて大幅リファクタリング
    self.tensor_reward = {}   # ← TensorDict もクリア

    logger_agent.debug('Agent.DropReward:df_reward %s' %(self.df_reward))

  def SaveAgentModel(self, AgentFilePath, BrainFilePath,eval_value):
      with open(AgentFilePath , mode='w') as f:
        f.write('[COMMOM]\n')
        f.write('ESTIMATE_FLOAT_PL = ' + str(self.est_float_pl) + '\n')
        f.write('EVAL_VALUE = ' + str(eval_value) + '\n')

      self.main_brain.saveModel(BrainFilePath)

  def SetModelMode(self, train_mode):
    self.train_mode = train_mode
    self.main_brain.train(train_mode)
    self.target_brain.train(train_mode)

  def GetMainHiddenCellState(self):
    return self.main_brain.getHiddenCellState()

  def GetTargetHiddenCellState(self):
    return self.target_brain.getHiddenCellState()

  def GetOutput(self):
    return self.main_brain.getOutput()


## クラス Brain
Agent内のニューラルネット部分をBrainクラスとして別途定義

In [68]:
#@title class Brain { output-height: 200 }

class Brain(nn.Module):

  def __init__(self,input_num,output_num,hidden_num):
    self.input_num = input_num
    self.output_num = output_num
    self.hidden_num = hidden_num

    super(Brain, self).__init__()
    #引数の各素子数に応じたニューラルネットを作成する
    #モデルとしては入力層(input_num)→LSTM→LSTM出力層（=hidden_num)→出力層(output_num)
    self.lstm = nn.LSTM(input_num, hidden_num)
    self.lstm2action = nn.Linear(hidden_num, output_num)
    # DuelingNetwork対応。Value層(lstm2value)とAdvantage層(lstm2action)に分ける。
    self.lstm2value = nn.Linear(hidden_num, 1)

    # NN　ModelをGPUで実行する。可能な場合
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    logger_agent.info('Brain.__init__:torch.device=%s' %(self.device))

    self.lstm.to(self.device)
    self.lstm2action.to(self.device)
    self.lstm2value.to(self.device)

    # logger.debug('Brain.__init__:self.lstm.to(self.device)=%s, self.lstm2action.to(self.device)=%s' %(self.lstm.is_cuda(), self.lstm2action.is_cuda()))

    #hidden_stateとcell_stateを初期化する。hidden_cell_stateは(hidden_state,cell_state)のタプルにする
    self.resetHiddenCellState()
    # 損失関数に渡すデータの中身を初期化する
    # self.clearLossData()

    #損失関数と最適化手法を定義する
    #損失関数は、（ひとまず）SmoothL1Loss(state_action_values(=output),expected_state_action_values(=target))を採用
    #optimiserは、optim.Adam(self.model.parameters(), lr=0.0001)
    self.optimizer = optim.Adam(self.parameters(), lr=0.0001)
    self.criterion = nn.SmoothL1Loss()

    # .setファイルに書き込む素子数をグローバル変数に代入する
    global MODEL_INPUT_NUM, MODEL_OUTPUT_NUM, MODEL_HIDDEN_NUM
    MODEL_INPUT_NUM = self.input_num
    MODEL_OUTPUT_NUM = self.output_num
    MODEL_HIDDEN_NUM = self.hidden_num
    logger_agent.debug('Brain.__init__:MODEL_INPUT_NUM=%s, MODEL_OUTPUT_NUM=%s, MODEL_HIDDEN_NUM=%s' %(MODEL_INPUT_NUM, MODEL_OUTPUT_NUM, MODEL_HIDDEN_NUM))

    return

  # 2026/2/14 Copilotの助けを借りて大幅リファクタリング
  def forward(self, input_data):

    # numpy → tensor
    x = torch.FloatTensor(input_data).to(self.device)

    # LSTM forward
    lstm_out, (self.hidden_state, self.cell_state) = self.lstm(
        x.view(-1, 1, self.input_num),
        (self.hidden_state.to(self.device), self.cell_state.to(self.device))
    )

    # Dueling Network
    self.action_score = self.lstm2action(lstm_out.view(-1, self.hidden_num))
    logger_agent.debug('Brain.forward:self.action_score====\n%s' %(self.action_score))
    self.value_score  = self.lstm2value(lstm_out.view(-1, self.hidden_num)).expand(-1, self.action_score.size(1))
    logger_agent.debug('Brain.forward:self.value_score====\n%s' %(self.value_score))

    # Q(s,a)
    self.output = self.value_score + self.action_score - self.action_score.mean(1, keepdim=True).expand(-1, self.action_score.size(1))
    logger_agent.debug('Brain.forward:self.output====\n%s' %(self.output))

    # ← ログ用に保存しない（Agent が必要なら受け取って保存する）
    return self.output

  def evaluateLossFunction(self, output, target):
    self.loss = self.criterion(output, target)
    return

  def resetHiddenCellState(self):
    #それぞれのテンソルのサイズは(num_layers * num_directions, batch, hidden_size)となる。
    #初期化はrandn()で行う。
    # self.hidden_state = torch.randn(1, 1, self.hidden_num)
    # self.cell_state = torch.randn(1, 1, self.hidden_num)
    # 初期化はzeros()で行う。
    self.hidden_state = torch.zeros(1, 1, self.hidden_num)
    self.cell_state = torch.zeros(1, 1, self.hidden_num)
    return

  # 2026/2/14 Copiotの助けを借りて大幅リファクタリング
  def decideAction(self, action_mask, action_score, episode=-2, train_mode=True):
    #取得したaction_score[1,output_num]にその時に可能な行動をaction_mask[output_num]によりマスクし、
    #その中で最大の行動（インデックス）を返す
    # ε-greedy法を適用する
    masked = torch.where(
        torch.BoolTensor(action_mask).to(self.device),
        action_score,
        torch.full_like(action_score, float('-inf'))
    )
    self.masked_action_scores = masked
    logger_agent.debug('Brain.decideAction:masked_action_scores======\n%s\n=======' %(masked))

    max_value, max_index = masked.max(1)

    epsilon = 0.5 * (1 / (episode + 1))
    # 推論モードの時はε-greedy法は適用しない(ランダムで選ばない)
    if train_mode and epsilon > np.random.uniform(0, 1):
      logger_agent.debug('Brain.decideAction ==Action Random Select==')
      random_scores = torch.where(
          torch.BoolTensor(action_mask).to(self.device),
          torch.rand_like(action_score),
          torch.full_like(action_score, float('-inf'))
      )
      logger_agent.debug('Brain.decideAction:random_action_scores======\n%s\n=======' %(random_scores))
      action_value, action_index = random_scores.max(1)
    else:
        action_index = max_index
        action_value = max_value

    # ← ログ用に保存しない（Agent が必要なら受け取って保存する）
    logger_agent.debug('Brain.decideAction:max_action_index======\n%s\n======\naction_index=======\n%s\n=======' %(action_index,action_value))
    return (action_index, max_index)

  def updateNN(self):
    # ネットワークを更新します
    self.optimizer.zero_grad()  # 勾配をリセット
    self.loss.backward()  # バックプロパゲーションを計算
    self.optimizer.step()  # 結合パラメータを更新
    return

  def setHiddenCellState(self, hidden_cell_state):
    (self.hidden_state, self.cell_state) = hidden_cell_state
    return

  def getHiddenCellState(self):
    return (self.hidden_state, self.cell_state)

  def saveModel(self, PATH):
    # デフォルト状態(_use_new_zipfile_serialization=True)で、runtime errorとなったときは
    # _use_new_zipfile_serialization=Falseで再試行する
    try:
      torch.save(self.state_dict(), PATH)
    except RuntimeError as e:
      logger_agent.error('Brain.saveModel:%s' %(e))
      torch.save(self.state_dict(), PATH, _use_new_zipfile_serialization=False)
    logger_agent.info('Brain.saveModel: Model saved successfully.')
    return

  def loadModel(self, PATH):
    self.load_state_dict(torch.load(PATH,map_location=torch.device(self.device)))
    #hidden_stateとcell_stateを初期化する。hidden_cell_stateは(hidden_state,cell_state)のタプルにする
    self.resetHiddenCellState()
    return

  def copyNN(self, model_dict):
    self.load_state_dict(model_dict)
    return

  def GetDevice(self):
    return self.device

  def getOutput(self):
    # OutputとMaskOutputを返す
    return (self.action_score,self.masked_action_scores)


# Package-Trainer
- NNをトレーニングするための機能を実装する

## Logger
- TrainerパッケージのLoggerインスタンスはlogger_trainerとする

In [69]:
#@title Logger Trainer
import logging
import logging.config

# logging.config.fileConfig('./drive/My Drive/Colab Notebooks/02_DRLTradingSystem2020/Logging.ini')
# logging.config.fileConfig(LOGGING_INIFILE_PATH)
logconfigfile = configparser.ConfigParser()
logconfigfile.read(LOGGING_INIFILE_PATH,ENC)
logging.config.fileConfig(logconfigfile)
logger_trainer = logging.getLogger('DRLTrainer')
logger_trainer.debug('Debug level massage.')
logger_trainer.info('Info level massage.')
logger_trainer.warning('Warning level massage.')
logger_trainer.error('Error level massage.')
logger_trainer.critical('Critical level massage.')

2026-02-15 15:44:32,469 DRLTrainer   INFO     Info level massage.
2026-02-15 15:44:32,470 DRLTrainer   WARNING  Warning level massage.
2026-02-15 15:44:32,471 DRLTrainer   ERROR    Error level massage.
2026-02-15 15:44:32,471 DRLTrainer   CRITICAL Critical level massage.


## クラス Trainer

In [70]:
#@title class Trainer
class Trainer():

  def __init__(self, epinum, sbl, tf, start_dt, tdur, gnum, anum, train_data, periods_dict, load_flg, filepth=None, prds=None, cp_frq=2, grd='X', mgc='yyyymmddHHMMSS',rsf=False):

    self.epi_num = epinum
    self.symbol = sbl
    self.period = tf
    self.start_date_time = start_dt
    self.test_duration = tdur
    self.gen_num = gnum
    self.agent_num = anum
    self.load_flg = load_flg
    self.file_path = filepth
    self.periods = prds
    self.copy_frq = cp_frq
    self.magic = mgc
    self.real_spread_flg = rsf
    # 2026/2/14 Copilotの助けにより大幅なリファクタリング
    self.periods_dict = periods_dict
    self.train_data_dict = { self.period: train_data }

    # Trainerの初期化
    logger_trainer.debug('◆◆%s Initialize Trainer ◆◆' %(self.start_date_time))
    # トレーニングの結果(P/Lの合計と標準偏差)を格納するDataFrame
    self.df_result = pd.DataFrame(columns=['tradenum','sum','mean','sd'])

    # Accountを初期化する
    self.acnt = Account(self.symbol,self.period,self.start_date_time, self.test_duration, self.load_flg, self.file_path, self.periods,self.real_spread_flg)
    # 2025/3/15 「空トレード」を行う期間(週数)は、agentから取得する
    self.pre_trade_weeks = self.acnt.GetPreTradeWeeks()
    logger_trainer.debug('Trainer.__init__:pre_trade_weeks=%s' %(self.pre_trade_weeks))
    # オブジェクトのトレード期間(start_date_time(0:00),end_date_time(23:00))を取得する
    self.start_date_time, self.end_date_time = self.acnt.GetTradePeriod()
    logger_trainer.debug('start_date_time:%s, end_date_time:%s' %(self.start_date_time, self.end_date_time))

    # Agentを初期化する
    # この時、input_numとしてAccount('has_long','has_short','float_pl','countdown')とTrainDataの長さを渡す必要がある
    self.delta_time = EnvironmentCommon.GetPeriodTimeDelta(self.period)
    self.input_num = train_data.GetTrainDataColLen() + 4
    self.output_num = 6
    self.agnt = Agent(self.symbol,self.period,self.input_num,self.output_num,self.start_date_time,self.load_flg,self.file_path)

    # トレード期間(Mon 1:00-Fri 23:00)のprace_dataをまとめて取得する
    self.UpdateTradePeriod(self.start_date_time, self.test_duration)

    # 詳細のテスト結果(acnt.df_account)を格納する
    self.df_detail_testresult = pd.DataFrame()

    # 詳細のテストの情報(input,hidden,cellstate,output,maskoutput)を書き出すファイル名
    self.now_date_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    self.input_filename = TRAIN_DETAIL_INPUT_FILE_NAME+'_'+self.symbol+'_'+self.period+'_'+self.magic+'_'+self.now_date_time+'.csv'
    self.hidden_filename = TRAIN_DETAIL_HIDDEN_FILE_NAME+'_'+self.symbol+'_'+self.period+'_'+self.magic+'_'+self.now_date_time+'.csv'
    self.cellstate_filename = TRAIN_DETAIL_CELLSTATE_FILE_NAME+'_'+self.symbol+'_'+self.period+'_'+self.magic+'_'+self.now_date_time+'.csv'
    self.output_filename = TRAIN_DETAIL_OUTPUT_FILE_NAME+'_'+self.symbol+'_'+self.period+'_'+self.magic+'_'+self.now_date_time+'.csv'
    self.maskoutput_filename = TRAIN_DETAIL_MASKOUTPUT_FILE_NAME+'_'+self.symbol+'_'+self.period+'_'+self.magic+'_'+self.now_date_time+'.csv'

  # 2025/3/16 NNのhidden_cell stateをリセットしないフラグを追加する→やはりなし
  def TrainAgent(self, epinum=-1, train_mode=True, return_detail=False):
    # epinumが-1より大きい場合は、引数を代入する
    if epinum > -1:
      self.epi_num = epinum
    # モデルのモード(トレーニング/推論)の設定
    self.agnt.SetModelMode(train_mode)
    # トレーニングの開始
    logger_trainer.info('Trainer.TrainAgent:◆◆TrainAgent Start from %s to %s◆◆' %(self.start_date_time,self.end_date_time))
    # Backtestの場合は、1周だけ
    for self.enum in range(self.epi_num):
      # 「空トレード」を行う。空トレード期間の月曜リストを取得する
      from_date = self.start_date_time - datetime.timedelta(weeks=self.pre_trade_weeks)
      to_date = self.start_date_time - datetime.timedelta(weeks=1)
      every_monday = pd.date_range(from_date, to_date, freq='W-MON')

      # １週間ごとに「空トレード」をする。トレード処理はプライベートメソッドに書き出す
      # AccountDatFrameとRewardDataFrameは都度破棄する
      # NNの更新はしない
      for w in every_monday:
        self._ExecTrade(w,train_mode,return_detail,pretrade_flg=True)
      # トレード・トレーニングを実施(１週間分)
      self._ExecTrade(self.start_date_time,train_mode,return_detail,pretrade_flg=False)

    # すべてのトレーニングが終了したら結果を返す
    if return_detail:
      return self.df_result, self.df_detail_testresult
    else:
      return self.df_result

  def _ExecTrade(self,stdt,train_mode,return_detail,pretrade_flg):
    if pretrade_flg:
      logger_trainer.info('Trainer.TrainAgent:◇◇PreTraning Epsorde %d Start on %s◇◇' %(self.enum+1,stdt))
    else:
      logger_trainer.info('Trainer.TrainAgent:◆◆Traning Epsorde %d Start on %s◆◆' %(self.enum+1,stdt))
        
    self.UpdateTradePeriod(stdt, self.test_duration)
    lstdt = self._GetLastDateTime(stdt, self.delta_time)
    self.acnt.SetAccount(lstdt)
    self.agnt.SetReward(lstdt, self.acnt.GetAccountInfo(lstdt,['float_pl','close_pl']))

    # 2026/2/14 Copilotの力を借りて大幅リファクタリング
    price_obj = self.periods_dict[self.period][self.symbol]

    # 2023/06/03 実際にテストをするのは、スタート期間のひとつ前の足から
    for lstdt, self.df_train in self.df_trade_data_period.iterrows():
      # ここで、時間の調整
      # 現在時刻(t)のひとつ前(t-1)の時系列データを扱うときのdataetime
      # tstdt(test_date_time) → t, lstdt(last_date_time) → t-1
      tstdt = self._GetNextDateTime(lstdt, self.delta_time)
      logger_trainer.debug('Trainer.TrainAgent:last_date=%s(weekday:%d)' %(lstdt,lstdt.isoweekday()))
      logger_trainer.debug('Trainer.TrainAgent:◆◆Traing test_date%s (weekday:%d)◆◆' %(tstdt,tstdt.isoweekday()))

      # 1.直近足(t-1)の価格データを使ってTP,LCのチェックを行う
      if price_obj.GetPriceData(lstdt).empty:
        logger_trainer.warning('Trainer.TrainAgent:[Empty PriceData]Traing %s skipped.' %(lstdt))
        continue

      self.acnt.CheckTpLc(lstdt, price_obj.GetPriceData(lstdt))

      # 2.Account情報(t-1)の確定損益(close_pl(t-1))と評価損益(float_pl(t-1))をReward(t)として登録する
      # 例：close_pl(1:00),float_pl(1:00)→Reward(2:00)に格納
      self.agnt.SetReward(tstdt, self.acnt.GetAccountInfo(lstdt,['float_pl','close_pl']))

      # 3. トレーニングを行い、その結果、t(2:00)におけるq_max, a_max, q_taken. a_takenを取得する
      #  3-1. t-1カウントダウンの計算
      #       23:00の時点で、22:00のカウントダウンを計算して、22:00のdf_accountに格納する(?)
      self.acnt.CalcCountdown(lstdt)
      #  3-2. t-1(1:00)のAccount情報として取得する
      #       23:00の時点で、22:00.closeのdf_accountから('has_position','float_pl','countdown')を取得する
      #       22:00のcountdownで全てクローズとさせたい
      # self.df_acnt = self.acnt.GetAccountInfo(lstdt,['has_position','float_pl','countdown'])
      # longとshortを分割
      self.df_acnt = self.acnt.GetAccountInfo(lstdt,['has_long','has_short','float_pl','countdown'])
      #  3-3. Accountの状態('has_position','float_pl','countdown')から、取り得るアクションを限定する
      # self.action_mask = EnvironmentCommon.getAvailableAction(self.df_acnt.float_pl, self.df_acnt.has_position, self.df_acnt.countdown)
      self.action_mask = EnvironmentCommon.getAvailableAction(self.df_acnt.float_pl, self.df_acnt.has_long, self.df_acnt.has_short, self.df_acnt.countdown)
      #  3-4. 直近足(t-1)のトレーニングデータを取得する
      # df_train = train.GetTrainData(last_date_time)
      #  3-5. トレーニングデータにアカウント情報を結合する
      self.df_acnt_train = pd.concat([self.df_acnt,self.df_train])
      #  3-6. トレーニングデータなどをもとに次の行動を決定する
      #       この時、q_max、a_max、q_taken、a_takenをdf_reward(t)(2:00)に格納する
      self.action_index = self.agnt.DecideAction(tstdt,self.df_acnt_train.to_list(), self.action_mask, self.enum)
      # 4.選択した行動から損益を評価する。Rewardへの格納は次の時間のループの最初(2.)に行う
      self.acnt.EvaluateRewrd(self.action_index, tstdt, lstdt, price_obj.GetPriceData(tstdt))
      # 5. 2:00のRewardと行動関数から教師データを作成する
      self.agnt.CalcExpectStateActionValue(tstdt)
      # 6. (option)トレードの詳細をファイルに書き出す。return_detail=Trueの時のみ
      if return_detail:
        self._WriteDetailFile(tstdt)

      tstdt += self.delta_time
    # end of for loop
    # Sat0:00の処理　★この処理はいらないかもしれない
    # この時は、取引をしないで直前の時間(23:00)のdf_accountのclose_plとfloat_plを
    # Evaluate Reward, Calc
    # 現時点(0:00)のRewardに格納する
    logger_trainer.debug('◆◆Final Traing on %s◆◆' %(tstdt))
    # 現在時刻(t)のひとつ前(t-1)の時系列データを扱うときのdataetime
    # test_date_time → t, last_date_time → t-1
    lstdt = self._GetLastDateTime(tstdt, self.delta_time)

    # 1.直近足(t-1)の価格データを使ってTP,LCのチェックを行う
    self.acnt.CheckTpLc(lstdt, self.periods_dict[self.period][self.symbol].GetPriceData(lstdt))

    # 2.Account情報(t-1)の確定損益(close_pl(t-1))と評価損益(float_pl(t-1))をReward(t)として登録する
    # 例：close_pl(23:00),float_pl(23:00)→Reward(0:00)に格納
    self.agnt.SetReward(tstdt, self.acnt.GetAccountInfo(lstdt,['float_pl','close_pl']))
    # Step3,4は実施しない。
    #  3-1. t-1カウントダウンの計算
    #       0:00の時点で、23:00のカウントダウンを計算して、23:00のdf_accountに格納する(?)
    self.acnt.CalcCountdown(lstdt)
    # 5. 0:00のRewardと行動関数から教師データを作成する
    #    q_maxは 0 で評価する
    self.agnt.CalcExpectStateActionValue(tstdt)

    # 「空トレード」の時は、隠れ層の状態を確認する
    if pretrade_flg:
      hdn,cel=self.agnt.GetMainHiddenCellState()
      logger_trainer.debug('Trainer.TrainAgent:\n[HiddenState]%s \n[CellState]%s' %(hdn,cel)) 
      logger_trainer.debug('Trainer.TrainAgent:◇◇PreTraning Epsorde %d Finished on %s◇◇' %(self.enum+1,tstdt))
      logger_trainer.debug('Total P/L:%.3f' %(self.acnt.df_account.close_pl.sum()))
    else:
      # トレードの結果を評価する。
      logger_trainer.info('◆◆Training Episorde %d Finish.◆◆' %(self.enum+1))
      logger_trainer.info('Total P/L:%.3f' %(self.acnt.df_account.close_pl.sum()))
      logger_trainer.debug('Total Avg:%.3f' %(self.acnt.df_account.close_pl.mean()))
      logger_trainer.debug('Total SD:%.3f' %(self.acnt.df_account.close_pl.std()))
      # 結果をdf_resultに格納する
      self.df_result.loc[self.enum] = [(self.acnt.df_account.close_pl != 0).sum(), self.acnt.df_account.close_pl.sum(), self.acnt.df_account.close_pl.mean(), self.acnt.df_account.close_pl.std()]
      # トレーニングデータ(q_taken(mon 1:00-fri 23:00))と教師データ(exp_s_a(mon 2:00- sat 0:00))を
      # 2026/2/14 Copilotの助けを借りて大幅リファクタリング
      # 計算用のTensorとログ用のDataFrameを分離する
      # self.q_taken_tensor = torch.cat([s for s in self.agnt.df_reward.iloc[1:-1,self.agnt.df_reward.columns.get_loc('q_taken')]])
      # self.exp_s_a_tensor = torch.cat([s for s in self.agnt.df_reward.iloc[2:,self.agnt.df_reward.columns.get_loc('exp_s_a')]])

      # TensorDict から学習用 Tensor を取り出す
      tensor_list_q_taken = []
      tensor_list_exp_s_a = []
    
      # df_reward の index を使って時系列順に Tensor を取り出す
      reward_index = list(self.agnt.df_reward.index)
    
      for i in range(1, len(reward_index)-1):
          t = reward_index[i]
          rt = self.agnt.tensor_reward[t]
          tensor_list_q_taken.append(rt["q_taken"])
    
      for i in range(2, len(reward_index)):
          t = reward_index[i]
          rt = self.agnt.tensor_reward[t]
          tensor_list_exp_s_a.append(rt["exp_s_a"])
    
      # 連結（計算グラフ保持）
      self.q_taken_tensor = torch.cat(tensor_list_q_taken)
      self.exp_s_a_tensor = torch.cat(tensor_list_exp_s_a)
          
      if self.enum + 1 < self.epi_num:
        # トレーニングモードの時に損失関数に送りNNの更新をする。
        if train_mode:
          self.agnt.UpdateMainNN(self.q_taken_tensor.view(-1,1), self.exp_s_a_tensor.detach().view(-1,1))
          # copy_frq(=2)の回数ごとにTargetQ-NNをMainQ-NNと同じにする
          if((self.enum+1) % self.copy_frq == 0):
            logger_trainer.debug('Trainer.TrainAgent:Copy MainNN to TargetNN Execute.')
            self.agnt.CopyMainNNToTargetNN()
    
      # バックテスト用にdf_accountを退避させる
      if return_detail:
        self.df_detail_testresult = self.acnt.df_account.copy()
      # 隠れ層をリセットする
      self.agnt.ResetHiddenCellState()
      hdn,cel=self.agnt.GetMainHiddenCellState()
      logger_trainer.debug('Trainer.TrainAgent:\n[HiddenState]%s \n[CellState]%s' %(hdn,cel)) 

    # df_account、df_rewardを空にする。
    self.acnt.DropAccount()
    self.agnt.DropReward()
    return

  def _WriteDetailFile(self,test_dt):
    # input情報
    with open(TRAIN_RESULT_PATH+self.input_filename, mode='a',newline='') as f:
      writer = csv.writer(f)
      data = [test_dt,*self.df_acnt_train.to_list()]
      writer.writerow(data)
    # hidden情報、cell情報
    hdnst, clst = self.agnt.GetMainHiddenCellState()
    with open(TRAIN_RESULT_PATH+self.hidden_filename, mode='a',newline='') as f:
      writer = csv.writer(f)
      data = [test_dt]
      lst = hdnst.tolist()
      for l in lst:
        for m in l:
          data.extend(m)
      writer.writerow(data)
    with open(TRAIN_RESULT_PATH+self.cellstate_filename, mode='a',newline='') as f:
      writer = csv.writer(f)
      data = [test_dt]
      lst = clst()
      for l in lst:
        for m in l:
          data.extend(m)
      writer.writerow(data)
    # output,maskoutput情報
    outpt, mskoutpt = self.agnt.GetOutput()
    with open(TRAIN_RESULT_PATH+self.output_filename, mode='a',newline='') as f:
      writer = csv.writer(f)
      data = [test_dt]
      lst = outpt.tolist()
      for l in lst:
        data.extend(l)
      writer.writerow(data)
    with open(TRAIN_RESULT_PATH+self.maskoutput_filename, mode='a',newline='') as f:
      writer = csv.writer(f)
      data = [test_dt]
      lst = mskoutpt.tolist()
      for l in lst:
        data.extend(l)
      writer.writerow(data)

  def ExecBacktest(self, start_year, end_year, test_duration):
    # バックテストの結果を格納するデータフレーム
    # 一行にテスト期間(例えば１週間)毎に結果を集計する
    self.df_backtest_result = pd.DataFrame(columns=['tradenum','sum','mean','sd','cum_sum'])
    self.df_backtest_result_detail = pd.DataFrame(columns=['tradenum','sum','mean','sd','cum_sum'])

    self.acnt.DropAccount()
    self.agnt.DropReward()
    self._DropResult()

    # テスト期間が週間の場合、１週間ごとに結果を集計する
    # 各年の2週目～51週目までをテストする。
    logger_trainer.info('Trainer.ExecBacktest:◆◇◆Start Backtesting.◆◇◆')
    if(test_duration == 'W'):
      for y in range(start_year,end_year+1):
        for w in range(2,52):
          # 初めの時間足の初期値を作る
          self.start_date_time = datetime.datetime.fromisocalendar(y, w, 1)
          self.UpdateTradePeriod(self.start_date_time, test_duration)
          # 推論モードでバックテストを実施する
          self.TrainAgent(1,train_mode=False)
          logger_trainer.debug('Trainer.ExecBacktest:self.df_result:%s' %(self.df_result))
          # 1週間ごとに結果を記録する
          self.df_backtest_result.loc[self.start_date_time] = self.df_result.iloc[-1]
          self.df_backtest_result.loc[self.start_date_time,'cum_sum'] = self.df_backtest_result['sum'].sum()
          self._DropResult()
    elif(test_duration == 'Y'):
      # 未実装
      pass
    elif(test_duration == 'D'):
      # 未実装
      pass
    else:
      # 未実装
      pass
    logger_trainer.info('Trainer.ExecBacktest:◇◆◇Finish Backtesting.◇◆◇')
    return self.df_backtest_result

  def ExecBacktestDetail(self, start_day, end_day, test_duration):
    # バックテストの結果を格納するデータフレーム
    # 一行にテスト期間(例えば１週間)毎に結果を集計する
    self.df_backtest_result = pd.DataFrame(columns=['tradenum','sum','mean','sd','cum_sum'])
    self.df_backtest_detail_result = pd.DataFrame(columns=['pos_open_datetime','pos_open_price','has_long','has_short','close_pl'])

    self.acnt.DropAccount()
    self.agnt.DropReward()
    self._DropResult()

    # start_dayとend_dayから、毎週月曜日のリストを作成する
    every_monday = pd.date_range(start_day, end_day, freq='W-MON')
    # テスト期間が週間の場合、１週間ごとに結果を集計する
    # 各年の2週目～51週目までをテストする。
    logger_trainer.info('Trainer.ExecBacktest:◆◇◆Start DetailBacktesting. From:%s, To:%s◆◇◆' %(start_day,end_day))
    if(test_duration == 'W'):
      for w in every_monday:
        #ISOWEEKが1週目、52週目以降の時はテストしない
        if w.isocalendar().week == 1 or w.isocalendar().week >= 52:
          logger_trainer.info('Trainer.ExecBacktestDetail: %d week test skiped.' %(w.isocalendar().week))
          continue

        # 初めの時間足の初期値を作る
        # self.UpdateTradePeriod(w, test_duration)
        self.acnt.SetTradePeriod(w, test_duration)
        # オブジェクトのトレード期間(start_date_time(0:00),end_date_time(23:00))を取得する
        self.start_date_time, self.end_date_time = self.acnt.GetTradePeriod()
        # 推論モードでバックテストを実施する
        self.df_result, self.df_detail_testresult = self.TrainAgent(1,train_mode=False,return_detail=True)
        logger_trainer.debug('Trainer.ExecBacktest:self.df_result:%s' %(self.df_result))
        # 1週間ごとに結果を記録する
        self.df_backtest_result.loc[self.start_date_time] = self.df_result.iloc[-1]
        self.df_backtest_result.loc[self.start_date_time,'cum_sum'] = self.df_backtest_result['sum'].sum()
        self._DropResult()
        self.df_backtest_detail_result = pd.concat([self.df_backtest_detail_result,self.df_detail_testresult],join='inner')
    elif(test_duration == 'Y'):
      # 未実装
      pass
    elif(test_duration == 'D'):
      # 未実装
      pass
    else:
      # 未実装
      pass
    logger_trainer.info('Trainer.ExecBacktest:◇◆◇Finish Backtesting.◇◆◇')
    return self.df_backtest_result,self.df_backtest_detail_result

  def UpdateTradePeriod(self, stdt, tstdl):
    self.acnt.SetTradePeriod(stdt, tstdl)
    # オブジェクトのトレード期間(start_date_time(0:00),end_date_time(23:00))を取得する
    tmp_start_date_time, tmp_end_date_time = self.acnt.GetTradePeriod()
    logger_trainer.debug('Trainer.UpdateTradePeriod:tmp_start_date_time:%s' %(tmp_start_date_time))
    logger_trainer.debug('Trainer.UpdateTradePeriod:tmp_end_date_time:%s' %(tmp_end_date_time))
    # 2026/2/14 self.end_date_timeが更新されないのでここでする
    self.end_date_time = tmp_end_date_time
    # トレード期間(Mon 0:00-Fri 23:00)のprace_dataをまとめて取得する
    # self.df_trade_data_period = train_data_dict[self.period].GetTrainDataPeriod(self.start_date_time, self.end_date_time)
    # 2023/06/03 取得するトレードデータは、トレード期間のひとつ前の足から終了のひとつ前まで
    previous_start_datetime = self._GetLastDateTime(tmp_start_date_time, self.delta_time)
    previous_end_date_time = self._GetLastDateTime(tmp_end_date_time, self.delta_time)
    logger_trainer.debug('Trainer.UpdateTradePeriod:previous_start_datetime:%s' %(previous_start_datetime))
    logger_trainer.debug('Trainer.UpdateTradePeriod:previous_end_date_time:%s' %(previous_end_date_time))
    self.df_trade_data_period = self.train_data_dict[self.period].GetTrainDataPeriod(previous_start_datetime, previous_end_date_time)

    # 2025/2/22 accountにトレード期間中(開始の１つ前と終了のひとつ後)のtick dataをdataFrameで渡す
    self.acnt.DropTickDataPeriod()
    self.acnt.SetTickDataPeriod(previous_start_datetime,tmp_end_date_time+self.delta_time)
    return

  def SaveParameters(self,Parameters,Modelname=None):
    self.acnt.SaveIniFile(Parameters.account)
    self.agnt.SaveAgentModel(Parameters.agent,Parameters.brain,Parameters.eval_value)
    if Modelname != None:
      self.acnt.SaveSetFile(Modelname)

  # 現在のDateTime(cur_dt)からDeltaTime(dlt_dt)分前のDateTimeを返す
  # ただし、dlt_dt分戻した曜日が土日の場合は金曜日の同時刻までさかのぼる
  def _GetLastDateTime(self,cur_dt, dlt_dt):
    last_dt = cur_dt - dlt_dt
    # 2023/9/9 曜日に関係なく、PriceDataとTrainDataが揃っている時間までさかのぼる
    # if last_dt.isoweekday() >= 6:
    #   last_dt -= (last_dt.isoweekday() % 5)*datetime.timedelta(days=1)
    # LastDateTimeに対応するPriceDataかTrainDataがなければもう1つ時間を戻す
    while self.periods_dict[self.period][self.symbol].GetPriceData(last_dt).empty or\
      self.train_data_dict[self.period].GetTrainDataPeriod(last_dt, last_dt).empty:
      logger_trainer.debug('Trainer._GetLastDateTime:PriceData or TrainData in %s is not exist.' %(last_dt))
      last_dt -= dlt_dt
    return last_dt

  # 現在のDateTime(cur_dt)からDeltaTime(dlt_dt)分"後"のDateTimeを返す
  # ただし、dlt_dt分進めた曜日が土日の場合は月曜日の同時刻まで進める
  def _GetNextDateTime(self,cur_dt, dlt_dt):
    next_dt = cur_dt + dlt_dt
    # 2023/9/9 曜日に関係なく、PriceDataとTrainDataが揃っている時間まで進める
    # if next_dt.isoweekday() >= 6:
    #   next_dt += (3 - (next_dt.isoweekday() % 5))*datetime.timedelta(days=1)
    # NextDateTimeに対応するPriceDataかTrainDataがなければもう1つ時間を進める
    while self.periods_dict[self.period][self.symbol].GetPriceData(next_dt).empty or\
      self.train_data_dict[self.period].GetTrainDataPeriod(next_dt, next_dt).empty:
      logger_trainer.debug('Trainer._GetNextDateTime:PriceData or TrainData in %s is not exist.' %(next_dt))
      next_dt += dlt_dt
    return next_dt

  def _DropResult(self):
    # df_resultの表を削除する
    self.df_result = self.df_result.drop(self.df_result.index)
    logger_trainer.debug('Train._DropResult:df_result %s' %(self.df_result))

## クラス TrainDataMaker
- PraiceData, StaticData, TrainDataの作成をコントロールする

In [71]:
#@title class TrainDataMaker
class TrainDataMaker():

  def __init__(self,period_random_select=False,prmlst=None):
    # PriceData, StaticDataを取り扱うPriceDataオブジェクトを格納する辞書、時間足と通貨ペアの２重構造
    # prices_dict = {'EURUSD': pricedata(eurusd), 'USDJPY': pricedata(usdjpy),...}
    # periods_dict = {'D1': prices_dict(d1), 'H4': prices_dict(h4), ...}
    self.periods_dict = {}
    self.prices_dict = {}
    # train_dataを時間足毎に格納する辞書。train_dataはすべての通貨ペアが結合されているものか、取引対象通貨ペアのみのいずれか(２重構造ではない)
    # train_data_dict = {'D1': traindata(all_symbols/trade_symbol), 'H4': traindata(all_symbols/trade_symbol), ...}
    self.train_data_dict = {}

    # train_dataの窓関数のピリオドを決定する。
    # トレーニング再開の場合は、設定ファイル(account.ini)に記録されているので読み込む
    # 設定ファイルが存在しない(或いは設定ファイルにピリオドの記載がない)場合は、ランダムに選ぶかデフォルト値を使う
    self.train_prm_list = prmlst
    self.periods = [LONG_PERIOD,SHORT_PERIOD,TICKVOL_PERIOD]

    if self.train_prm_list != None and self.train_prm_list.account != None:
      logger_trainer.debug('TrainDataMaker.__init__: INIFile loading. \n %s' %(self.train_prm_list.account))
      #Load FlagがTrueでファイルパスが指定されている場合は、そのファイルを読み込む
      acc_inifile = configparser.ConfigParser()
      acc_inifile.read(self.train_prm_list.account)
      try:
        self.periods[0] = int(acc_inifile.get('COMMOM', 'LONG_PERIOD'))
        self.periods[1] = int(acc_inifile.get('COMMOM', 'SHORT_PERIOD'))
        self.periods[2] = int(acc_inifile.get('COMMOM', 'TICKVOL_PERIOD'))
      except configparser.NoOptionError:
        # Account.iniに設定がない場合は、デフォルトの値を使う
        logger_trainer.warning('TrainDataMaker.__init__:No Such Option Found.')
        pass
    elif period_random_select:
      # ランダムで設定する
      self.periods = EnvironmentCommon.periodRandomSelect()
    else:
      # デフォルト値を使う
      pass

  def MakePriceData(self,train_period_list,train_symbol_list):
    # 時間足、通貨ペア別の価格データをcsvファイルから取り出して、複数年分を１つに結合する

    for prd in train_period_list:
      for sbl in train_symbol_list:
        logger_trainer.info('TrainDataMaker.MakePriceData:Making %s, %s pricedata.' %(sbl,prd))
        prices = PriceData(sbl,prd)
        prices.ConvertPriceDataFileToDataFrame()
        if(self.periods_dict.setdefault(prd) is not None):
          self.prices_dict = self.periods_dict[prd]
        self.prices_dict[sbl] = prices
        self.periods_dict[prd] = self.prices_dict.copy()
        self.prices_dict.clear()
        logger_trainer.info('TrainDataMaker.MakePriceData:Finish %s, %s pricedata.' %(sbl,prd))

  def MakeTrainData(self, train_period, train_symbol, input_all_symbols_flg=True):
    # 対象となる時間足のTrainDataを作成する
    # まず、PriceDataからStaticDataを作成する
    self.prices_dict = self.periods_dict[train_period]
    for symbol, prices in self.prices_dict.items():
      prices.AddStaticInfo(self.periods)
      self.prices_dict[symbol]

    # TrainDataを作成する
    if input_all_symbols_flg:
      logger_trainer.info('TrainDataMaker.MakeTrainData:Making traindata. All %s pricedata combined.' %(train_period))
      self.train_data_dict[train_period] = TrainData(list(self.periods_dict[train_period].values()), self.periods_dict[train_period][train_symbol].price_list.index)
      logger_trainer.info('TrainDataMaker.MakeTrainData:Finish traindata.')
    else:
      logger_trainer.info('TrainDataMaker.MakeTrainData:Making %s, %s traindata.' %(train_symbol,train_period))
      self.train_data_dict[train_period] = TrainData([self.periods_dict[train_period][train_symbol]])
      logger_trainer.info('TrainDataMaker.MakeTrainData:Finish traindata.')

  def GetPeriodsDict(self):
    return self.periods_dict

  def GetPricesDict(self, train_period):
    return self.periods_dict[train_period]

  def SetPeriodsDict(self, pddct):
    # ポインタ渡し
    self.periods_dict = pddct

  def SetPricesDict(self, train_period, pcdct):
    # コピー
    self.periods_dict[train_period] = pcdct.copy()

  def GetTrainDataDict(self):
    return self.train_data_dict

  def GetPeriods(self):
    return self.periods

# Package-Portfolio
トレード候補となるNN Modelの過去の成績から効率的フロンティアを作成し、ポートフォリオを計算する。

## Dependency
このパッケージを実行する前にPyPortfolioOptをインストールする

In [72]:
from pypfopt.efficient_frontier import EfficientFrontier
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import MetaTrader5 as mt5

## Logger Portfolio

In [73]:
#@title Logger Portfolio
import logging
import logging.config

# logging.config.fileConfig('./drive/My Drive/Colab Notebooks/02_DRLTradingSystem2020/Logging.ini')
# logging.config.fileConfig(LOGGING_INIFILE_PATH)
logconfigfile = configparser.ConfigParser()
logconfigfile.read(LOGGING_INIFILE_PATH,ENC)
logging.config.fileConfig(logconfigfile)
logger_portfolio = logging.getLogger('DRLPortfolio')
logger_portfolio.debug('Debug level massage.')
logger_portfolio.info('Info level massage.')
logger_portfolio.warning('Warning level massage.')
logger_portfolio.error('Error level massage.')
logger_portfolio.critical('Critical level massage.')

2026-02-15 15:44:32,568 DRLPortfolio INFO     Info level massage.
2026-02-15 15:44:32,569 DRLPortfolio WARNING  Warning level massage.
2026-02-15 15:44:32,569 DRLPortfolio ERROR    Error level massage.
2026-02-15 15:44:32,570 DRLPortfolio CRITICAL Critical level massage.


## クラス PortfolioConstructer

In [74]:
#@title PortfolioConstructer
class PortfolioConstructer:

  def __init__(self,target_ex):
    self.target_excepted_value = target_ex
    pass

  def ConstructPortfolio(self,df_models):
    # df_modelの内容をプロットする？

    # モデルの平均、分散・共分散行列を計算する
    self.np_models = df_models.to_numpy()
    self.np_cov = np.cov(self.np_models,rowvar=False,bias=True)
    logger_portfolio.debug('EfficientFrontier.ConstructPortfolio:CovMatrix=======\n%s' %(self.np_cov))
    self.np_mean = np.mean(self.np_models,axis=0)
    logger_portfolio.debug('EfficientFrontier.ConstructPortfolio:Means=======\n%s'%(self.np_mean))
    logger_portfolio.debug('EfficientFrontier.ConstructPortfolio:Min mean=%.4f'%(self.np_mean.min()))
    logger_portfolio.debug('EfficientFrontier.ConstructPortfolio:Max mean=%.4f'%(self.np_mean.max()))

    self.S = pd.DataFrame(self.np_cov,columns=df_models.columns)
    self.mu = pd.Series(self.np_mean,index=df_models.columns)
    self.ef = EfficientFrontier(self.mu, self.S)
    self.w_min_dict = self.ef.min_volatility() # 戻り値は辞書型
    self.w_min_np = pd.Series(self.w_min_dict).values
    self.pf_mu_min = np.dot(self.mu,self.w_min_np.T)
    self.v_min = np.sqrt(np.dot(np.dot(self.w_min_np,self.np_cov),self.w_min_np.T)) # 標準偏差の計算
    logger_portfolio.info('EfficientFrontier.ConstructPortfolio:min_volatility pf_mu_min=%.4f, v_min=%.4f' %(self.pf_mu_min,self.v_min))

    # ポートフォリオを構成する各nnの期待値がtarget_expected_valeより小さい場合は、ポートフォリオ内の最大の期待値(np_mean.max())とする
    if self.np_mean.max() < self.target_excepted_value:
      self.ef = EfficientFrontier(self.mu, self.S)
      self.w_dict = ef.efficient_return(target_return=self.np_mean.max())
      self.w_np = pd.Series(self.w_dict).values
      self.pf_mu = np.dot(self.mu,self.w_np.T) # ポートフォリオの期待値
      self.v = np.sqrt(np.dot(np.dot(self.w_np,self.np_cov),self.w_np.T)) # 分散の計算
    # リスク最小の期待値がtarget_expected_valeより小さい場合は期待値が1.0のポートフォリオを採用する
    elif self.pf_mu_min < self.target_excepted_value:
      self.ef = EfficientFrontier(self.mu, self.S)
      self.w_dict = self.ef.efficient_return(target_return=self.target_excepted_value)
      self.w_np = pd.Series(self.w_dict).values
      self.pf_mu = np.dot(self.mu,self.w_np.T) # ポートフォリオの期待値
      self.v = np.sqrt(np.dot(np.dot(self.w_np,self.np_cov),self.w_np.T)) # 分散の計算
    # リスク最小の期待値がtarget_expected_vale以上の場合はリスク最小におけるポートフォリオを採用する
    else:
      self.w_dict = self.w_min_dict
      self.w_np = self.w_min_np
      self.pf_mu = self.pf_mu_min # ポートフォリオの期待値
      self.v = self.v_min

    self.w_dict['mu'] = self.pf_mu
    self.w_dict['sigma'] = self.v
    logger_portfolio.info('EfficientFrontier.ConstructPortfolio:pf_mu=%.4f, v=%.4f' %(self.pf_mu,self.v))
    logger_portfolio.debug('EfficientFrontier.ConstructPortfolio:Portfolio=====\n%s' %(self.w_dict))

    return self.w_dict

  def SavePortfolioFile(self,m_dict):
    # NN モデルのポートフォリオをModelPortfolio.iniファイルに保存する
    self.p_ini = configparser.ConfigParser()
    self.p_ini['LOCAL'] = m_dict

    with open (TRADE_MODEL_PATH+MODEL_PORTFOLIO_FILE_NAME,'w') as fp:
      self.p_ini.write(fp)
    return

  def DrawEfficientFrontier(self,df_models):
    # モデルの平均、分散・共分散行列を計算する
    self.np_models = df_models.to_numpy()
    self.np_cov = np.cov(self.np_models,rowvar=False,bias=True)
    logger_portfolio.debug('EfficientFrontier.ConstructPortfolio:CovMatrix=======\n%s' %(self.np_cov))
    self.np_mean = np.mean(self.np_models,axis=0)
    logger_portfolio.debug('EfficientFrontier.ConstructPortfolio:Means=======\n%s'%(self.np_mean))
    self.S = pd.DataFrame(self.np_cov,columns=df_models.columns)
    self.mu = pd.Series(self.np_mean,index=df_models.columns)
    self.ef = EfficientFrontier(self.mu, self.S)

    self.trets = [i/100 for i in range(round(self.np_mean.min()*100),round(self.np_mean.max()*100))]
    self.tvols = []
    for tr in self.trets:
      w = self.ef.efficient_return(target_return=tr)
      w = pd.Series(w).values
      v = np.sqrt(np.dot(np.dot(w,np.array(self.S)),w.T)) # 分散の計算
      self.tvols += [v]

    plt.style.use('ggplot')
    fig = plt.figure(figsize=(16, 8))
    ax = fig.add_subplot(1, 1, 1)
    ax.scatter(self.tvols, self.trets, marker='x')
    ax.set_xlim([0.0, max(self.tvols)*1.2])
    ax.set_ylim([0.0, self.np_mean.max()*1.2])
    ax.set_xlabel('Volatility')
    ax.set_ylabel('Expected return')
    ax.grid(True)
    plt.show()

    def CalcOrderLots(self,trdmd,mname=None):
        # この関数は、ローカル環境でのみ機能する
        if EXEC_ENV == 'COLABO':
            sys.exit("EfficientFrontier.CalcOrderLots:This function will not work in COLABO.")

        # MT5クライアントに接続して、口座情報(残高)を取得する
        # トレードモード(MT5より借用)
        if trdmd == ACCOUNT_TRADE_MODE_DEMO:
          MT5_PATH = inifile.get(EXEC_ENV, 'mt5_demo_path')
          growth_rate = float(inifile.get(EXEC_ENV, 'GROWTH_RATE_DEMO'))
        elif trdmd == ACCOUNT_TRADE_MODE_REAL:
          MT5_PATH = inifile.get(EXEC_ENV, 'mt5_real_path')
          growth_rate = float(inifile.get(EXEC_ENV, 'GROWTH_RATE_REAL'))
        else:
          sys.exit("EfficientFrontier.CalcOrderLots:Account Trade Mode is not defined.")

        portfolio.read(TRADE_MODEL_PATH+MODEL_PORTFOLIO_FILE_NAME,'UTF-8')
        logger_portfolio.debug("EfficientFrontier.CalcOrderLots:MT5_PATH=%s" %(MT5_PATH))

        # 取引ロットを計算する
        if not mt5.initialize(MT5_PATH):
            sys.exit("EfficientFrontier.CalcOrderLots:initialize() failed, error code =", mt5.last_error())

        account_info = mt5.account_info()
        # mt5.shutdown()

        logger_portfolio.debug("EfficientFrontier.CalcOrderLots:account_info=%s" %(str(account_info)))
        equity = account_info.equity
        logger_portfolio.info("EfficientFrontier.CalcOrderLots:account_info.equity=%s" %(str(equity)))

        # ロット計算に必要な情報を取得する
        if mname is None:
          investment_rate = 1.0
        else:
          investment_rate = float(portfolio.get(EXEC_ENV,mname))
        expected_value = float(portfolio.get(EXEC_ENV,'mu'))
        # logger_trader.debug("Environment:%s growth_rate=%s, investment_rate=%s" %(mname,str(growth_rate),str(investment_rate)))
        logger_portfolio.debug("EfficientFrontier.CalcOrderLots:%s growth_rate=%s, investment_rate=%s, expected_value=%s" %(mname,str(growth_rate),str(investment_rate),str(expected_value)))
        # order_lots = (equity/100000)*growth_rate*investment_rate
        order_lots = (equity/100000)*(growth_rate/expected_value)*investment_rate
        # OrderLotsを少数第3位を四捨五入する
        order_lots = round(order_lots,2)
        logger_portfolio.debug("EfficientFrontier.CalcOrderLots:order_lots=%s" %(str(order_lots)))

        return order_lots

# Stab
- Google Colab から実行するためのスタブ。Mainを実行する前に、GoogleDriveをマウントする。

## Logger
- StabのLoggerインスタンスはlogger_rootとする

In [75]:
#@title Logger Stab
import logging
import logging.config

# logging.config.fileConfig('./drive/My Drive/Colab Notebooks/02_DRLTradingSystem2020/Logging.ini')
# logging.config.fileConfig(LOGGING_INIFILE_PATH)
logconfigfile = configparser.ConfigParser()
logconfigfile.read(LOGGING_INIFILE_PATH,ENC)
logging.config.fileConfig(logconfigfile)
logger_root = logging.getLogger('DRLRoot')
logger_root.debug('Debug level massage.')
logger_root.info('Info level massage.')
logger_root.warning('Warning level massage.')
logger_root.error('Error level massage.')
logger_root.critical('Critical level massage.')

2026-02-15 15:44:32,607 DRLRoot      INFO     Info level massage.
2026-02-15 15:44:32,608 DRLRoot      WARNING  Warning level massage.
2026-02-15 15:44:32,609 DRLRoot      ERROR    Error level massage.
2026-02-15 15:44:32,609 DRLRoot      CRITICAL Critical level massage.


## Make Train Data and Train Model
テストモデル毎にLong/Short/Tickvol Periodを変更するため、都度TrainDataを作成する

In [76]:
if __name__ == '__main__':
  ACCOUNT_TRADE_MODE = ACCOUNT_TRADE_MODE_DEMO #@param ["ACCOUNT_TRADE_MODE_DEMO", "ACCOUNT_TRADE_MODE_CONTEST", "ACCOUNT_TRADE_MODE_REAL"] {type:"raw"}
  # TRAIN_SYMBOL_LIST = ['EURUSD','USDJPY','GBPUSD','AUDUSD','NZDUSD','EURJPY','EURGBP','EURAUD','EURNZD','GBPJPY','AUDJPY','NZDJPY','GBPAUD','GBPNZD','AUDNZD']
  TRAIN_SYMBOL_LIST = ['EURUSD','USDJPY','GBPUSD','EURJPY','EURGBP','GBPJPY']
  TRADE_SYMBOL = "GBPUSD" # @param ["EURUSD", "USDJPY", "GBPUSD", "EURJPY", "EURGBP", "GBPJPY", "GBPAUD", "GBPNZD"]
  TRADE_PERIOD = "D1" #@param ["D1","H12","H8", "H6","H4", "H1", "M30"] # M5とM1はローカル環境では実行禁止とする
  # 20240713 DETAIL_CHECK_PERIDは読み込まない
  DETAIL_CHECK_PERID = 'M1'
  # TRAIN_PERIOD_LIST = [TRADE_PERIOD,DETAIL_CHECK_PERID]
  TRAIN_PERIOD_LIST = [TRADE_PERIOD]
  INPUT_ALL_SYMBOLS_FLG = True
  # periods_dict = {}
  # prices_dict = {}
  # train_dataを時間足毎に格納する辞書
  # train_data_dict = {}

  delta_time = EnvironmentCommon.GetPeriodTimeDelta(TRADE_PERIOD)
  # ファイルパスを設定する
  EnvironmentCommon.setTradeModePath(ACCOUNT_TRADE_MODE)

  LOAD_FLAG = True
  FILE_PATH = None
  # LONG,SHORT,TICKVO+_PERIODをランダムに選択するか
  PERIOD_RANDOM_SELECT = True #@param {type:"boolean"}

  # 1つの世代で作成するTrainAgentの数(default:5)
  TRAIN_AGENT_NUM = 1 #@param {type:"number"}
  # 次の世代に行けるTrainAgentの数(上位)(default:3)
  AGENT_TOGO_NEXT_NUM = 1 #@param {type:"number"}
  # 訓練を最大何世代行うか。最大世代数までに条件にあうTrainAgent(聖杯！）が見つかれば訓練中断(default:5)
  MAX_GEN_NUM = 1 #@param {type:"number"}
  # 1つのエージェントに対して繰り返す訓練の数
  ## EPISODE_NUMは辞書で持って、TRADE_PERIODに対応して取り出すか。(default:200)
  EPISODE_NUM = 3 #@param {type:"integer"}
  # テスト期間は1episodeの長さ。1日(D)、1週間(W)、1年(Y)を選択する
  TEST_DURATION = 'W' #@param ['D', 'W', 'Y']
  # BackTestを行う期間(default:2020-2023)
  BACKTEST_START_YEAR = 2024 #@param {type:"slider", min:2018, max:2030, step:1}
  BACKTEST_END_YEAR = 2024 #@param {type:"slider", min:2018, max:2030, step:1}
  BACKTEST_DURATION = 'W' #@param ['D', 'W', 'Y']

  # For DDQN Target Q-NNとMain Q-NNを同期する頻度(何episode毎か)(defoult:3)
  COPY_FREQ_TARGET_Q_NN = 3 # @param {type:"integer"}
  # エントリーする際のスプレッドを制限するか(しない場合は一律1,000pips)
  REAL_SPREAD_FLG = False

  # 世代別TradeAgent事のBacktestの結果とObjectを格納するDataFrame
  # result:Backtestの損益の合計、trade_avg:1週間当たりの平均損益、std:1週間当たりの損益の標準偏差
  # cdp_0:pips=0の時の累積確率密度関数(cumulative distribution probability function)の値、agt_obj:TradeAgentのObject
  df_gen_rank = pd.DataFrame(columns=['gen','agt_num','result','trade_avg','std','cdp_0','eval_value','agt_obj'])
  # ParameterFileは、AccountのINIファイルとAgentのINIファイル、BrainのPTHファイルがある
  # それぞれのファイルパスをランクごとにNamedTupleに格納する
  # TrainParametersList = [None] * TRAIN_AGENT_NUM
  # 2026/01/22 NNModelの評価値(eval_value)を格納できるようにする
  TrainParameters = namedtuple('TrainParameters', ['account','agent','brain','eval_value'])
  TrainParametersList=[TrainParameters(None,None,None,None)]*TRAIN_AGENT_NUM
  # Trainerを格納するリスト
  AgentList = [None] * TRAIN_AGENT_NUM
  # periods_dictを格納するリスト
  PriceDataDicts = [None] * TRAIN_AGENT_NUM
  # train_data_dictを格納するリスト
  TrainDataDicts = [None] * TRAIN_AGENT_NUM
  # ModelNameとMAGICを格納するNamedTaple
  ModelName = namedtuple('ModelName',['model_name','magic','set_file_path'])

  # 2025/02/11 MT5との接続をやめて、DB(postgresql)と接続する
  # クラス関数で、engineを取得する
  # MT5との接続を開始する
  '''
  logger_root.debug("MT5 initializing:%s" %(MT5_PATH))
  while not mt5.initialize(MT5_PATH):
    logger.info("MT5 Retrying Connecting...")
  logger.debug("MT5 initialized ")
  '''
  EnvironmentCommon.connectDB()
  logger.debug("DB Conneced.")

  # PriceDataはすべての世代、エージェントで共通して使用するため1つだけ作成する
  # 個々のTrainDataを作成する際に、ポインタを渡す
  price_data_common = TrainDataMaker()
  price_data_common.MakePriceData(TRAIN_PERIOD_LIST,TRAIN_SYMBOL_LIST)
  # 20240713 DetailCheckはテスト対象の通貨ペア分だけ作る
  # 2025/2/15 DetailCheckデータは作らない
  # price_data_common.MakePriceData([DETAIL_CHECK_PERID],[TRADE_SYMBOL])

  for gnum in range(MAX_GEN_NUM):
    # 訓練する世代のNN Paramファイルが存在していれば、それを取得し
    # 当該世代の訓練をスキップする
    # ParameterFileは、AccountのINIファイルとAgentのINIファイル、BrainのPTHファイルがある
    # 各世代において、それぞれのファイルの数が同じかを確認し、同じであればRank毎にNamedTupleを生成する
    tmp_account_params=glob.glob('%s%s_%s_Account_Gen%s_Rank*.ini'%(TMP_TRAIN_MODEL_PATH,TRADE_SYMBOL,TRADE_PERIOD,gnum+1))
    logger_root.debug('tmp_account_params:%s' %(tmp_account_params))
    tmp_agent_params=glob.glob('%s%s_%s_Agent_Gen%s_Rank*.ini'%(TMP_TRAIN_MODEL_PATH,TRADE_SYMBOL,TRADE_PERIOD,gnum+1))
    logger_root.debug('tmp_agent_params:%s' %(tmp_agent_params))
    tmp_brain_params=glob.glob('%s%s_%s_Brain_Gen%s_Rank*.pth'%(TMP_TRAIN_MODEL_PATH,TRADE_SYMBOL,TRADE_PERIOD,gnum+1))
    logger_root.debug('tmp_brain_params:%s' %(tmp_brain_params))

    # AccounのINIファイルと、AgentのINIファイル、BrainのPTHファイルの数があっている場合
    # ランクごとにパラメータファイルを読み込む
    if len(tmp_account_params) > 0 and len(tmp_account_params)==len(tmp_agent_params) and len(tmp_brain_params)==len(tmp_account_params):
      TrainParametersList.clear()
      TrainParametersList=[TrainParameters(None,None,None,None)]*TRAIN_AGENT_NUM
      # 当該世代のモデルファイルを読み込む
      for rnk in range(len(tmp_account_params)):
        TrainParametersList[rnk] = TrainParameters(account=TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Account_Gen'+str(gnum+1)+'_Rank'+str(rnk+1)+'.ini',
                                      agent=TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Agent_Gen'+str(gnum+1)+'_Rank'+str(rnk+1)+'.ini',
                                      brain=TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Brain_Gen'+str(gnum+1)+'_Rank'+str(rnk+1)+'.pth')
        logger_root.debug('TrainParameters:%s' %(str(TrainParametersList[rnk])))
        logger_root.info('TrainModel:Generation %s Parameters Loaded.' %(str(gnum+1)))

      # 次の世代へ
      continue

    # 訓練する時間(開始日)をランダムに設定する。
    # どの訓練期間であっても、年は共通(2018～2022)
    # 2024/12/21 MT5サーバのtickが取得できないため、2020～2023に変更
    YEAR = random.randint(2021,2023)
    if(TEST_DURATION == 'Y'):
      # 訓練期間が年単位の場合は、第2週目の月曜日(weekday=1)から
      WEEKNUM = 2
      WEEKDAY = 1
    elif(TEST_DURATION == 'W'):
      # 訓練期間が週単位の場合は、2～51週(ランダム)の月曜日(weekday=1)から
      WEEKNUM = random.randint(2,51)
      WEEKDAY = 1
    elif(TEST_DURATION == 'D'):
      # 訓練期間が日単位の場合は、2～51週(ランダム)の月曜日(weekday=1)～金曜日(weekday=5)(ランダム)
      WEEKNUM = random.randint(2,51)
      WEEKDAY = random.randint(1,5)
    else:
      sys.exit('This period is not implemented.')

    # fromisocalendar()は、python3.8以降でないと実装されていない
    # 月曜日(weekday=1)と最終週の日曜日(weekday=7)
    test_date_time = datetime.datetime.fromisocalendar(YEAR,WEEKNUM,WEEKDAY)
    logger_root.info('Stab.MakeTrainDataAndTrainModel:◆◆Generation %d Start Training.' %(gnum+1))

    for p, prmlst in enumerate(TrainParametersList):
      logger_root.debug('Stab.MakeTrainDataAndTrainModel: Gen %d, Agent %d, Start Training.' %(gnum+1, p+1))
      # AgentListの中身がNoneの場合に新たにTrainerAgentを作成する
      # その時、TrainParameterにファイルパスが格納されている場合はファイルをロードする
      # TrainParameterの値がNoneの場合は、新規作成
      if AgentList[p] == None:
        # Agentが無い場合、Agentに渡すためのPriceDataとTrainDataを作成する。
        # PriceDataは全体で共通のものをポイント参照
        # TrainDataは都度作成する
        price_train_data = TrainDataMaker(PERIOD_RANDOM_SELECT, prmlst)
        price_train_data.SetPeriodsDict(price_data_common.GetPeriodsDict())
        price_train_data.MakeTrainData(TRADE_PERIOD, TRADE_SYMBOL, INPUT_ALL_SYMBOLS_FLG)
        train_data_dict = price_train_data.GetTrainDataDict()
        periods_dict = price_train_data.GetPeriodsDict()
        periods = price_train_data.GetPeriods()

        PriceDataDicts[p] = periods_dict
        TrainDataDicts[p] = train_data_dict
        #__init__(self, epinum, sbl, tf, start_dt, tdur,
        #  gnum, anum, train_data, periods_dict, load_flg, filepth=None, prds=None, cp_frq=2, grd='X', mgc='yyyymmddHHMMSS',rsf=False)
        AgentList[p] = Trainer(EPISODE_NUM, TRADE_SYMBOL, TRADE_PERIOD, test_date_time, TEST_DURATION
                        , gnum+1, p+1, train_data_dict[TRADE_PERIOD], periods_dict, LOAD_FLAG, filepth=prmlst, prds=periods, cp_frq=COPY_FREQ_TARGET_Q_NN, rsf=REAL_SPREAD_FLG)

      else:
        # AgentList[p]にオブジェクトがある場合(前世代からの生き残り)でも、訓練期間(test_date_time)を更新する
        AgentList[p].UpdateTradePeriod(test_date_time,TEST_DURATION)
        # PriceDataとTrainDataも取得する
        periods_dict = PriceDataDicts[p]
        train_data_dict = TrainDataDicts[p]
      '''
      # 2025/3/16 トレーニングやトレードを行う前に一定期間「空トレード」をする
      df_pretrade_result = AgentList[p].ExecPerTrade()
      '''
      # 実際のトレーニング
      df_result = AgentList[p].TrainAgent(epinum=EPISODE_NUM, train_mode=True)
      logger_root.info('■□■TrainModel:%s_%s Gen %d, Agent %d, Train Finish.■□■\n %s' %(TRADE_SYMBOL, TRADE_PERIOD, gnum+1, p+1,df_result.tail()))

      # ★★ForTest
      # sys.exit()

      df_result.to_csv(TMP_TRAIN_RESULT_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Gen'+str(gnum+1)+'_Agent'+str(p+1)+'_TrainResult.csv')

      df_backtest_result = AgentList[p].ExecBacktest(BACKTEST_START_YEAR,BACKTEST_END_YEAR,BACKTEST_DURATION)
      # トレーニングの結果をdf_gen_rankに登録する
      # ['gen','agt_num','result','trade_avg','std','cdp_0','evae','agt_obj']
      '''
      df_gen_rank.loc['%s%s'%(gnum+1, p+1)]=[gnum+1,p+1, df_backtest_result['sum'].sum(),df_backtest_result['sum'].mean(),
                                             df_backtest_result['sum'].std(),
                                             stats.norm(loc=df_backtest_result['sum'].mean(),scale=df_backtest_result['sum'].std()).cdf(0),
                                             df_backtest_result['sum'].mean()*(1-stats.norm(loc=df_backtest_result['sum'].mean(),scale=df_backtest_result['sum'].std()).cdf(0)),
                                             AgentList[p]]
      '''
      # 2023/9/22 eval_valueの評価方法を変更。trade_avgが負の場合は、pips0における累積分布確率関数の左側を掛ける
      result = df_backtest_result['sum'].sum()
      trade_avg = df_backtest_result['sum'].mean()
      std = df_backtest_result['sum'].std()
      cdp_0 = stats.norm(loc=trade_avg,scale=std).cdf(0)
      eval_value = trade_avg*cdp_0 if trade_avg<0 else trade_avg*(1-cdp_0)
      df_gen_rank.loc['%s%s'%(gnum+1, p+1)]=[gnum+1,p+1, result, trade_avg, std, cdp_0, eval_value, AgentList[p]]

      logger.info('TrainModel:Gen %d, Agent %d, BackTest Finish.\n %s' %(gnum+1, p+1,df_gen_rank.loc['%s%s'%(gnum+1, p+1)]))
      df_backtest_result.to_csv(TMP_TRAIN_RESULT_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Gen'+str(gnum+1)+'_Agent'+str(p+1)+'_BacktestResult.csv')
      # Exit Criteriaを満たしているかチェック
      # 1.年間平均週間獲得pips1,000(=1.000)以上かつ獲得pips0における累積分布確率関数が5%以下
      # if(df_gen_rank.loc['%s%s'%(gnum+1, p+1)]['trade_avg'] >1 and df_gen_rank.loc['%s%s'%(gnum+1, p+1)]['cdp_0'] < 0.05):
      # 1.年間平均週間獲得pips(千単位)と、[1-獲得pips0における累積分布確率関数]の積(eval_value)が0.95より大きい
      # 2026/2/14 
      if(df_gen_rank.loc['%s%s'%(gnum+1, p+1)]['eval_value'] >0.95):
        # 【聖杯の発見！】
        logger_root.info('TrainModel:%s_%s ★☆★☆★Grail has got!★☆★☆★' %(TRADE_SYMBOL, TRADE_PERIOD))
        # 1-1. ACCOUNT.INI、AGRNT.INI、MODEL.PTHファイルをMODELディレクトリに保存
        now_date_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        GrailParameters = TrainParameters(account=TRAIN_MODEL_PATH+'Account_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_G_'+now_date_time+'.ini',
                                          agent=TRAIN_MODEL_PATH+'Agent_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_G_'+now_date_time+'.ini',
                                          brain=TRAIN_MODEL_PATH+'Model_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_G_'+now_date_time+'.pth',
                                          eval_value=eval_value)
        GrailModelName = ModelName(model_name=TRADE_SYMBOL+'_'+TRADE_PERIOD+'_G_'+now_date_time,
                                   magic=now_date_time,
                                   set_file_path=TRAIN_MODEL_PATH+'EA_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_G_'+now_date_time+'.set')
        # 2026/2/14 現在存在している.Agent.iniファイルのevel_valueを比較して、大きければ既存ファイルを退避させる
        # TRAIN_MODEL_PATH (ex.D:\ColabNotebooks\06_DuelNetTradingSystem2025\Demo\Model\Train\)配下のファイルは
        # TRAIN_MODEL_PATH_VOID (ex.D:\ColabNotebooks\06_DuelNetTradingSystem2025\Demo\Model\Train\Void\)へ→setteing.iniへ追加
        #    Account_EURJPY_D1_B_20231222124709.ini,Agent_CHFJPY_D1_B_20240721000428.ini,EA_CHFJPY_D1_B_20240721000428.set,Model_EURJPY_D1_B_20231222124709.pth
        # TRAIN_RESULT_PATH (ex.D:\ColabNotebooks\06_DuelNetTradingSystem2025\Demo\Model\Train\Result\)配下のファイルは
        # TRAIN_RESULT_PATH_VOID (ex. D:\ColabNotebooks\06_DuelNetTradingSystem2025\Demo\Model\Train\Result\Void\)へ→setting.iniへ追加
        #   BacktestResult_CHFJPY_D1_B_20240721000428.csv,Result_CHFJPY_D1_B_20240721000428.csv,TrainResult_CHFJPY_D1_B_20240721000428.csv
        if EnvironmentCommon.compareEvalValue(TRADE_SYMBOL,TRADE_PERIOD,eval_value):
            AgentList[p].SaveParameters(GrailParameters,GrailModelName)
            df_gen_rank.loc['%s%s'%(gnum+1, p+1)].to_csv(TRAIN_RESULT_PATH+'Result_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_G_'+now_date_time+'.csv')
            df_result.to_csv(TRAIN_RESULT_PATH+'TrainResult_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_G_'+now_date_time+'.csv')
            df_backtest_result.to_csv(TRAIN_RESULT_PATH+'BacktestResult_'+TRADE_SYMBOL+'_G_'+TRADE_PERIOD+'_'+now_date_time+'.csv')
            # 1-2. MODEL/TMP以下のファイルを削除
            '''テスト用に残しておく
            for p in glob.glob(TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'*'):
              if os.path.isfile(p):
                  os.remove(p)
            '''
        EnvironmentCommon.send_line_notify(TRADE_SYSTEM+'\n'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_G_'+now_date_time+'\n★☆★Grail has got!★☆★\n'+str(df_gen_rank.loc['%s%s'%(gnum+1, p+1),['agt_num','result','eval_value']]))
        # トレーニングは継続する
        # sys.exit()

      # agentの訓練が完了の都度Modelが採用できるかチェックする
      # 採用条件：年間平均週間獲得pips500(=0.500)以上かつ獲得pips0における累積分布確率関数が40%以下
      # if(df_cur_gen_rank.iloc[0]['trade_avg'] >0.5 and df_cur_gen_rank.iloc[0]['cdp_0'] < 0.4):
      # 採用条件：年間平均週間獲得pips(千pips)と[1-獲得pips0における累積分布確率関数]の積が0.3より大きい
      # 2023/5/4 採用条件の変更0.3→0.6(μ=1.0かつ１σ1.0以内)
      # 2023/6/14 採用条件の変更0.6→0.3(μ=0.5かつ１σ1.0以内)
      elif(df_gen_rank.loc['%s%s'%(gnum+1, p+1)]['eval_value'] >0.3):
        # モデルとして採用する
        logger_root.info('TrainModel:%s_%s ★☆★Silver Model has got.★☆★' %(TRADE_SYMBOL,TRADE_PERIOD))
        # ACCOUNT.INI、AGRNT.INI、MODEL.PTHファイルをMODELディレクトリに保存
        now_date_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        ModelParameters = TrainParameters(account=TRAIN_MODEL_PATH+'Account_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_S_'+now_date_time+'.ini',
                                          agent=TRAIN_MODEL_PATH+'Agent_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_S_'+now_date_time+'.ini',
                                          brain=TRAIN_MODEL_PATH+'Model_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_S_'+now_date_time+'.pth',
                                          eval_value=eval_value)
        ModelNNName = ModelName(model_name=TRADE_SYMBOL+'_'+TRADE_PERIOD+'_S_'+now_date_time,
                                magic=now_date_time,
                                set_file_path=TRAIN_MODEL_PATH+'EA_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_S_'+now_date_time+'.set')
        # 2026/2/14 現在存在している.Agent.iniファイルのevel_valueを比較して、大きければ既存ファイルを退避させる
        if EnvironmentCommon.compareEvalValue(TRADE_SYMBOL,TRADE_PERIOD,eval_value):
            AgentList[p].SaveParameters(ModelParameters,ModelNNName)
            df_gen_rank.loc['%s%s'%(gnum+1, p+1)].to_csv(TRAIN_RESULT_PATH+'Result_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_S_'+now_date_time+'.csv')
            df_result.to_csv(TRAIN_RESULT_PATH+'TrainResult_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_S_'+now_date_time+'.csv')
            df_backtest_result.to_csv(TRAIN_RESULT_PATH+'BacktestResult_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_S_'+now_date_time+'.csv')
            '''
            # 採用したモデルの訓練データとバックテストデータをコピーする
            shutil.copyfile(TMP_TRAIN_RESULT_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Gen'+str(df_cur_gen_rank.iloc[0]['gen'])
                        +'_Agent'+str(df_cur_gen_rank.iloc[0]['agt_num'])+'_TrainResult.csv',
                        TRAIN_RESULT_PATH+'TrainResult_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.csv')
            shutil.copyfile(TMP_TRAIN_RESULT_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Gen'+str(df_cur_gen_rank.iloc[0]['gen'])
                        +'_Agent'+str(df_cur_gen_rank.iloc[0]['agt_num'])+'_BacktestResult.csv',
                        TRAIN_RESULT_PATH+'BacktestResult_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.csv')
            '''
        EnvironmentCommon.send_line_notify(TRADE_SYSTEM+'\n'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_S_'+now_date_time+'\n★☆Silver Model has got.★☆\n'+str(df_gen_rank.loc['%s%s'%(gnum+1, p+1),['agt_num','result','eval_value']]))

      # 評価値が0.15より大きいモデルについても採用する
      elif(df_gen_rank.loc['%s%s'%(gnum+1, p+1)]['eval_value'] >0.15):
        # モデルとして採用する
        logger_root.info('TrainModel:%s_%s ★★Blonze Model has got.★★' %(TRADE_SYMBOL,TRADE_PERIOD))
        # ACCOUNT.INI、AGRNT.INI、MODEL.PTHファイルをMODELディレクトリに保存
        now_date_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        ModelParameters = TrainParameters(account=TRAIN_MODEL_PATH+'Account_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_B_'+now_date_time+'.ini',
                                          agent=TRAIN_MODEL_PATH+'Agent_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_B_'+now_date_time+'.ini',
                                          brain=TRAIN_MODEL_PATH+'Model_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_B_'+now_date_time+'.pth',
                                          eval_value=eval_value)
        ModelNNName = ModelName(model_name=TRADE_SYMBOL+'_'+TRADE_PERIOD+'_B_'+now_date_time,
                                magic=now_date_time,
                                set_file_path=TRAIN_MODEL_PATH+'EA_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_B_'+now_date_time+'.set')
        # 2026/2/14 現在存在している.Agent.iniファイルのevel_valueを比較して、大きければ既存ファイルを退避させる
        if EnvironmentCommon.compareEvalValue(TRADE_SYMBOL,TRADE_PERIOD,eval_value):
            AgentList[p].SaveParameters(ModelParameters,ModelNNName)
            df_gen_rank.loc['%s%s'%(gnum+1, p+1)].to_csv(TRAIN_RESULT_PATH+'Result_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_B_'+now_date_time+'.csv')
            df_result.to_csv(TRAIN_RESULT_PATH+'TrainResult_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_B_'+now_date_time+'.csv')
            df_backtest_result.to_csv(TRAIN_RESULT_PATH+'BacktestResult_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_B_'+now_date_time+'.csv')
        EnvironmentCommon.send_line_notify(TRADE_SYSTEM+'\n'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_B_'+now_date_time+'\n★★Blonze Model has got.★★\n'+str(df_gen_rank.loc['%s%s'%(gnum+1, p+1),['agt_num','result','eval_value']]))

      # 評価値が0.08より大きいモデルについても採用する
      elif(df_gen_rank.loc['%s%s'%(gnum+1, p+1)]['eval_value'] >0.08):
        # モデルとして採用する
        logger_root.info('TrainModel:%s_%s ★Test Model has got.★' %(TRADE_SYMBOL,TRADE_PERIOD))
        # ACCOUNT.INI、AGRNT.INI、MODEL.PTHファイルをMODELディレクトリに保存
        now_date_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        ModelParameters = TrainParameters(account=TRAIN_MODEL_PATH+'Account_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_T_'+now_date_time+'.ini',
                                          agent=TRAIN_MODEL_PATH+'Agent_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_T_'+now_date_time+'.ini',
                                          brain=TRAIN_MODEL_PATH+'Model_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_T_'+now_date_time+'.pth',
                                          eval_value=eval_value)
        ModelNNName = ModelName(model_name=TRADE_SYMBOL+'_'+TRADE_PERIOD+'_T_'+now_date_time,
                                magic=now_date_time,
                                set_file_path=TRAIN_MODEL_PATH+'EA_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_T_'+now_date_time+'.set')
        # 2026/2/14 現在存在している.Agent.iniファイルのevel_valueを比較して、大きければ既存ファイルを退避させる
        if EnvironmentCommon.compareEvalValue(TRADE_SYMBOL,TRADE_PERIOD,eval_value):
            AgentList[p].SaveParameters(ModelParameters,ModelNNName)
            df_gen_rank.loc['%s%s'%(gnum+1, p+1)].to_csv(TRAIN_RESULT_PATH+'Result_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_T_'+now_date_time+'.csv')
            df_result.to_csv(TRAIN_RESULT_PATH+'TrainResult_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_T_'+now_date_time+'.csv')
            df_backtest_result.to_csv(TRAIN_RESULT_PATH+'BacktestResult_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_T_'+now_date_time+'.csv')
        EnvironmentCommon.send_line_notify(TRADE_SYSTEM+'\n'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_T_'+now_date_time+'\n★Test Model has got.★\n'+str(df_gen_rank.loc['%s%s'%(gnum+1, p+1),['agt_num','result','eval_value']]))
      else:
        logger_root.debug('TrainModel:Continue training.')

    # Rankの並べ替え、世代別に評価値(eval_value)の大きい順に並べ替える。その後、最新の世代のランキングだけ取り出す
    df_gen_rank.sort_values(['gen','eval_value'],ascending=[True,False],inplace=True)
    df_cur_gen_rank = df_gen_rank.query('gen == %d' %(gnum+1))
    # df_cur_gen_rank.reset_index(inplace=True, drop=True)
    logger_root.info('TrainModel:Generation %d train finished.\n%s' %(gnum+1,df_cur_gen_rank))
    df_cur_gen_rank.to_csv(TMP_TRAIN_RESULT_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Gen'+str(gnum+1)+'_Rank.csv')

    # AgentToGoの数だけ生き残り、生き残り分のACCOUNT.INI、AGRNT.INI、MODEL.PTHファイルをMODEL/TMPディレクトリに保存
    # 足りない分はNoneで見たす
    # 2023/7/16 メモリを開放する
    # AgentList.clear()
    del AgentList
    AgentList = [None] * TRAIN_AGENT_NUM
    # TrainParametersList.clear()
    del TrainParametersList
    TrainParametersList=[TrainParameters(None,None,None,None)]*TRAIN_AGENT_NUM

    # 次の世代にperiods_dictを格納するリスト
    NextPriceDataDicts = [None] * TRAIN_AGENT_NUM
    # 次の世代にtrain_data_dictを格納するリスト
    NextTrainDataDicts = [None] * TRAIN_AGENT_NUM

    for rnk in range(AGENT_TOGO_NEXT_NUM):
      AgentList[rnk] = df_cur_gen_rank.iloc[rnk]['agt_obj']
      # ACCOUNT.INI、AGRNT.INI、MODEL.PTHファイルをMODEL/TMPディレクトリに保存
      TrainParametersList[rnk] = TrainParameters(account=TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Account_Gen'+str(gnum+1)+'_Rank'+str(rnk+1)+'.ini',
                                                 agent=TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Agent_Gen'+str(gnum+1)+'_Rank'+str(rnk+1)+'.ini',
                                                 brain=TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Brain_Gen'+str(gnum+1)+'_Rank'+str(rnk+1)+'.pth',
                                                 eval_value=eval_value)
      AgentList[rnk].SaveParameters(TrainParametersList[rnk],None)
      NextPriceDataDicts[rnk] = PriceDataDicts[int(df_cur_gen_rank.iloc[rnk]['agt_num'])-1]
      NextTrainDataDicts[rnk] = TrainDataDicts[int(df_cur_gen_rank.iloc[rnk]['agt_num'])-1]

    # 2023/7/15 メモリを解放する
    #PriceDataDicts.clear()
    del PriceDataDicts
    PriceDataDicts = NextPriceDataDicts.copy()
    #NextPriceDataDicts.clear()
    del NextPriceDataDicts

    # TrainDataDicts.clear()
    del TrainDataDicts
    TrainDataDicts = NextTrainDataDicts.copy()
    # NextTrainDataDicts.clear()
    del NextTrainDataDicts
    gc.collect()

    EnvironmentCommon.send_line_notify(TRADE_SYSTEM+'\n'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'\nGen '+str(gnum+1)+' Train has finished.\n'+str(df_cur_gen_rank.iloc[0,[1,2,6]]))

  # 所定の世代数のトレーニングが終了
  # 1-2. MODEL/TMP以下のファイルを削除
  '''
  テスト用にTMPファイルを残しておく
  for p in glob.glob(TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'*'):
    if os.path.isfile(p):
        os.remove(p)
  logger_root.info('TrainModel:TMP File Deleted.')
  '''

  # 2025/2/15 MT5との接続はしない。
  # MT5との接続を終了する
  # mt5.shutdown()
  logger_root.debug('MT5 ShutDown Completed.')

# logger_root.info('TrainModel:%s_%s ▼▼No Model has found.▼▼' %(TRADE_SYMBOL,TRADE_PERIOD))
  EnvironmentCommon.send_line_notify(TRADE_SYSTEM+'\n'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'\n▼▼Train has finished.▼▼')
  logger_root.info('TrainModel:%s_%s Train has finished.' %(TRADE_SYMBOL,TRADE_PERIOD))

2026-02-15 15:44:39,322 DRLTrainer   INFO     TrainDataMaker.MakePriceData:Making EURUSD, D1 pricedata.
2026-02-15 15:44:39,377 DRLTrainer   INFO     TrainDataMaker.MakePriceData:Finish EURUSD, D1 pricedata.
2026-02-15 15:44:39,378 DRLTrainer   INFO     TrainDataMaker.MakePriceData:Making USDJPY, D1 pricedata.
2026-02-15 15:44:39,424 DRLTrainer   INFO     TrainDataMaker.MakePriceData:Finish USDJPY, D1 pricedata.
2026-02-15 15:44:39,425 DRLTrainer   INFO     TrainDataMaker.MakePriceData:Making GBPUSD, D1 pricedata.
2026-02-15 15:44:39,455 DRLTrainer   INFO     TrainDataMaker.MakePriceData:Finish GBPUSD, D1 pricedata.
2026-02-15 15:44:39,456 DRLTrainer   INFO     TrainDataMaker.MakePriceData:Making EURJPY, D1 pricedata.
2026-02-15 15:44:39,537 DRLTrainer   INFO     TrainDataMaker.MakePriceData:Finish EURJPY, D1 pricedata.
2026-02-15 15:44:39,538 DRLTrainer   INFO     TrainDataMaker.MakePriceData:Making EURGBP, D1 pricedata.
2026-02-15 15:44:39,592 DRLTrainer   INFO     TrainDataMaker.Mak

In [ ]:
tensor_list_exp_s_a

In [ ]:
AgentList[1].acnt.ticks_tplc.iloc[0,AgentList[1].acnt.ticks_tplc.columns.get_loc('bid')]

## MakeTrainData and BackTestDetail

In [ ]:
import configparser

TRAIN_SYMBOL_LIST=None
acc_inifile = configparser.ConfigParser()
try:
  acc_inifile.read(r"G:\マイドライブ\Colab Notebooks\04_DuelNetTradingSystem2023\Demo\Model\Train\Account_GBPAUD_H6_B_20240606014207.ini")
  TRAIN_SYMBOL_LIST = eval(acc_inifile.get('COMMOM','TRAIN_SYMBOL_LIST'))
except configparser.NoOptionError:
  TRAIN_SYMBOL_LIST = ['EURUSD','USDJPY','GBPUSD','EURJPY','EURGBP','GBPJPY']

print(TRAIN_SYMBOL_LIST)

In [ ]:
if __name__ == '__main__':
  ACCOUNT_TRADE_MODE = ACCOUNT_TRADE_MODE_REAL #@param ["ACCOUNT_TRADE_MODE_DEMO", "ACCOUNT_TRADE_MODE_CONTEST", "ACCOUNT_TRADE_MODE_REAL"] {type:"raw"}
  # TRAIN_SYMBOL_LIST = ['EURUSD','USDJPY','GBPUSD','EURJPY','EURGBP','GBPJPY']
  TRAIN_SYMBOL_LIST = None
  TRADE_SYMBOL = '_' # ダミー
  TRADE_PERIOD = '_' #ダミー
  DETAIL_CHECK_PERID = 'M1'
  INPUT_ALL_SYMBOLS_FLG = True
  periods_dict = {}
  prices_dict = {}
  # train_dataを時間足毎に格納する辞書
  train_data_dict = {}
  # TrainディレクトリにあるNN Modelパラメータを取得する
  # 1つのエージェントに対して繰り返す訓練の数
  ## EPISODE_NUMは辞書で持って、TRADE_PERIODに対応して取り出すか。
  EPISODE_NUM = 200 #@param {type:"integer"}
  # テスト期間は1episodeの長さ。1日(D)、1週間(W)、1年(Y)を選択する
  TEST_DURATION = 'W' #@param ['D', 'W', 'Y']
  BACKTEST_START_DATE = '2024-08-05' #@param {type:"date"}
  BACKTEST_END_DATE = '2024-08-10' #@param {type:"date"}
  BACKTEST_DURATION = 'W' #@param ['D', 'W', 'Y']
  LOAD_FLAG = True
  # LONG,SHORT,TICKVO+_PERIODをランダムに選択するか
  PERIOD_RANDOM_SELECT = False

  # For DDQN Target Q-NNとMain Q-NNを同期する頻度(何episode毎か)
  # BackTestDetailでは使わない
  # COPY_FREQ_TARGET_Q_NN = 3 # @param {type:"integer"}

  # ファイルパスを設定する
  EnvironmentCommon.setTradeModePath(ACCOUNT_TRADE_MODE)

  TEST_SYMBOL = "GBPAUD" # @param ["*", "EURUSD", "USDJPY", "GBPUSD", "EURJPY", "EURGBP", "GBPJPY", "GBPAUD", "GBPZD"]
  TEST_PERIOD = "*" #@param ["*","D1","H12","H8", "H6","H4", "H1", "M30",  "M1"]
  TEST_GRADE = "*" #@param ["*","G","S","B","[GS]","[SB]","[BG]"]
  TEST_MAGIC = "*" #@param {type:"string"}
  # For test
  nn_model_params=glob.glob('%sModel_%s_%s_%s_%s.pth'%(TRAIN_MODEL_PATH,TEST_SYMBOL,TEST_PERIOD,TEST_GRADE,TEST_MAGIC))
  # nn_model_params=glob.glob('%sModel_*_*_*.pth'%(TRAIN_MODEL_PATH))

  TrainParameters = namedtuple('TrainParameters', ['account','agent','brain'])
  TrainParametersList=[TrainParameters(None,None,None)]*len(nn_model_params)
  # ModelNameとMAGICを格納するNamedTaple
  ModelName = namedtuple('ModelName',['model_name','magic'])
  ModelNameList = [ModelName(None,None)]*len(nn_model_params)

  # backtestの結果を格納するDataFrame
  df_models = pd.DataFrame()

  TRAIN_PERIOD_LIST = ["D1","H12","H8", "H6","H4", "H1", "M30", "M1"]
  # TRAIN_PERIOD_LIST = ["D1","H12","M1"]

　# MT5との接続を開始する
  logger_root.debug("MT5 initializing:%s" %(MT5_PATH))
  res = mt5.initialize(MT5_PATH)
  logger.debug("MT5 initialized result:%s" %(res))


  # NN Modelのパラメータを読み込む
  for i, mprm in enumerate(nn_model_params):
    s = mprm.split('_') # Account_EURJPY_H6_B_20230907090921.ini
    TRADE_SYMBOL = s[2]
    TRADE_PERIOD = s[3]
    TRADE_GRADE = s[4]
    MAGIC = s[5].split('.')[0]

    TrainParametersList[i] = TrainParameters(account=TRAIN_MODEL_PATH+'Account_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+TRADE_GRADE+'_'+MAGIC+'.ini',
                                  agent=TRAIN_MODEL_PATH+'Agent_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+TRADE_GRADE+'_'+MAGIC+'.ini',
                                  brain=mprm)
    ModelNameList[i] = ModelName(model_name=TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+TRADE_GRADE+'_'+MAGIC, magic=MAGIC)

    logger_root.debug('BackTestDetail:TrainParameters:%s' %(str(TrainParametersList[i])))
    logger_root.debug('BackTestDetail:ModelName:%s' %(str(ModelNameList[i])))
    logger_root.info('BackTestDetail:%s_%s_%s_%s Parameters Loaded.' %(TRADE_SYMBOL,TRADE_PERIOD,TRADE_GRADE,MAGIC))

    # PriceDataは全体で共通のものをポイント参照
    # 個々のTrainDataを作成する際に、ポインタを渡す
    # 2024/6/8 Modelによってインプットする通貨ペアが変わるので、Accountファイルから読み込む
    price_inifile = configparser.ConfigParser()
    try:
      price_inifile.read(TrainParametersList[i].account)
      TRAIN_SYMBOL_LIST = eval(price_inifile.get('COMMOM','TRAIN_SYMBOL_LIST'))
    except configparser.NoOptionError:
      TRAIN_SYMBOL_LIST = ['EURUSD','USDJPY','GBPUSD','EURJPY','EURGBP','GBPJPY']

    logger_root.debug('BackTestDetail:TRAIN_SYMBOL_LIST=%s' %(TRAIN_SYMBOL_LIST))

    price_data_common = TrainDataMaker()
    # price_data_common.MakePriceData(TRAIN_PERIOD_LIST,TRAIN_SYMBOL_LIST)
    # price_data_common.MakePriceData([TRADE_PERIOD,DETAIL_CHECK_PERID],TRAIN_SYMBOL_LIST)
    price_data_common.MakePriceData([TRADE_PERIOD],TRAIN_SYMBOL_LIST)
    # 20240713 DetailCheckはテスト対象の通貨ペア分だけ作る
    price_data_common.MakePriceData([DETAIL_CHECK_PERID],[TRADE_SYMBOL])

    # TrainDataは都度作成する
    price_train_data = TrainDataMaker(PERIOD_RANDOM_SELECT, TrainParametersList[i])
    price_train_data.SetPeriodsDict(price_data_common.GetPeriodsDict())
    price_train_data.MakeTrainData(TRADE_PERIOD, TRADE_SYMBOL, INPUT_ALL_SYMBOLS_FLG)
    train_data_dict = price_train_data.GetTrainDataDict()
    periods_dict = price_train_data.GetPeriodsDict()
    periods = price_train_data.GetPeriods()

    s_format = '%Y%m%d%H%M%S'
    test_date_time = datetime.datetime.strptime(MAGIC, s_format)
    trnr = Trainer(EPISODE_NUM, TRADE_SYMBOL, TRADE_PERIOD, test_date_time, TEST_DURATION
                          , 1, 1, train_data_dict[TRADE_PERIOD], periods_dict, LOAD_FLAG, filepth=TrainParametersList[i],prds=periods,grd=TRADE_GRADE,mgc=MAGIC)

    df_backtest_result,df_backtest_result_detail = trnr.ExecBacktestDetail(BACKTEST_START_DATE,BACKTEST_END_DATE,BACKTEST_DURATION)
    df_backtest_result.to_csv(TRAIN_RESULT_PATH+'BacktestResultD_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+MAGIC+'.csv')
    df_backtest_result_detail.to_csv(TRAIN_RESULT_PATH+'BacktestResultDetail_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+MAGIC+'.csv')
    df_models['sum'] = df_backtest_result['sum']
    df_models = df_models.rename(columns={'sum':ModelNameList[i].model_name})
    logger_root.info('BackTestDetail:%s_%s_%s_%s Backtest Finished.' %(TRADE_SYMBOL,TRADE_PERIOD,TRADE_GRADE,MAGIC))
    # それぞれの辞書を開放する
    # prices_dict.clear()
    # periods_dict.clear()
    train_data_dict.clear()

# MT5との接続を終了する
  mt5.shutdown()
  logger_root.debug('MT5 ShutDown Completed.')

  logger_root.debug('BackTestDetail:df_models %s' %(df_models))
  df_models.to_csv(TRAIN_RESULT_PATH+BACKTEST_RESULT_ALL_FILE_NAME)
  logger_root.info('BackTestDetail:All Synbol and Period Backtest Finished.')

In [ ]:
price_data_common

In [ ]:
TICKVOL_PERIOD

## MakeTrainData and Construct a Portfolio
### 処理の流れ
- TrainディレクトリにあるNN Modelパラメータ(.pthファイル)を取得する
- 2018-2022のバックテストを行う。結果をCSVファイルに出力する
- バックテストの結果をもとに、効率的フロンティアを作成しポートフォリオを構成する

### 今後の追加機能
- 特定の時間足だけを対象にポートフォリオを作成する
- バックテスト期間を設定できるようにする
  - 直前週から過去１年間など

In [ ]:
ACCOUNT_TRADE_MODE = ACCOUNT_TRADE_MODE_REAL #@param ["ACCOUNT_TRADE_MODE_DEMO", "ACCOUNT_TRADE_MODE_CONTEST", "ACCOUNT_TRADE_MODE_REAL"] {type:"raw"}
TEST_SYMBOL = "*" # @param ["*", "EURUSD", "USDJPY", "GBPUSD", "EURJPY", "EURGBP", "GBPJPY", "GBPAUD", "GBPNZD"]
TEST_PERIOD = "*" #@param ["*","D1", "H12", "H8", "H6","H4", "H1", "M30", "M5", "M1"]
TEST_GRADE = "*" #@param ["*","G","S","B","[GS]","[SB]","[BG]"]
TEST_MAGIC = "*" #@param {type:"string"}
# TRAIN_SYMBOL_LIST = ['EURUSD','USDJPY','GBPUSD','EURJPY','EURGBP','GBPJPY']
TRAIN_SYMBOL_LIST = None
TRADE_SYMBOL = '_' # ダミー
TRADE_PERIOD = '_' #ダミー
TRADE_GRADE = '_' # ダミー

DETAIL_CHECK_PERID = 'M1'
INPUT_ALL_SYMBOLS_FLG = True
# periods_dict = {}
# prices_dict = {}
# train_dataを時間足毎に格納する辞書
# train_data_dict = {}
# TrainディレクトリにあるNN Modelパラメータを取得する
# 1つのエージェントに対して繰り返す訓練の数
## EPISODE_NUMは辞書で持って、TRADE_PERIODに対応して取り出すか。
EPISODE_NUM = 200 #@param {type:"integer"}
# テスト期間は1episodeの長さ。1日(D)、1週間(W)、1年(Y)を選択する
TEST_DURATION = 'W' #@param ['D', 'W', 'Y']
BACKTEST_START_YEAR = 2021 #@param {type:"slider", min:2018, max:2030, step:1}
BACKTEST_END_YEAR = 2023 #@param {type:"slider", min:2018, max:2030, step:1}
BACKTEST_DURATION = 'W' #@param ['D', 'W', 'Y']
LOAD_FLAG = True
# LONG,SHORT,TICKVO+_PERIODをランダムに選択するか
PERIOD_RANDOM_SELECT = False
# ファイルパスを設定する
EnvironmentCommon.setTradeModePath(ACCOUNT_TRADE_MODE)
nn_model_params=glob.glob('%sModel_%s_%s_%s_%s.pth'%(TRAIN_MODEL_PATH,TEST_SYMBOL,TEST_PERIOD,TEST_GRADE,TEST_MAGIC))
# nn_model_params=glob.glob('%sModel_EURUSD_H1_20230327130606.pth'%(TRAIN_MODEL_PATH))

TrainParameters = namedtuple('TrainParameters', ['account','agent','brain'])
TrainParametersList=[TrainParameters(None,None,None)]*len(nn_model_params)
# ModelNameとMAGICを格納するNamedTaple
ModelName = namedtuple('ModelName',['model_name','magic'])
ModelNameList = [ModelName(None,None)]*len(nn_model_params)

# backtestの結果を格納するDataFrame
df_models = pd.DataFrame()

# Protfolioファイルの読み込み
portfolio = configparser.ConfigParser()

# MT5との接続を開始する
logger_root.debug("MT5 initializing:%s" %(MT5_PATH))
res = mt5.initialize(MT5_PATH)
logger.debug("MT5 initialized result:%s" %(res))

# NN Modelのパラメータを読み込む
for i, mprm in enumerate(nn_model_params):
  periods_dict = {}
  prices_dict = {}
  train_data_dict = {}

  s = mprm.split('_') # Model_GBPJPY_H6_B_20230908223249.pth
  TRADE_SYMBOL = s[2]
  TRADE_PERIOD = s[3]
  TRADE_GRADE = s[4]
  MAGIC = s[5].split('.')[0]
  TrainParametersList[i] = TrainParameters(account=TRAIN_MODEL_PATH+'Account_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+TRADE_GRADE+'_'+MAGIC+'.ini',
                                agent=TRAIN_MODEL_PATH+'Agent_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+TRADE_GRADE+'_'+MAGIC+'.ini',
                                brain=mprm)
  ModelNameList[i] = ModelName(model_name=TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+TRADE_GRADE+'_'+MAGIC, magic=MAGIC)

  logger_root.debug('ConstructPortfolio:TrainParameters:%s' %(str(TrainParametersList[i])))
  logger_root.debug('ConstructPortfolio:ModelName:%s' %(str(ModelNameList[i])))
  logger_root.info('ConstructPortfolio:%s_%s_%s_%s Parameters Loaded.' %(TRADE_SYMBOL,TRADE_PERIOD,TRADE_GRADE,MAGIC))

  # 価格データ及びトレーニングデータを作成する際に
  # Modelごとに、Long,Short,Tickvol_periodの値が変わるため、その都度初期化する
  # 時間足毎×通貨ペア毎の価格データを辞書（price_dict)に格納する
  # LONG_PERIOD,SHORT_PERIOD,TICKVOL_PRIOD はエージェント毎に取得する
  # ここはリファクタリング対象
  periods = [LONG_PERIOD,SHORT_PERIOD,TICKVOL_PERIOD]
  logger.debug('ConstructPortfolio: INIFile loading. \n %s' %(TrainParametersList[i].account))
  # Account.iniに書き込まれているLONG_PERIOD,SHORT_PERIOD,TICKVOL_PRIOを読み込む
  # 2024/6/8 Modelによってインプットする通貨ペアが変わるので、Accountファイルから読み込む
  acc_inifile = configparser.ConfigParser()
  acc_inifile.read(TrainParametersList[i].account)
  '''
  2024/7/20 ここは、TrainDataMakerで行う
  try:
    periods[0] = int(acc_inifile.get('COMMOM', 'LONG_PERIOD'))
    periods[1] = int(acc_inifile.get('COMMOM', 'SHORT_PERIOD'))
    periods[2] = int(acc_inifile.get('COMMOM', 'TICKVOL_PERIOD'))
  except configparser.NoOptionError:
    # Account.iniに設定がない場合は、デフォルトの値を使う
    logger.warning('PriceData.__init__:No Such Option Found.')
  '''
  try:
    TRAIN_SYMBOL_LIST = eval(acc_inifile.get('COMMOM','TRAIN_SYMBOL_LIST'))
  except configparser.NoOptionError:
    TRAIN_SYMBOL_LIST = ['EURUSD','USDJPY','GBPUSD','EURJPY','EURGBP','GBPJPY']
  logger_root.debug('ConstructPortfolio:TRAIN_SYMBOL_LIST=%s' %(TRAIN_SYMBOL_LIST))

  '''
  2024/7/20 以下の処理はTrainDataMakerで行う
  TRAIN_PERIOD_LIST = [TRADE_PERIOD,DETAIL_CHECK_PERID]
  for prd in TRAIN_PERIOD_LIST:
    for sbl in TRAIN_SYMBOL_LIST:
      logger_root.info('Stab.MakeTrainDataAndTrainModel:Making %s, %s pricedata.' %(sbl,prd))
      # prices = PriceData(sbl,prd,periods)
      prices = PriceData(sbl,prd)
      prices.ConvertPriceDataFileToDataFrame()
      prices.AddStaticInfo(periods)
      if(periods_dict.setdefault(prd) is not None):
        prices_dict = periods_dict[prd]
      prices_dict[sbl] = prices
      periods_dict[prd] = prices_dict.copy()
      prices_dict.clear()
      logger_root.info('Stab.MakeTrainDataAndTrainModel:Finish %s, %s pricedata.' %(sbl,prd))
  # 訓練データとして、すべての通貨ペアを結合したものとのするか(True)、取引ペアのみとするか(False)
  for prd in TRAIN_PERIOD_LIST:
    if INPUT_ALL_SYMBOLS_FLG:
      logger_root.info('Stab.MakeTrainDataAndTrainModel:Making traindata. All %s pricedata combined.' %(prd))
      train_data_dict[prd] = TrainData(list(periods_dict[prd].values()), periods_dict[prd][TRADE_SYMBOL].price_list.index)
      logger_root.info('Stab.MakeTrainDataAndTrainModel:Finish traindata.')
    else:
      logger_root.info('Stab.MakeTrainDataAndTrainModel:Making %s, %s traindata.' %(TRADE_SYMBOL,prd))
      train_data_dict[prd] = TrainData([periods_dict[prd][TRADE_SYMBOL]])
      logger_root.info('Stab.MakeTrainDataAndTrainModel:Finish traindata.')

  '''
  # priceDataも都度時間足が変わるため都度作成する
  price_data_common = TrainDataMaker()
  price_data_common.MakePriceData([TRADE_PERIOD],TRAIN_SYMBOL_LIST)
  price_data_common.MakePriceData([DETAIL_CHECK_PERID],[TRADE_SYMBOL])

  # TrainDataは都度作成する
  price_train_data = TrainDataMaker(PERIOD_RANDOM_SELECT, TrainParametersList[i])
  price_train_data.SetPeriodsDict(price_data_common.GetPeriodsDict())
  price_train_data.MakeTrainData(TRADE_PERIOD, TRADE_SYMBOL, INPUT_ALL_SYMBOLS_FLG)
  train_data_dict = price_train_data.GetTrainDataDict()
  periods_dict = price_train_data.GetPeriodsDict()
  periods = price_train_data.GetPeriods()
  #
  s_format = '%Y%m%d%H%M%S'
  test_date_time = datetime.datetime.strptime(MAGIC, s_format)
  trnr = Trainer(EPISODE_NUM, TRADE_SYMBOL, TRADE_PERIOD, test_date_time, TEST_DURATION
                        , 1, 1, train_data_dict[TRADE_PERIOD], periods_dict, LOAD_FLAG, filepth=TrainParametersList[i],prds=periods,grd=TRADE_GRADE,mgc=MAGIC)

  df_backtest_result = trnr.ExecBacktest(BACKTEST_START_YEAR,BACKTEST_END_YEAR,BACKTEST_DURATION)
  df_backtest_result.to_csv(TRAIN_RESULT_PATH+'BacktestResultP_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+TRADE_GRADE+'_'+MAGIC+'.csv')
  df_models['sum'] = df_backtest_result['sum']
  df_models = df_models.rename(columns={'sum':ModelNameList[i].model_name})
  logger_root.info('ConstructPortfolio:%s_%s_%s_%s Backtest Finished.' %(TRADE_SYMBOL,TRADE_PERIOD,TRADE_GRADE,MAGIC))

  # メモリを開放する
  del prices_dict
  del periods_dict
  del train_data_dict
  gc.collect()

# MT5との接続を終了する
mt5.shutdown()
logger_root.debug('MT5 ShutDown Completed.')

logger_root.debug('ConstructPortfolio:df_models %s' %(df_models))
df_models.to_csv(TRAIN_RESULT_PATH+BACKTEST_ALL_FILE_NAME)
logger_root.info('ConstructPortfolio:All Synbol and Period Backtest Finished.')

In [ ]:
test_dt = datetime.datetime(2021,1,8)

In [ ]:
trnr.agnt.df_reward['q_taken'][1:-1]

In [ ]:
trnr.agnt.df_reward.iloc[1:-1,trnr.agnt.df_reward.columns.get_loc('q_taken')]

In [ ]:
ACCOUNT_TRADE_MODE = ACCOUNT_TRADE_MODE_DEMO #@param ["ACCOUNT_TRADE_MODE_DEMO", "ACCOUNT_TRADE_MODE_CONTEST", "ACCOUNT_TRADE_MODE_REAL"] {type:"raw"}
# ファイルパスを設定する
EnvironmentCommon.setTradeModePath(ACCOUNT_TRADE_MODE)
# portfolio系から得たい1週間当たりのpips(1,000pips =1.0)
# Gold model:1.0 Silver model:0.5 Bronz model:0.25
TARGET_EXPECTED_VALUE = 0.4 # @param {type:"raw"}
# 効率的フロンティアの計算とポートフォリオの構築
pt_const = PortfolioConstructer(TARGET_EXPECTED_VALUE)
# CSVファイルからDataFrameを作成
df_models = pd.read_csv(TRAIN_RESULT_PATH+BACKTEST_ALL_FILE_NAME, index_col=0)
model_portfolio = pt_const.ConstructPortfolio(df_models)
pt_const.SavePortfolioFile(model_portfolio)

# ポートフォリオの割合をファイルに書き出す

# トレードに使うNN ModelをTradeフォルダに移動する。現在TradeフォルダにあるModelは削除する

# Mrtatrader5に読み込ませる.setファイルを作成する。もしくは作成済みの.setファイルをTradeフォルダに移動する
# ※これは後程実装

In [ ]:
model_portfolio

In [ ]:
pt_const.ef.efficient_return(target_return=0.4)

In [ ]:
# ポートフォリオを最適化する
# 1. MT5の口座残高、成長率、ポートフォリオの収益力から投資可能ロット数を計算する
# 2. 投資可能ロット数×100以上のNNモデルがある場合は、投資比率の高い投資可能ロット数×100まで削る
# 　→例えば、投資可能ロット数が0.05の場合は、NNモデルを5つ(以下)以下にする

In [ ]:
# 効率的フロンティアを描画する
pt_const.DrawEfficientFrontier(df_models)

In [ ]:
# このモジュールを実行する前に、Make train dataを実行する
# TrainディレクトリにあるNN Modelパラメータを取得する
# 1つのエージェントに対して繰り返す訓練の数
## EPISODE_NUMは辞書で持って、TRADE_PERIODに対応して取り出すか。
EPISODE_NUM = 200 #@param {type:"integer"}
# テスト期間は1episodeの長さ。1日(D)、1週間(W)、1年(Y)を選択する
TEST_DURATION = 'W' #@param ['D', 'W', 'Y']
BACKTEST_START_YEAR = 2018 #@param {type:"slider", min:2018, max:2030, step:1}
BACKTEST_END_YEAR = 2022 #@param {type:"slider", min:2018, max:2030, step:1}
BACKTEST_DURATION = 'W' #@param ['D', 'W', 'Y']
LOAD_FLAG = True
ACCOUNT_TRADE_MODE = ACCOUNT_TRADE_MODE_DEMO #@param ["ACCOUNT_TRADE_MODE_DEMO", "ACCOUNT_TRADE_MODE_CONTEST", "ACCOUNT_TRADE_MODE_REAL"] {type:"raw"}
# ファイルパスを設定する
EnvironmentCommon.setTradeModePath(ACCOUNT_TRADE_MODE)
nn_model_params=glob.glob('%sModel_*_*_*.pth'%(TRAIN_MODEL_PATH))
# nn_model_params=glob.glob('%sModel_EURUSD_H1_20230327130606.pth'%(TRAIN_MODEL_PATH))

TrainParameters = namedtuple('TrainParameters', ['account','agent','brain'])
TrainParametersList=[TrainParameters(None,None,None)]*len(nn_model_params)
# ModelNameとMAGICを格納するNamedTaple
ModelName = namedtuple('ModelName',['model_name','magic'])
ModelNameList = [ModelName(None,None)]*len(nn_model_params)

# backtestの結果を格納するDataFrame
df_models = pd.DataFrame()



# NN Modelのパラメータを読み込む
for i, mprm in enumerate(nn_model_params):
  s = mprm.split('_') # Agent_EURJPY_H1_20230329180431.ini
  TRADE_SYMBOL = s[2]
  TRADE_PERIOD = s[3]
  MAGIC = s[4].split('.')[0]
  TrainParametersList[i] = TrainParameters(account=TRAIN_MODEL_PATH+'Account_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+MAGIC+'.ini',
                                agent=TRAIN_MODEL_PATH+'Agent_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+MAGIC+'.ini',
                                brain=mprm)
  ModelNameList[i] = ModelName(model_name=TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+MAGIC, magic=MAGIC)

  logger_root.debug('ConstructPortfolio:TrainParameters:%s' %(str(TrainParametersList[i])))
  logger_root.debug('ConstructPortfolio:ModelName:%s' %(str(ModelNameList[i])))
  logger_root.info('ConstructPortfolio:%s_%s_%s Parameters Loaded.' %(TRADE_SYMBOL,TRADE_PERIOD,MAGIC))

  #
  s_format = '%Y%m%d%H%M%S'
  test_date_time = datetime.datetime.strptime(MAGIC, s_format)
  trnr = Trainer(EPISODE_NUM, TRADE_SYMBOL, TRADE_PERIOD, test_date_time, TEST_DURATION
                        , 1, 1, train_data_dict[TRADE_PERIOD], periods_dict, LOAD_FLAG, TrainParametersList[i])

  df_backtest_result = trnr.ExecBacktest(BACKTEST_START_YEAR,BACKTEST_END_YEAR,BACKTEST_DURATION)
  df_backtest_result.to_csv(TRAIN_RESULT_PATH+'BacktestResultP_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+MAGIC+'.csv')
  df_models['sum'] = df_backtest_result['sum']
  df_models = df_models.rename(columns={'sum':ModelNameList[i].model_name})
  logger_root.info('ConstructPortfolio:%s_%s_%s Backtest Finished.' %(TRADE_SYMBOL,TRADE_PERIOD,MAGIC))

logger_root.debug('ConstructPortfolio:df_models %s' %(df_models))
df_models.to_csv(TRAIN_RESULT_PATH+BACKTEST_ALL_FILE_NAME)
logger_root.info('ConstructPortfolio:All Synbol and Period Backtest Finished.')


In [ ]:
# 効率的フロンティアの計算とポートフォリオの構築
pt_const = PortfolioConstructer()
# CSVファイルからDataFrameを作成
df_models = pd.read_csv(TRAIN_RESULT_PATH+BACKTEST_ALL_FILE_NAME, index_col=0)
model_portfolio = pt_const.ConstructPortfolio(df_models)
pt_const.SavePortfolioFile(model_portfolio)

# ポートフォリオの割合をファイルに書き出す

# トレードに使うNN ModelをTradeフォルダに移動する。現在TradeフォルダにあるModelは削除する

# Mrtatrader5に読み込ませる.setファイルを作成する。もしくは作成済みの.setファイルをTradeフォルダに移動する
# ※これは後程実装

In [ ]:
# 効率的フロンティアを描画する
pt_const.DrawEfficientFrontier(df_models)

In [ ]:
if __name__ == '__main__':

  TRAIN_SYMBOL_LIST = ['EURUSD','USDJPY','GBPUSD','EURJPY','EURGBP','GBPJPY']
  # TRAIN_SYMBOL_LIST = ['EURUSD','USDJPY','GBPUSD']
  TRADE_SYMBOL = "GBPUSD" #@param ["EURUSD", "USDJPY", "GBPUSD", "EURJPY", "EURGBP", "GBPJPY"]
  TRADE_PERIOD = "H1" #@param ["D1", "H6","H4", "H1", "M30", "M5", "M1"]
  DETAIL_CHECK_PERID = 'M1'
  TRAIN_PERIOD_LIST = ['D1','H4','H1','M30',DETAIL_CHECK_PERID]
  INPUT_ALL_SYMBOLS_FLG = True
  periods_dict = {}
  prices_dict = {}
  # train_dataを時間足毎に格納する辞書
  train_data_dict = {}

  ACCOUNT_TRADE_MODE = ACCOUNT_TRADE_MODE_DEMO #@param ["ACCOUNT_TRADE_MODE_DEMO", "ACCOUNT_TRADE_MODE_CONTEST", "ACCOUNT_TRADE_MODE_REAL"] {type:"raw"}
  # ファイルパスを設定する
  EnvironmentCommon.setTradeModePath(ACCOUNT_TRADE_MODE)

  # TRADE_PERIODはTRAIN_PERIOD_LISTに含まれていること
  if TRADE_PERIOD in TRAIN_PERIOD_LIST:
    pass
  else:
    sys.exit('TRADE_PERIOD does not find in TRAIN_PERIOD_LIST')

  # 時間足毎×通貨ペア毎の価格データを辞書（price_dict)に格納する
  for prd in TRAIN_PERIOD_LIST:
    for sbl in TRAIN_SYMBOL_LIST:
      logger_root.info('Stab.MakeTrainData:Making %s, %s pricedata.' %(sbl,prd))
      prices = PriceData(sbl,prd)
      prices.ConvertPriceDataFileToDataFrame()
      prices.AddStaticInfo()
      if(periods_dict.setdefault(prd) is not None):
        prices_dict = periods_dict[prd]
      prices_dict[sbl] = prices
      periods_dict[prd] = prices_dict.copy()
      prices_dict.clear()
      logger_root.info('Stab.MakeTrainData:Finish %s, %s pricedata.' %(sbl,prd))


  # 訓練データとして、すべての通貨ペアを結合したものとのするか(True)、取引ペアのみとするか(False)
  for prd in TRAIN_PERIOD_LIST:
    if INPUT_ALL_SYMBOLS_FLG:
      logger_root.info('Stab.MakeTrainData:Making traindata. All %s pricedata combined.' %(prd))
      # train = TrainData(list(prices_dict.values()), prices_dict[TRADE_SYMBOL].price_list.index)
      # train_data = TrainData(list(periods_dict[TRADE_PERIOD].values()), periods_dict[TRADE_PERIOD][TRADE_SYMBOL].price_list.index)
      train_data_dict[prd] = TrainData(list(periods_dict[prd].values()), periods_dict[prd][TRADE_SYMBOL].price_list.index)
      logger_root.info('Stab.MakeTrainData:Finish traindata.')
    else:
      logger_root.info('Stab.MakeTrainData:Making %s, %s traindata.' %(TRADE_SYMBOL,prd))
      # train = TrainData([prices_dict[TRADE_SYMBOL]])
      # train_data = TrainData([periods_dict[TRADE_PERIOD][TRADE_SYMBOL]])
      train_data_dict[prd] = TrainData([periods_dict[prd][TRADE_SYMBOL]])
      logger_root.info('Stab.MakeTrainData:Finish traindata.')




In [ ]:
PRICEDATA_PATH

In [ ]:
  # テスト用に特定の時間足を取得するようにする
  # 現在時刻と、直近の時間足データを取得するようにする
  # 2019/12/27 22:00:00 時点で、21:00:00足を取得する
  # 実際にはTimeDeltaを使うようにする
  # test_date_time = datetime.datetime(2018,1,10,15)
  # delta_time = datetime.timedelta(hours=1)
  delta_time = EnvironmentCommon.GetPeriodTimeDelta(TRADE_PERIOD)
  # CheckTpLcで、詳細にTpLcをチェックするときの時間足

  LOAD_FLAG = True
  FILE_PATH = None

  # 1つの世代で作成するTrainAgentの数
  TRAIN_AGENT_NUM = 5 #@param {type:"number"}
  # 次の世代に行けるTrainAgentの数(上位)
  AGENT_TOGO_NEXT_NUM = 3 #@param {type:"number"}
  # 訓練を最大何世代行うか。最大世代数までに条件にあうTrainAgent(聖杯！）が見つかれば訓練中断
  MAX_GEN_NUM = 5 #@param {type:"number"}
  # 1つのエージェントに対して繰り返す訓練の数
  ## EPISODE_NUMは辞書で持って、TRADE_PERIODに対応して取り出すか。
  EPISODE_NUM = 200 #@param {type:"integer"}
  # テスト期間は1episodeの長さ。1日(D)、1週間(W)、1年(Y)を選択する
  TEST_DURATION = 'W' #@param ['D', 'W', 'Y']
  BACKTEST_START_YEAR = 2018 #@param {type:"slider", min:2018, max:2030, step:1}
  BACKTEST_END_YEAR = 2022 #@param {type:"slider", min:2018, max:2030, step:1}
  BACKTEST_DURATION = 'W' #@param ['D', 'W', 'Y']

  # 世代別TradeAgent事のBacktestの結果とObjectを格納するDataFrame
  # result:Backtestの損益の合計、trade_avg:1週間当たりの平均損益、std:1週間当たりの損益の標準偏差
  # cdp_0:pips=0の時の累積確率密度関数(cumulative distribution probability function)の値、agt_obj:TradeAgentのObject
  df_gen_rank = pd.DataFrame(columns=['gen','agt_num','result','trade_avg','std','cdp_0','eval','agt_obj'])
  # ParameterFileは、AccountのINIファイルとAgentのINIファイル、BrainのPTHファイルがある
  # それぞれのファイルパスをランクごとにNamedTupleに格納する
  # TrainParametersList = [None] * TRAIN_AGENT_NUM
  TrainParameters = namedtuple('TrainParameters', ['account','agent','brain'])
  TrainParametersList=[TrainParameters(None,None,None)]*TRAIN_AGENT_NUM
  # Trainerを格納するリスト
  AgentList = [None] * TRAIN_AGENT_NUM
  # ModelNameとMAGICを格納するNamedTaple
  ModelName = namedtuple('ModelName',['model_name','magic','set_file_path'])

  for gnum in range(MAX_GEN_NUM):
    # 訓練する世代のNN Paramファイルが存在していれば、それを取得し
    # 当該世代の訓練をスキップする
    # ParameterFileは、AccountのINIファイルとAgentのINIファイル、BrainのPTHファイルがある
    # 各世代において、それぞれのファイルの数が同じかを確認し、同じであればRank毎にNamedTupleを生成する
    tmp_account_params=glob.glob('%s%s_%s_Account_Gen%s_Rank*.ini'%(TMP_TRAIN_MODEL_PATH,TRADE_SYMBOL,TRADE_PERIOD,gnum+1))
    logger_root.debug('tmp_account_params:%s' %(tmp_account_params))
    tmp_agent_params=glob.glob('%s%s_%s_Agent_Gen%s_Rank*.ini'%(TMP_TRAIN_MODEL_PATH,TRADE_SYMBOL,TRADE_PERIOD,gnum+1))
    logger_root.debug('tmp_agent_params:%s' %(tmp_agent_params))
    tmp_brain_params=glob.glob('%s%s_%s_Brain_Gen%s_Rank*.pth'%(TMP_TRAIN_MODEL_PATH,TRADE_SYMBOL,TRADE_PERIOD,gnum+1))
    logger_root.debug('tmp_brain_params:%s' %(tmp_brain_params))

    # AccounのINIファイルと、AgentのINIファイル、BrainのPTHファイルの数があっている場合
    # ランクごとにパラメータファイルを読み込む
    if len(tmp_account_params) > 0 and len(tmp_account_params)==len(tmp_agent_params) and len(tmp_brain_params)==len(tmp_account_params):
      TrainParametersList.clear()
      TrainParametersList=[TrainParameters(None,None,None)]*TRAIN_AGENT_NUM
      # 当該世代のモデルファイルを読み込む
      for rnk in range(len(tmp_account_params)):
        TrainParametersList[rnk] = TrainParameters(account=TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Account_Gen'+str(gnum+1)+'_Rank'+str(rnk+1)+'.ini',
                                      agent=TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Agent_Gen'+str(gnum+1)+'_Rank'+str(rnk+1)+'.ini',
                                      brain=TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Brain_Gen'+str(gnum+1)+'_Rank'+str(rnk+1)+'.pth')
        logger_root.debug('TrainParameters:%s' %(str(TrainParametersList[rnk])))
        logger_root.info('TrainModel:Generation %s Parameters Loaded.' %(str(gnum+1)))

      # 次の世代へ
      continue

    # 訓練する時間(開始日)をランダムに設定する。
    # どの訓練期間であっても、年は共通(2018～2022)
    YEAR = random.randint(2018,2022)
    if(TEST_DURATION == 'Y'):
      # 訓練期間が年単位の場合は、第2週目の月曜日(weekday=1)から
      WEEKNUM = 2
      WEEKDAY = 1
    elif(TEST_DURATION == 'W'):
      # 訓練期間が週単位の場合は、2～51週(ランダム)の月曜日(weekday=1)から
      WEEKNUM = random.randint(2,51)
      WEEKDAY = 1
    elif(TEST_DURATION == 'D'):
      # 訓練期間が日単位の場合は、2～51週(ランダム)の月曜日(weekday=1)～金曜日(weekday=5)(ランダム)
      WEEKNUM = random.randint(2,51)
      WEEKDAY = random.randint(1,5)
    else:
      sys.exit('This period is not implemented.')

    # fromisocalendar()は、python3.8以降でないと実装されていない
    # 月曜日(weekday=1)と最終週の日曜日(weekday=7)
    test_date_time = datetime.datetime.fromisocalendar(YEAR,WEEKNUM,WEEKDAY)
    logger_root.info('Train Model:◆◆Generation %d Start Training.' %(gnum+1))

    for p, prmlst in enumerate(TrainParametersList):
      logger_root.debug('TrainModel: Gen %d, Agent %d, Start Training.' %(gnum+1, p+1))
      # AgentListの中身がNoneの場合に新たにTrainerAgentを作成する
      # その時、TrainParameterにファイルパスが格納されている場合はファイルをロードする
      # TrainParameterの値がNoneの場合は、新規作成
      if AgentList[p] == None:
        AgentList[p] = Trainer(EPISODE_NUM, TRADE_SYMBOL, TRADE_PERIOD, test_date_time, TEST_DURATION
                        , gnum+1, p+1, train_data_dict[TRADE_PERIOD], periods_dict, LOAD_FLAG, prmlst)
      else:
        # AgentList[p]にオブジェクトがある場合(前世代からの生き残り)でも、訓練期間(test_date_time)を更新する
        AgentList[p].UpdateTradePeriod(test_date_time,TEST_DURATION)

      df_result = AgentList[p].TrainAgent(epinum=EPISODE_NUM, train_mode=True)
      logger_root.info('■□■TrainModel:Gen %d, Agent %d, Train Finish.■□■\n %s' %(gnum+1, p+1,df_result.tail()))
      df_result.to_csv(TMP_TRAIN_RESULT_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Gen'+str(gnum+1)+'_Agent'+str(p+1)+'_TrainResult.csv')

      df_backtest_result = AgentList[p].ExecBacktest(BACKTEST_START_YEAR,BACKTEST_END_YEAR,BACKTEST_DURATION)
      # トレーニングの結果をdf_gen_rankに登録する
      # ['gen','agt_num','result','trade_avg','std','cdp_0','agt_obj']
      df_gen_rank.loc['%s%s'%(gnum+1, p+1)]=[gnum+1,p+1, df_backtest_result['sum'].sum(),df_backtest_result['sum'].mean(),
                                             df_backtest_result['sum'].std(),
                                             stats.norm(loc=df_backtest_result['sum'].mean(),scale=df_backtest_result['sum'].std()).cdf(0),
                                             df_backtest_result['sum'].mean()*(1-stats.norm(loc=df_backtest_result['sum'].mean(),scale=df_backtest_result['sum'].std()).cdf(0)),
                                             AgentList[p]]
      logger.info('TrainModel:Gen %d, Agent %d, BackTest Finish.\n %s' %(gnum+1, p+1,df_gen_rank.loc['%s%s'%(gnum+1, p+1)]))
      df_backtest_result.to_csv(TMP_TRAIN_RESULT_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Gen'+str(gnum+1)+'_Agent'+str(p+1)+'_BacktestResult.csv')
      # Exit Criteriaを満たしているかチェック
      # 1.年間平均週間獲得pips1,000(=1.000)以上かつ獲得pips0における累積分布確率関数が5%以下
      # if(df_gen_rank.loc['%s%s'%(gnum+1, p+1)]['trade_avg'] >1 and df_gen_rank.loc['%s%s'%(gnum+1, p+1)]['cdp_0'] < 0.05):
      # 1.年間平均週間獲得pips(千単位)と、[1-獲得pips0における累積分布確率関数]の積(eval)が0.95より大きい
      if(df_gen_rank.loc['%s%s'%(gnum+1, p+1)]['eval'] >0.95):
        # 【聖杯の発見！】
        logger_root.info('TrainModel:★☆★☆★Grail has found!★☆★☆★')
        # 1-1. ACCOUNT.INI、AGRNT.INI、MODEL.PTHファイルをMODELディレクトリに保存
        now_date_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        GrailParameters = TrainParameters(account=TRAIN_MODEL_PATH+'Account_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.ini',
                                          agent=TRAIN_MODEL_PATH+'Agent_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.ini',
                                          brain=TRAIN_MODEL_PATH+'Model_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.pth')
        GrailModelName = ModelName(model_name=TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time,
                                   magic=now_date_time,
                                   set_file_path=TRAIN_MODEL_PATH+'EA_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.set')
        AgentList[p].SaveParameters(GrailParameters,GrailModelName)
        df_gen_rank.loc['%s%s'%(gnum+1, p+1)].to_csv(TRAIN_RESULT_PATH+'Result_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.csv')
        df_result.to_csv(TRAIN_RESULT_PATH+'TrainResult_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.csv')
        df_backtest_result.to_csv(TRAIN_RESULT_PATH+'BacktestResult_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.csv')
      # 1-2. MODEL/TMP以下のファイルを削除
        '''テスト用に残しておく
        for p in glob.glob(TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'*'):
          if os.path.isfile(p):
              os.remove(p)
        '''
        sys.exit()
      else:
        logger_root.debug('TrainModel:Continue training.')

    # Rankの並べ替え、世代別に評価値(eval)の大きい順に並べ替える。その後、最新の世代のランキングだけ取り出す
    df_gen_rank.sort_values(['gen','eval'],ascending=[True,False],inplace=True)
    df_cur_gen_rank = df_gen_rank.query('gen == %d' %(gnum+1))
    # df_cur_gen_rank.reset_index(inplace=True, drop=True)
    logger_root.info('TrainModel:Generation %d train finished.\n%s' %(gnum+1,df_cur_gen_rank))
    df_cur_gen_rank.to_csv(TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Gen'+str(gnum+1)+'_Rank.csv')

    # AgentToGoの数だけ生き残り、生き残り分のACCOUNT.INI、AGRNT.INI、MODEL.PTHファイルをMODEL/TMPディレクトリに保存
    # 足りない分はNoneで見たす
    AgentList.clear()
    AgentList = [None] * TRAIN_AGENT_NUM
    TrainParametersList.clear()
    TrainParametersList=[TrainParameters(None,None,None)]*TRAIN_AGENT_NUM

    for rnk in range(AGENT_TOGO_NEXT_NUM):
      AgentList[rnk] = df_cur_gen_rank.iloc[rnk]['agt_obj']
      # ACCOUNT.INI、AGRNT.INI、MODEL.PTHファイルをMODEL/TMPディレクトリに保存
      TrainParametersList[rnk] = TrainParameters(account=TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Account_Gen'+str(gnum+1)+'_Rank'+str(rnk+1)+'.ini',
                                                 agent=TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Agent_Gen'+str(gnum+1)+'_Rank'+str(rnk+1)+'.ini',
                                                 brain=TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Brain_Gen'+str(gnum+1)+'_Rank'+str(rnk+1)+'.pth')
      AgentList[rnk].SaveParameters(TrainParametersList[rnk],None)

  # 所定の世代の訓練が完了したら、最後の世代の1位のModelが採用できるかチェックする
  # 採用条件：年間平均週間獲得pips500(=0.500)以上かつ獲得pips0における累積分布確率関数が40%以下
  # if(df_cur_gen_rank.iloc[0]['trade_avg'] >0.5 and df_cur_gen_rank.iloc[0]['cdp_0'] < 0.4):
  # 採用条件：年間平均週間獲得pips(千pips)と[1-獲得pips0における累積分布確率関数]の積が0.3より大きい
  # 2023/5/4 採用条件の変更0.3→0.6(μ=1.0かつ１σ1.0以内)
  if(df_cur_gen_rank.iloc[0]['eval'] >0.6):
    # モデルとして採用する
    logger_root.info('TrainModel:★★Model has got.★★')
    # ACCOUNT.INI、AGRNT.INI、MODEL.PTHファイルをMODELディレクトリに保存
    now_date_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    ModelParameters = TrainParameters(account=TRAIN_MODEL_PATH+'Account_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.ini',
                                      agent=TRAIN_MODEL_PATH+'Agent_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.ini',
                                      brain=TRAIN_MODEL_PATH+'Model_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.pth')
    ModelNNName = ModelName(model_name=TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time,
                            magic=now_date_time,
                            set_file_path=TRAIN_MODEL_PATH+'EA_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.set')
    AgentList[0].SaveParameters(ModelParameters,ModelNNName)
    df_cur_gen_rank.iloc[0].to_csv(TRAIN_MODEL_PATH+'Result_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.csv')
    # 採用したモデルの訓練データとバックテストデータをコピーする
    shutil.copyfile(TMP_TRAIN_RESULT_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Gen'+str(df_cur_gen_rank.iloc[0]['gen'])
                +'_Agent'+str(df_cur_gen_rank.iloc[0]['agt_num'])+'_TrainResult.csv',
                TRAIN_RESULT_PATH+'TrainResult_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.csv')
    shutil.copyfile(TMP_TRAIN_RESULT_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Gen'+str(df_cur_gen_rank.iloc[0]['gen'])
                +'_Agent'+str(df_cur_gen_rank.iloc[0]['agt_num'])+'_BacktestResult.csv',
                TRAIN_RESULT_PATH+'BacktestResult_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.csv')

  else:
    # 採用できるモデルはなかった
    logger_root.info('TrainModel:▼▼No Model has found.▼▼')

  # 1-2. MODEL/TMP以下のファイルを削除
  ''' テスト用にTMPファイルを残しておく
  for p in glob.glob(TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'*'):
    if os.path.isfile(p):
        os.remove(p)
  '''
  logger_root.info('TrainModel:TMP File Deleted.')
  logger_root.info('TrainModel:Train has finished.')

In [ ]:
# このモジュールを実行する前に、Make train dataを実行する
# TrainディレクトリにあるNN Modelパラメータを取得する
# 1つのエージェントに対して繰り返す訓練の数
## EPISODE_NUMは辞書で持って、TRADE_PERIODに対応して取り出すか。
EPISODE_NUM = 200 #@param {type:"integer"}
# テスト期間は1episodeの長さ。1日(D)、1週間(W)、1年(Y)を選択する
TEST_DURATION = 'W' #@param ['D', 'W', 'Y']
BACKTEST_START_DATE = '2023-06-12' #@param {type:"date"}
BACKTEST_END_DATE = '2023-06-16' #@param {type:"date"}
BACKTEST_DURATION = 'W' #@param ['D', 'W', 'Y']
LOAD_FLAG = True

ACCOUNT_TRADE_MODE = ACCOUNT_TRADE_MODE_DEMO #@param ["ACCOUNT_TRADE_MODE_DEMO", "ACCOUNT_TRADE_MODE_CONTEST", "ACCOUNT_TRADE_MODE_REAL"] {type:"raw"}
# ファイルパスを設定する
EnvironmentCommon.setTradeModePath(ACCOUNT_TRADE_MODE)

TEST_SYMBOL = "GBPUSD" #@param ["*","EURUSD", "USDJPY", "GBPUSD", "EURJPY", "EURGBP", "GBPJPY"]
TEST_PERIOD = "H1" #@param ["*","D1", "H6","H4", "H1", "M30", "M5", "M1"]
TEST_MAGIC = "20230329061823" #@param {type:"string"}
# For test
nn_model_params=glob.glob('%sModel_%s_%s_%s.pth'%(TRAIN_MODEL_PATH,TEST_SYMBOL,TEST_PERIOD,TEST_MAGIC))
# nn_model_params=glob.glob('%sModel_*_*_*.pth'%(TRAIN_MODEL_PATH))

TrainParameters = namedtuple('TrainParameters', ['account','agent','brain'])
TrainParametersList=[TrainParameters(None,None,None)]*len(nn_model_params)
# ModelNameとMAGICを格納するNamedTaple
ModelName = namedtuple('ModelName',['model_name','magic'])
ModelNameList = [ModelName(None,None)]*len(nn_model_params)

# backtestの結果を格納するDataFrame
df_models = pd.DataFrame()

# NN Modelのパラメータを読み込む
for i, mprm in enumerate(nn_model_params):
  s = mprm.split('_') # Agent_EURJPY_H1_20230329180431.ini
  TRADE_SYMBOL = s[2]
  TRADE_PERIOD = s[3]
  MAGIC = s[4].split('.')[0]
  TrainParametersList[i] = TrainParameters(account=TRAIN_MODEL_PATH+'Account_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+MAGIC+'.ini',
                                agent=TRAIN_MODEL_PATH+'Agent_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+MAGIC+'.ini',
                                brain=mprm)
  ModelNameList[i] = ModelName(model_name=TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+MAGIC, magic=MAGIC)

  logger_root.debug('BackTestDetail:TrainParameters:%s' %(str(TrainParametersList[i])))
  logger_root.debug('BackTestDetail:ModelName:%s' %(str(ModelNameList[i])))
  logger_root.info('BackTestDetail:%s_%s_%s Parameters Loaded.' %(TRADE_SYMBOL,TRADE_PERIOD,MAGIC))

  #
  s_format = '%Y%m%d%H%M%S'
  test_date_time = datetime.datetime.strptime(MAGIC, s_format)
  trnr = Trainer(EPISODE_NUM, TRADE_SYMBOL, TRADE_PERIOD, test_date_time, TEST_DURATION
                        , 1, 1, train_data_dict[TRADE_PERIOD], periods_dict, LOAD_FLAG, TrainParametersList[i])

  df_backtest_result,df_backtest_result_detail = trnr.ExecBacktestDetail(BACKTEST_START_DATE,BACKTEST_END_DATE,BACKTEST_DURATION)
  df_backtest_result.to_csv(TRAIN_MODEL_PATH+'BacktestResultD_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+MAGIC+'.csv')
  df_backtest_result_detail.to_csv(TRAIN_MODEL_PATH+'BacktestResultDetail_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+MAGIC+'.csv')
  df_models['sum'] = df_backtest_result['sum']
  df_models = df_models.rename(columns={'sum':ModelNameList[i].model_name})
  logger_root.info('BackTestDetail:%s_%s_%s Backtest Finished.' %(TRADE_SYMBOL,TRADE_PERIOD,MAGIC))

logger_root.debug('BackTestDetail:df_models %s' %(df_models))
df_models.to_csv(TRAIN_MODEL_PATH+BACKTEST_ALL_FILE_NAME)
logger_root.info('BackTestDetail:All Synbol and Period Backtest Finished.')

# SandBox

## ファイルの書き出し、取り込みのテスト
- 結果ファイルを書き出す
- ファイルを取り込んで、新しい結果のほうが良ければ入れ替え。その時古いファイルはvoidフォルダへ
- ファイルを取り込んで、新しい結果よりもともとの結果が良ければそのまま

In [ ]:
import glob
import os
TRADE_SYMBOL = 'USDJPY'
TRADE_PERIOD = 'D1'
FILE_PATH=r'D:\ColabNotebooks\06_DuelNetTradingSystem2025\Demo\Model\Train'+'\\'
l = glob.glob(FILE_PATH+'Agent_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_*.ini')
print(l)
print('FileCount:'+str(len(l)))

In [ ]:
import glob
import os
import configparser
import datetime
import shutil

# ファイルの書き出し
TRADE_SYMBOL = 'USDJPY'
TRADE_PERIOD = 'D1'
FILE_PATH=r'D:\ColabNotebooks\06_DuelNetTradingSystem2025\Demo\Model\Train'+'\\'
DIR_PATH_VOID=r'D:\ColabNotebooks\06_DuelNetTradingSystem2025\Demo\Model\Train\Void'+'\\'
now_date_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# FILE_NAME_ACCOUNT:Account_CHFJPY_D1_B_20240721000428.ini
FILE_NAME_ACCOUNT='Account_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_B_'+now_date_time+'.ini'
# FILE_NAME_AGENT:Agent_CHFJPY_H6_B_20240721212832.ini
FILE_NAME_AGENT='Agent_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_B_'+now_date_time+'.ini'
# FILE_NAME_MODEL:Model_CHFJPY_H6_B_20240721212832.pth
FILE_NAME_MODEL='Model_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_B_'+now_date_time+'.pth'
# FILE_NAME_SET:EA_GBPAUD_D1_B_20240616090106.set
FILE_NAME_SET='EA_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_B_'+now_date_time+'.set'

# ファイルに書き込むパラメータ
lc_level=1.2
tp_level=0.8
long_period=31
short_period=13
tickvol_period=23
pos_scale=0.75
TRAIN_SYMBOL_LIST=['EURUSD','USDJPY','GBPUSD','EURJPY','EURGBP','GBPJPY']
real_sprad_limit=30
pre_trade_weeks=2
est_float_pl=0.9
eval_value=0.69
#GBPAUD_D1_B_20240616090106
model_name=TRADE_SYMBOL+'_'+TRADE_PERIOD+'_B_'+now_date_time
MODEL_INPUT_NUM=75
MODEL_HIDDEN_NUM=75
MODEL_OUTPUT_NUM=6
magic=now_date_time
TRADE_SYSTEM='DuelNet2026'



# 同じ通貨ペアと時間足のファイルAgentファイルがあるかを確認する
# フォルダにある同じ通貨ペア、時間足のファイルを取得する
filelist = glob.glob(FILE_PATH+'Agent_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_*.ini')

# 既存ファイルのほうが結果が良い場合は1位のファイル名が入る
win_file=None

for l in filelist:
    print('AGENT FILE NAME:'+l)
    # AgentFileの内容を読みだしてeval_valueの値を取り出す
    inifile = configparser.ConfigParser()
    # .iniファイルのエンコード
    ENC = 'UTF-8'
    inifile.read(l,ENC)
    # eval_valueを取得する
    ev=float(inifile.get('COMMOM', 'EVAL_VALUE'))
    print('Evalvale:'+str(ev)+' ['+l+']')

    #新しいeval_valueと既存のevを比較する
    if ev >= eval_value:
        # 既存のほうが大きい場合は残すファイル名とeval_valueを更新する
        eval_value=ev
        win_file=l

# win_fileがNoneのであれば、既存ファイルをすべてvoidへ移して、新規ファイルを作成する
# win_fileに名前があれば、そのファイル以外はvoidへ移す
filelist = glob.glob(FILE_PATH+'*_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_*')
if win_file is None:
    for l in filelist:
        # voidへ移す。voidディレクトリが存在していないときはディレクトリを作成する
        if not os.path.isdir(DIR_PATH_VOID):
            print('Make Void Directry.')
            os.makedirs(DIR_PATH_VOID)
        new_path = shutil.move(l, DIR_PATH_VOID)
        print(l+' is moved to '+new_path)

    print('AccountFile Crate:'+(FILE_PATH+FILE_NAME_ACCOUNT))
    #AccountFile
    with open(FILE_PATH+FILE_NAME_ACCOUNT , mode='w') as f:
      f.write('[COMMOM]\n')
      f.write('LOSSCUT_LEVEL=' + str(lc_level) + '\n')
      f.write('TAKEPROFIT_LEVEL=' + str(tp_level) + '\n')
      f.write('LONG_PERIOD='+str(long_period)+'\n')
      f.write('SHORT_PERIOD='+str(short_period)+'\n')
      f.write('TICKVOL_PERIOD='+str(tickvol_period)+'\n')
      f.write('POS_SCALE='+str(pos_scale)+'\n')
      f.write('TRAIN_SYMBOL_LIST='+str(TRAIN_SYMBOL_LIST)+'\n')
      f.write('REAL_SPREAD_LIMIT='+str(real_sprad_limit)+'\n')
      f.write('PRE_TRADE_WEEKS='+str(pre_trade_weeks)+'\n')
    
    #AgentFile
    with open(FILE_PATH+FILE_NAME_AGENT , mode='w') as f:
      f.write('[COMMOM]\n')
      f.write('ESTIMATE_FLOAT_PL = ' + str(est_float_pl) + '\n')
      f.write('EVAL_VALUE = ' + str(eval_value) + '\n')
    
    #NNModelFile
    with open(FILE_PATH+FILE_NAME_MODEL , mode='w') as f:
      f.write('## This File is Dummy.\n')
    
    #SetFile
    with open(FILE_PATH+FILE_NAME_SET , mode='w') as f:
      f.write('MODEL_NAME='+model_name+'\n')
      f.write('LC_LEVEL=' + str(lc_level) + '\n')
      f.write('TP_LEVEL=' + str(tp_level) + '\n')
      f.write('LONG_PERIOD='+str(long_period)+'\n')
      f.write('SHORT_PERIOD='+str(short_period)+'\n')
      f.write('TICKVOL_PERIOD='+str(tickvol_period)+'\n')
      f.write('POS_SCALE='+str(pos_scale)+'\n')
      f.write('MODEL_INPUT_NUM='+str(MODEL_INPUT_NUM)+'\n')
      f.write('MODEL_HIDDEN_NUM='+str(MODEL_HIDDEN_NUM)+'\n')
      f.write('MODEL_OUTPUT_NUM='+str(MODEL_OUTPUT_NUM)+'\n')
      f.write('TRAIN_SYMBOL_LIST='+str(TRAIN_SYMBOL_LIST)+'\n')
      f.write('MAGIC='+magic+'\n')
      f.write('TRADE_SYSTEM='+TRADE_SYSTEM+'\n')
      f.write('REAL_SPREAD_LIMIT='+str(real_sprad_limit)+'\n')
      f.write('PRE_TRADE_WEEKS='+str(pre_trade_weeks)+'\n')
else:
    #win_file以外をvoidへ移す
    print('WinFile:'+win_file)
    # D:\ColabNotebooks\06_DuelNetTradingSystem2025\Demo\Model\Train\Agent_USDJPY_D1_B_20260125225640.ini
    MAGIC = win_file.split('\\')[-1].split('_')[4].split('.')[0] 
    print('MAGIC:'+MAGIC)
    for l in filelist:
        print('File:'+l)
        # MAGICが同じファイルは移動しない
        if MAGIC not in l:
            # voidへ移す。voidディレクトリが存在していないときはディレクトリを作成する
            if not os.path.isdir(DIR_PATH_VOID):
                print('Make Void Directry.')
                os.makedirs(DIR_PATH_VOID)
            new_path = shutil.move(l, DIR_PATH_VOID)
            print(l+' is moved to '+new_path)

## pd.date_rangeのテスト

In [ ]:
import pandas as pd
from datetime import datetime
from datetime import timedelta

start_date = datetime(2025,3,17,0,0)
to_date = start_date - timedelta(weeks=1)
from_date = start_date - timedelta(weeks=0)

print('from_date:%s, start_date:%s' %(from_date,start_date))
every_monday = pd.date_range(from_date, to_date, freq='W-MON')

for i,d in enumerate(every_monday):
  print(i,d)


## MetaTrader5との連携テスト

In [ ]:
from datetime import datetime
from datetime import timedelta
import MetaTrader5 as mt5
import pandas as pd
import pytz

Symbol ='GBPJPY'

# MetaTrader 5ターミナルとの接続を確立する
if not mt5.initialize():
   print("initialize() failed, error code =",mt5.last_error())
   exit()

# タイムゾーンをUTCに設定する
timezone = pytz.timezone("Etc/UTC")
# ローカルタイムゾーンオフセットの実装を回避するために、UTCタイムゾーンで「datetime」オブジェクトを作成する
utc_from = datetime(2019, 4, 12, 0, 0, tzinfo=timezone)
# utc_to = datetime(2020, 1, 10, 16, 1, tzinfo=timezone)
utc_to = utc_from + timedelta(days=1)
# 10.01.2019 UTCから開始して100 000 EURUSDティックをリクエストする
ticks_first = mt5.copy_ticks_from(Symbol, utc_from, 1, mt5.COPY_TICKS_ALL)
ticks_period = mt5.copy_ticks_range(Symbol, utc_from, utc_to, mt5.COPY_TICKS_ALL)
print("Ticks received:",len(ticks_first),len(ticks_period))

# MetaTrader 5ターミナルへの接続をシャットダウンする
mt5.shutdown()

In [ ]:
# 日足の最初のTickからBID(売値)、ASK(買値)を取得する
sell_price = 0.0
buy_price = 0.0
(sell_price,buy_price)=ticks_first[['bid','ask']].item()
print("Sell price:",sell_price,"Buy price:",buy_price)
tplc_level = 0.4
sell_tp = sell_price - tplc_level
sell_lc = sell_price + tplc_level
buy_tp = buy_price + tplc_level
buy_lc = buy_price - tplc_level
print("Sell TP:",sell_tp,"LC:",sell_lc,"Buy TP:",buy_tp,"LC:",buy_lc)

In [ ]:
# 取得したデータからDataFrameを作成する
ticks_frame = pd.DataFrame(ticks_period)
# 秒での時間をdatetime形式に変換する
ticks_frame['time']=pd.to_datetime(ticks_frame['time'], unit='s')
ticks_frame['time_msc']=pd.to_datetime(ticks_frame['time_msc'], unit='ms')
ticks_frame = ticks_frame.set_index('time_msc')

In [ ]:
ticks_frame.loc[[ticks_frame.index[0],ticks_frame.index[-1]],['ask','bid']]

In [ ]:
open_ask, open_bid=ticks_frame.loc[ticks_frame.index[0],['ask','bid']]
close_ask, close_bid=ticks_frame.loc[ticks_frame.index[-1],['ask','bid']]
print("BuyOpen:",open_ask,"SellOpen:",open_bid,"BuyClose:",close_ask,"SellClose:",close_bid)

In [ ]:
# sell priceのTPかLCにかかるticksを抽出する
ticks_tplc = ticks_frame.query('ask <= %f | ask >= %f' %(sell_tp,sell_lc))
#ticks_tplc = ticks_frame.query('ask <= 142.641')
print("Count of rows:",len(ticks_tplc))
ticks_tplc.sort_index(inplace=True)
ticks_tplc[0:1]

In [ ]:
ticks_tplc.loc[ticks_tplc.index[0],'ask']

In [ ]:
train_data.train_data.head(100).to_csv(TRAIN_MODEL_PATH+'TrainDataHead100.csv')

In [ ]:
import shutil
shutil.copyfile(TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Gen'+df_cur_gen_rank.iloc[0]['gen']
                +'_Agent'+df_cur_gen_rank.iloc[0]['agt_num']+'_TrainResult.csv',
                TRAIN_MODEL_PATH+'TrainResult_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.csv')
shutil.copyfile(TMP_TRAIN_MODEL_PATH+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_Gen'+df_cur_gen_rank.iloc[0]['gen']
                +'_Agent'+df_cur_gen_rank.iloc[0]['agt_num']+'_BacktestResult.csv',
                TRAIN_MODEL_PATH+'BacktestResult_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.csv')

In [ ]:
df_backtest_result

In [ ]:
df_gen_rank

In [ ]:
df_cur_gen_rank

In [ ]:
df_gen_rank.loc['33'].to_csv(TRAIN_MODEL_PATH+'df_gen_rank_test.csv')

In [ ]:
!ls

In [ ]:
from scipy import stats
stats.norm(loc=df_gen_rank.loc['11']['trade_avg'],scale=df_gen_rank.loc['11']['std']).cdf(0)

In [ ]:
df_gen_rank.query('gen == %d' %(1)).sort_values('result',ascending=False)

In [ ]:
AgentList[-1].df_trade_data_period

In [ ]:
df_rslt = pd.DataFrame(columns=['gen','par_agt','agt_num','agt_obj','sum','mean','sd'])
df_rslt = train.df_result.loc[19][['sum','mean','sd']]
df_rslt

In [ ]:
AgentList[TRAIN_AGENT_NUM-1].agnt.df_reward

In [ ]:
AgentList[TRAIN_AGENT_NUM-1].acnt.df_account

In [ ]:
df_train

In [ ]:
start_date_time = datetime.datetime(2017,1,3,15)
end_date_time = datetime.datetime(2017,1,3,20)
df_train_data_priod = train.train_data.loc[start_date_time:end_date_time]

for idx, rowdt in df_train_data_priod.iterrows():
  print(idx, rowdt)
  break

## 任意の期間の毎週月曜日のリストを取得する
- Pandas.date_rangeを利用する

In [ ]:
# 今日の日付を取得する
dt_today = datetime.datetime.now()
# ISOWEEKDAYは、月曜日=1、日曜日=7となる。月～金の時は終了日を先週の土曜日の00:00:00にする
# 土日の場合は、終了日を今週の土曜日の00:00:00とする
if dt_today.isoweekday() <= 5:
    end_day = datetime.datetime.fromisocalendar(dt_today.year,dt_today.isocalendar().week-1 ,6)
else:
    end_day = datetime.datetime.fromisocalendar(dt_today.year,dt_today.isocalendar().week ,6)

# 開始日はend_day の一年前
start_day = end_day - datetime.timedelta(weeks=52)

print('start_day=%s, end_day=%s' %(start_day,end_day))

# 開始日以降の毎週月曜日のリストを取得する
every_monday = pd.date_range(start_day, end_day, freq='W-MON')
for d in every_monday:
  print('%s, type:%s' %(d, type(d)))

## CheckTpLc for minutes
試験コード

In [ ]:
# TP/LCの試験。AccountのCheckTpLc
start_date = datetime.datetime(2018,1,3,15)
delta_time = datetime.timedelta(hours=1)
pos_open_price = 1.2122
has_position = -1.0
CHECK_PERIOD = 'M1'
TRADE_SYMBOL = 'EURUSD'

tp_level = -1.0
lc_level = -1.0

margin = MARGIN_DICT[TRADE_SYMBOL]
magnification = EnvironmentCommon.GetDigitMagnification(TRADE_SYMBOL)

# テスト対象となる期間の1分足データを取得する
df_test_price_data = periods_dict[CHECK_PERIOD][TRADE_SYMBOL].price_list.loc[start_date:(start_date + delta_time)]
# 時間足を一つずつ取り出してTP/LCのチェック
for prc in df_test_price_data.itertuples():
  logger.debug("Account.CheckTpLc:PriceData[%s, %s, %s]" %(prc.Index,prc.high,prc.low))

  # 通貨ペアによらず、1,000pips=1.000となるように調整する
  # スプレッド分をマージンとして差し引く
  if(has_position == 1.0):
    # Long positionを持っている場合
    max_profit = (prc.high - pos_open_price) * magnification - margin
    max_loss = (prc.low - pos_open_price) * magnification - margin
  elif(has_position == -1.0):
    # Short positionを持っている場合
    max_profit = (pos_open_price - prc.low) * magnification - margin
    max_loss = (pos_open_price - prc.high) * magnification - margin
  else:
    # ここに入ったらエラー
    sys.exit("Account.CheckTpLc:Position opened but no flags.")

  logger.debug("Account.CheckTpLc:max_profit=%.3f, max_loss=%.3f" %(max_profit, max_loss))

  # 1つの時間足でTPとLCに同時に到達している場合は、TPとLCをランダムに決める
  if(max_profit >= tp_level and max_loss <= -lc_level):
    if(bool(random.getrandbits(1))):
      # TPの実行
      logger.debug('Account.CheckTpLc:Take profit was executed.(1)')
      # 実際はメソッドを呼び出す
      break
    else:
      # LCの実行（実際やメソッドを呼び出す）
      logger.debug('Account.CheckTpLc:Loss cut was executed.(1)')
      # 実際はメソッドを呼び出す
      break
  elif(max_profit >= tp_level):
    # TPにかかった時には、直近のAccountに利益を追加して保有ポジションを解除する
    logger.debug('Account.CheckTpLc:Take profit was executed.(2)')
    # 実際にはメソッドを呼び出す
    break
  elif(max_loss <= -lc_level):
    # LCにかかった時には、直近のAccountに利益を追加して保有ポジションを解除する
    logger.debug('Account.CheckTpLc:Loss cut was executed.(2)')
    # 実際にはメソッドを呼び出す
    break
  else:
    # どちらにも引っかからない場合は、何もしない
    logger.debug('Account.CheckTpLc:Neither Take profit nor Loss cut was executed.(2)')
    pass

テストコード

In [ ]:
import random, math
for i in range(10):
  logger.debug(random.choices([random.randrange(TP_LC_MIN_LEVEL,TP_LC_MAX_LEVEL + 1, TP_LC_STEP),0],\
                              weights=[math.floor((math.floor(TP_LC_MAX_LEVEL-TP_LC_MIN_LEVEL)+1)/TP_LC_STEP),1], k=2))
  [tp,lc]=random.choices([random.randrange(TP_LC_MIN_LEVEL,TP_LC_MAX_LEVEL + 1, TP_LC_STEP),0],\
                              weights=[math.floor((math.floor(TP_LC_MAX_LEVEL-TP_LC_MIN_LEVEL)+1)/TP_LC_STEP),1], k=2)
  tp/=1000
  lc/=1000
  logger.debug('tp:%.3f, lc:%.3f' %(tp,lc))

In [ ]:
print(torch.tensor([GAMMA]))
print(agnt.df_reward.loc[test_date_time]['q_max'][0][0])

print(GAMMA * agnt.df_reward.loc[test_date_time]['q_max'][0][0])
print((GAMMA * agnt.df_reward.loc[test_date_time]['q_max'][0][0]).detach())

In [ ]:
agnt.df_reward.loc[test_date_time]['exp_s_a'] = agnt.df_reward.loc[test_date_time]['reward'] + GAMMA * agnt.df_reward.loc[test_date_time]['q_max'][0][0]
print(agnt.df_reward.loc[test_date_time]['exp_s_a'])

In [ ]:
ds_reward_test_temp = pd.DataFrame(columns=['symbol','period','q_max','a_max','q_taken','a_taken'],\
                                  index=[test_date_time],\
                                  data=[[agnt.symbol, agnt.period, agnt.state_action_values[0][agnt.max_action_index.item()], agnt.max_action_index
                                        , agnt.state_action_values[0][agnt.acition_index.item()],agnt.acition_index]])

In [ ]:
agnt.df_reward.loc[test_date_time-delta_time,['q_max']]=[[agnt.state_action_values[0][agnt.max_action_index.item()]]]

In [ ]:
df_reward_test_temp = pd.DataFrame(columns=['q_max'],\
                                  index=[test_date_time-delta_time],\
                                  data=[[agnt.state_action_values[0][agnt.max_action_index.item()]]])

In [ ]:
df_reward_test = pd.DataFrame(columns=['q_max'],\
                                  index=[test_date_time],\
                                  data=[[torch.zeros(1,requires_grad=True)]])

In [ ]:
df_reward_test.loc[test_date_time-delta_time,['q_max']]=agnt.state_action_values[0][agnt.max_action_index.item()]

In [ ]:
df_reward_test

In [ ]:
df_reward_test_temp

In [ ]:
df_reward_test.append(df_reward_test_temp)

In [ ]:
logger.debug('Agent \n-----\n%s\n------'%(agnt.df_reward.loc[test_date_time]))

In [ ]:
 prices_dict['USDZAR'].static_data.to_csv('./drive/My Drive/Colab Notebooks/02_DRLTradingSystem2020/StaticData_Result_USDZAR_2017_H1.csv')


In [ ]:
train.train_data.to_csv('./drive/My Drive/Colab Notebooks/02_DRLTradingSystem2020/tmp/StaticData_Result_2017_2020_H1_'+datetime.datetime.now().strftime("%Y%m%d%H%M%S")+'.csv')


In [ ]:
prices_dict[TRADE_SYMBOL].GetPriceData(last_date_time).at[0,0]

In [ ]:
prices_dict[TRADE_SYMBOL].GetPriceData(last_date_time).loc[last_date_time,'close']

In [ ]:
print(df_acnt_train)

In [ ]:
now()

In [ ]:
prices.price_list.to_csv('./drive/My Drive/Colab Notebooks/02_DRLTradingSystem2020/tmp/price_list.csv')

In [ ]:
 prices_dict['USDJPY'].static_data.to_csv('./drive/My Drive/Colab Notebooks/02_DRLTradingSystem2020/tmp/static_data_USDJPY.csv')

In [ ]:
train.train_data.to_csv('./drive/My Drive/Colab Notebooks/02_DRLTradingSystem2020/tmp/train_data.csv')

In [ ]:
SMA_Short = prices.price_list.rolling(13).mean()['<CLOSE>']
SMA_long = prices.price_list.rolling(31).mean()['<CLOSE>']
SMA_Short - SMA_long

In [ ]:
train.price_data_list[(train.price_data_list.index.year == 2017) & (train.price_data_list.index.isocalendar().week == 31)]

In [ ]:
import datetime
slice_adte = datetime.datetime(2019,12,20,21,0,0)
cls = prices.price_list[prices.price_list.index == slice_adte]
print(type(cls))

In [ ]:
prices.GetPriceData(slice_adte)

In [ ]:
test_df = prices_dict[TRADE_SYMBOL].GetPriceData(test_date_time)
(close,open) = test_df.loc[test_date_time,['close','open']]
open

In [ ]:
act.df_account

In [ ]:
import pandas as pd
import numpy as np

df_simple = pd.DataFrame(np.arange(12).reshape(3, 4))

In [ ]:
df_simple.loc['bill']=df_simple.loc['alis']
df_simple

In [ ]:
for row in df_test_price_data.itertuples():
  print(row.Index, row.high, row.low)

In [ ]:
test_date_time = datetime.datetime(2017,1,3,15)
lstprc = periods_dict[TRADE_PERIOD][TRADE_SYMBOL].GetPriceData(test_date_time)
last_close_price = lstprc.loc[test_date_time,'close']
last_high_price = lstprc.loc[test_date_time,'high']
last_low_price = lstprc.loc[test_date_time,'low']
print(lstprc)
print(last_close_price, last_high_price, last_low_price)

## Sample:Pands.DataFrameに格納したtensorを取り出して結合する方法

In [ ]:
import torch
reward = torch.zeros(1,1, requires_grad=True)
close_pl = 0.5
float_pl = 0.25
delta = 0.1
reward = reward + close_pl + delta * float_pl
reward

In [ ]:
import pandas as pd
df_rwd = pd.DataFrame(columns=['reward'],\
                              data=[[torch.zeros(1,1,requires_grad=True)],[torch.zeros(1,1,requires_grad=True)],[torch.zeros(1,1,requires_grad=True)],[torch.zeros(1,1,requires_grad=True)]])
df_rwd.reward = df_rwd.reward + close_pl + delta * float_pl
df_rwd.loc[2]['reward'] = df_rwd.iloc[2].reward + close_pl + delta * float_pl
print(df_rwd.reward)
tns_rwd = torch.cat([s for s in df_rwd.reward])
print(tns_rwd)

In [ ]:
[s for s in df_rwd.reward]

## ファイルの入出力

In [ ]:
TRADE_SYMBOL = 'USDJPY'
TRADE_PERIOD = 'D1'
FILE_PATH = r'./drive/MyDrive/Colab Notebooks/02_DRLTradingSystem2020/Model/Train/Tmp/'
now_date_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
FILE_NAME=TRAIN_MODEL_PATH+'TestAccount_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.ini'
S_FILE_NAME = FILE_NAME.split('_')
print(S_FILE_NAME)

In [ ]:
TRADE_SYMBOL = 'USDJPY'
TRADE_PERIOD = 'D1'
FILE_PATH = r'./drive/MyDrive/Colab Notebooks/02_DRLTradingSystem2020/Model/Train/Tmp/'
now_date_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
FILE_NAME=TRAIN_MODEL_PATH+'TestAccount_'+TRADE_SYMBOL+'_'+TRADE_PERIOD+'_'+now_date_time+'.ini'

losscut_level = 1000
takeprofit_level = 300
with open(FILE_PATH + FILE_NAME , mode='w') as f:
    f.write('[COMMOM]\n')
    f.write('LOSSCUT_LEVEL = ' + str(losscut_level) + '\n')
    f.write('TAKEPROFIT_LEVEL = ' + str(takeprofit_level) + '\n')

## 開始日と終了日

In [ ]:
import datetime as dt
test_date_time = dt.datetime(2023,3,5,15)
delta_period = dt.timedelta(hours=1)

# 日単位の訓練期間
print('test_date_time(Day): ' + str(test_date_time))
start_period = (test_date_time
                + dt.timedelta(days=(0 if test_date_time.isoweekday() < 6 else 8 - test_date_time.isoweekday()))).replace(hour=0,minute=0)

print('start_period(Day): ' + str(start_period))
end_period = (start_period
              + dt.timedelta(days=1)).replace(hour=0,minute=0)
end_period -= delta_period
end_trade = end_period - delta_period
print('end_period(Day): ' + str(end_period) + '\n'\
      + 'end_trade(Day): ' + str(end_trade))

# 年単位の訓練期間
# 開始日はその日(self.ini_datetime)の属する年の第2週の月曜日0:00
start_period = dt.datetime.fromisocalendar(test_date_time.year, 2, 1)
# トレードの終了は、その年の51週目の土曜日0:00の2つ前の時間足
end_period = dt.datetime.fromisocalendar(test_date_time.year, 51, 6)
end_period -= delta_period
end_trade = end_period - delta_period
print('test_date_time(Year): ' + str(test_date_time))
print('start_period(Year): ' + str(start_period))
print('end_period(Year): ' + str(end_period) + '\n'\
      + 'end_trade(Year): ' + str(end_trade))


In [ ]:
TrainerParameterList.clear()
TrainerParameterList=[None]*3
TrainerParameterList

In [ ]:
from collections import namedtuple

TradeParameters = namedtuple('TrainParameters',['Account','Agent','Brain'])
TrainParametersList = [TradeParameters(None,None,None)]*3
TrainParametersList

## Pytorch.saveで、ランタイムエラーが出た時の対処方法
- ローカルランタイムを使って、ローカルフォルダに.pt(.pth)ファイルを保存しようとすると"does not exist."や"cannot be opened."というエラーが発生する
 - "_use_new_zipfile_serialization=False"というパラメータを渡す

In [ ]:
try:
  torch.save(AgentList[0].agnt.brain.state_dict(),TrainParametersList[0].brain)
except RuntimeError as e:
  print(e)
  torch.save(AgentList[0].agnt.brain.state_dict(),TrainParametersList[0].brain,_use_new_zipfile_serialization=False)
print('Model saved successfilly.')

## LINE Notifyのテスト

In [ ]:
!pip install requests

In [ ]:
import requests

def send_line_notify(notification_message):
    """
    LINEに通知する
    """
    line_notify_token = '8aWFL1Ka3by4uSF29cywkElNB5qjy9rb0gPwv2tLKvd'
    line_notify_api = LINE_NOTIFY_TOKEN # setting.iniから取得する
    headers = {'Authorization': f'Bearer {line_notify_token}'}
    data = {'message': f'message: {notification_message}'}
    requests.post(line_notify_api, headers = headers, data = data)

if __name__ == "__main__":
    msg = 'This Massage is test.'
    send_line_notify(msg)
